A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
classdef QuasiGaussian < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        smoothAlpha;
        numOfFactor;
        lambda = {};
        slope = {};
        curvat = {};
        theta = [];
        eta = [];
        tau = [];
        kappa = [];
        corr_fwd = [];
        rho = [];
        inv_H_tau = [];
        swaption_calibration_yn=[];
    end
    
    methods
        function ch = QuasiGaussian(params)
            if nargin > 0
                ch.mktData =  params.mktData;
                ch.modelParams = params.modelParams;
                
                ch.zeroCurve = params.zeroCurve;
                
                ch.smoothAlpha =  params.modelParams('smoothAlpha');
                
                ch.numOfFactor = params.modelParams('numOfFactor');
                sizeN = ch.numOfFactor;
                ch.lambda = cell(sizeN,1);
                ch.lambda = params.modelParams('lambda');
                
                ch.slope = cell(sizeN,1);
                ch.slope = params.modelParams('slope');
                
                ch.curvat = cell(sizeN,1);
                ch.curvat = params.modelParams('curvat');
                
                ch.theta = params.modelParams('theta');
                
                ch.eta = params.modelParams('eta');
                
                ch.kappa = ones(sizeN,1);
                ch.kappa =  params.modelParams('kappa');
                
                
                
                ch.tau = ones(sizeN,1);
                ch.tau =  params.modelParams('tau');
                
                ch.inv_H_tau = ones(sizeN,sizeN);
                H_tau = ones(sizeN,sizeN);
                for i=1:sizeN
                    for j=1:sizeN
                        H_tau(i,j) = exp(-ch.kappa(j)* ch.tau(i));
                    end
                end
                ch.inv_H_tau = inv(H_tau);
                
                ch.corr_fwd = zeros(sizeN,sizeN);
                ch.corr_fwd =  params.modelParams('corr_fwd');
                
                ch.rho = zeros(sizeN,sizeN); 
                [PC,V]=eig(ch.corr_fwd);
                PCW= PC*sqrt(V);
                for i=1:sizeN
                    for j=1:sizeN
                        ch.rho(i,j)=PCW(i,j);
                    end
                end
                
                ch.swaption_calibration_yn = params.modelParams('swaptionCalibrationYN');
           
                
            end
        end
        
        function out = SimpleSRPrOverHedge(ch,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer))
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer))
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(ch,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(ch,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(ch,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = ch.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = ch.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = ch.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  CMS_PSA_SimpleDate(ch,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = ch.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = ch.DF(to);
           df_TN = ch.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = ch.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = ch.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = ch.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = ch.LocalVariance(0,to);
           
           sizeN = ch.numOfFactor;
            
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:sizeN
               for j=1:sizeN
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm;
               for j=1:sizeN
                cms_PCA_SimpleOut(i) = cms_PCA_SimpleOut(i) + c_nN(j)*modelStatesT(j,i);
               end
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = CheyetteGeneralVariance(ch,from,to,T1,T2)
            
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            
            sigma_f = zeros(sizeN,sizeN);
            
            % eta is short rate volatlity
            % eta2 is short rate covariance matrix

            for i=1:sizeN
                for j=1:sizeN
                    sigma_f(i,j)=sqrt(plambda{i,1}.quote(1)*plambda{i,1}.quote(1))*ch.rho(i,j);
                end
            end
            
           eta = zeros(sizeN,sizeN);
           eta = ch.inv_H_tau*sigma_f;
           eta2 = eta*eta';
 
           sigmaTimes = plambda{1,1}.tenor;
           sigmaSize = length(plambda{1,1}.tenor);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            lastExpScale = zeros(4,sizeN,sizeN);
            newExpScale  = zeros(4,sizeN,sizeN);
            variances = zeros(4,sizeN,sizeN);
            meanR = zeros(sizeN,1);
            
            for i=1:sizeN
                meanR(i) = ch.kappa(i);
            end
%             newExpScale = zeros(4,1);
%             lastExpScale = zeros(4,1);
%             variances = zeros(4,1);
            
            lastU  = from;
            
            for j=1:sizeN
                for k=1:sizeN
                    lastExpScale(2,j,k) = exp(meanR(j)*lastU);
                    lastExpScale(3,j,k) = exp(meanR(k)*lastU);
                    lastExpScale(4,j,k) = exp((meanR(j)+meanR(k))*lastU);
                end
            end
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) tindex = i;else tindex = sigmaSize;end;
            % eta is short rate volatlity
                for j=1:sizeN
                    for k=1:sizeN
                        sigma_f(j,k)=sqrt(plambda{j,1}.quote(tindex)*plambda{j,1}.quote(tindex))*ch.rho(j,k);
                    end
                end
                
                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';
                
                for j=1:sizeN
                    for k=1:sizeN
                        newExpScale(2,j,k) = exp(meanR(j)*U);
                        newExpScale(3,j,k) = exp(meanR(k)*U);
                        newExpScale(4,j,k) = exp((meanR(j)+meanR(k))*U);
                    
                        variances(1,j,k) = variances(1,j,k) + eta2(j,k) * (U - lastU);
                        variances(2,j,k) = variances(2,j,k) + eta2(j,k) * (1.0/ meanR(j))*(newExpScale(2,j,k) - lastExpScale(2,j,k));
                        variances(3,j,k) = variances(3,j,k) + eta2(j,k) * (1.0/ meanR(k))*(newExpScale(3,j,k) - lastExpScale(3,j,k));
                        variances(4,j,k) = variances(4,j,k) + eta2(j,k) * (1.0/ (meanR(j)+meanR(k)))*(newExpScale(4,j,k) - lastExpScale(4,j,k));
                    end
                end
                
                lastU = U;
                for j=1:sizeN
                    for k=1:sizeN
                        lastExpScale(2,j,k) = newExpScale(2,j,k);
                        lastExpScale(3,j,k) = newExpScale(3,j,k);
                        lastExpScale(4,j,k) = newExpScale(4,j,k);
                    end
                end
                
            end
            
            value = 0;
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + (1.0/ meanR(j))*(1.0/meanR(k))*variances(1,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + -1.0*exp(-1.0*meanR(j)*T1)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(2,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + -1.0*exp(-1.0*meanR(k)*T2)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(3,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value +  1.0*exp(-1.0*meanR(j)*T1)*exp(-1.0*meanR(k)*T2)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(4,j,k);
                end
            end
            
            out  = value;
        end
        
        function out = fwdZcVariance(ch, from, to, T1, T2)
            
            localVariance = zeros(2,1);
            
            localVariance(1) = ch.CheyetteGeneralVariance(from,to,T1,T1);
            localVariance(2) = ch.CheyetteGeneralVariance(from,to,to,to);
            
            
            value = localVariance(1) - localVariance(2);
            
            measureChange = zeros(2,1);
            
            measureChange(1) = ch.CheyetteGeneralVariance(from,to,T1,T2);
            measureChange(2) = ch.CheyetteGeneralVariance(from,to,to,T2);
            
            measureChangeTerm = measureChange(1) - measureChange(2);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactor(ch, startTime, endTime,numMatTime,modelStatesT)
           zcT = ch.DF(endTime/365.0);
           zct = ch.DF(startTime/365.0);
           beta = ch.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ch.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           sizeN = ch.numOfFactor;
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm);
               for j=1:sizeN
                    discountFactorOut(i) = discountFactorOut(i) * exp(-beta(j)*modelStatesT(j,i));
               end
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoff(ch,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ch.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ch.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(ch,fromTime, toTime)
            sizeN = ch.numOfFactor;
            drift = zeros(sizeN,sizeN);
            for i=1:sizeN
                drift(i,i) =  exp(-1.0*ch.kappa(i)*(toTime-fromTime)/365.0);
            end
            out = drift;
        end
        
        function out = stateLocalVariance(ch,fromTime, toTime)
            out = ch.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(ch,from,to)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f =[];
            sigma_f = zeros(sizeN,sizeN);
%% converting forward rate volatlity into short rate volatlitity
%% eta is short rate volatlity
            for i=1:sizeN
                for j=1:sizeN
%                     sigma_f(i,j)=plambda{i,1}.quote(1)*ch.Fwd(0,ch.tau(i))*ch.rho(i,j);
%                     sigma_f(i,j)=plambda{i,1}.quote(1)*ch.rho(i,j);
                    sigma_f(i,j)=sqrt(plambda{i,1}.quote(1)*plambda{i,1}.quote(1))*ch.rho(i,j);
                end
            end
            
           eta =[];
           eta = zeros(sizeN,sizeN);
           eta = ch.inv_H_tau*sigma_f;
           eta2 = eta*eta';
           %%
           alphaSize = length(plambda{1,1}.tenor);
            
           for i=1:alphaSize
               if from<= plambda{1,1}.tenor(i);
                   break;
               end
           end
            
           startIdx = i;
            
%             endIdx = size(sig(1).tenor,1); 
           for i=1:alphaSize
               if to<= plambda{1,1}.tenor(i);
                   break;
               end
           end
           endIdx = i;
            
           lastU  = from;
           
           tiny = zeros(sizeN,sizeN);
           lastExpScale = zeros(sizeN,sizeN);
           newExpScale  = zeros(sizeN,sizeN);
           scale = 0.0;
           meanR = zeros(sizeN,1);
           for i=1:sizeN
                meanR(i) = ch.kappa(i);
           end
           for i=1:sizeN
                for j=1:sizeN
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end;
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
           end
            
           variances = zeros(sizeN,sizeN);
           tindex = 1;
           for i=startIdx:endIdx
                if (i< endIdx) U = plambda{1,1}.tenor(i);else U = to;end;
                if (i< alphaSize) tindex = i;else tindex = alphaSize;end;
            %% converting forward rate volatlity into short rate volatlitity
            %% eta is short rate volatlity
                for j=1:sizeN
                    for k=1:sizeN
%                         sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.Fwd(0,ch.tau(j))*ch.rho(j,k);
%                         sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                        sigma_f(j,k)=sqrt(plambda{j,1}.quote(tindex)*plambda{j,1}.quote(tindex))*ch.rho(j,k);
                    end
                end
                
                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';
            %%
                for j=1:sizeN
                    for k=1:sizeN
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale = newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + eta2(j,k) * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
           for j=1:sizeN
                for k=1:sizeN
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
           end
            
            
        end

        function beta = Beta(ch,t,T)
            beta = [];
            sizeN = ch.numOfFactor;
            beta = zeros(sizeN,1);
            
            for i=1:sizeN
                if(abs(ch.kappa(i)) > 1.0E-12)
                    beta(i) = (1.0-exp(-ch.kappa(i)*(T-t)))/ch.kappa(i);
                else
                    beta(i) = (T-t);
                end
            end
        end
        
        function beta = BetaDate(ch,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            sizeN = ch.numOfFactor;
            beta = zeros(sizeN,1);
            
            for i=1:sizeN
                if(abs(ch.kappa(i)) > 1.0E-12)
                    beta(i) = (1.0-exp(-ch.kappa(i)*(T-t)))/ch.kappa(i);
                else
                    beta(i) = (T-t);
                end
            end
            
        end
        
        function c_nN = C_nN(ch,tn,tN,tau,pnN,ynN)
            p = @ch.DF;
            c_nN = [];
            sizeN = ch.numOfFactor;
            c_nN = zeros(sizeN,1);
            
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ch.Beta(tn,tp);
                df = p(tp);
                for j=1:sizeN
                    c_nN(j) = c_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)* ynN;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j) + beta(j)*df;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)/pnN;
            end
           
        end
        
        function c_nN = C_nNDate(ch,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ch.DF;
            sizeN = ch.numOfFactor;
            c_nN = zeros(sizeN,1);
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ch.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                for j=1:sizeN
                    c_nN(j) = c_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)* ynN;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j) + beta(j)*df;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)/pnN;
            end
        end
        
        function g_nN = G_nN(ch,tn,tN,tau,pnN)
            p = @ch.DF;
            g_nN = [];
            sizeN = ch.numOfFactor;
            g_nN = zeros(sizeN,1);
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ch.Beta(tn,tp);
                df = p(tp);
                for j=1:sizeN
                    g_nN(j) = g_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                g_nN(j) = g_nN(j)/pnN;
            end
            
        end
        
        function g_nN = G_nNDate(ch,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ch.DF;
            sizeN = ch.numOfFactor;
            g_nN = zeros(sizeN,1);
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ch.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                for j=1:sizeN
                    g_nN(j) = g_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                g_nN(j) = g_nN(j)/pnN;
            end
        end 
        
        function h_nTerm = H_nTerm(ch,tn,tTerm)
           h_nTerm =[];
           sizeN = ch.numOfFactor;
           h_nTerm = zeros(sizeN,1);
           beta = ch.Beta(tn,tTerm);
           
           for j=1:sizeN
                h_nTerm(j) = -1.0*beta(j);
           end
          
        end
        
        function h_nTerm = H_nTermDate(ch,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           sizeN = ch.numOfFactor;
           h_nTerm = zeros(sizeN,1);
           beta = ch.BetaDate(valueDate,tnDate,tTermDate);
           for j=1:sizeN
                h_nTerm(j) = -1.0*beta(j);
           end
        end
        
        
        
        function out = CMSDigitalArg(ch,to,tp,tenor1,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           p_nN1 = ch.PV01(to,to+tenor1,tau);
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ch.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ch.H_nTerm(to,tp);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1);
           if spreadVariance >=0
            out.sigma = sqrt(spreadVariance);
           else
            out.sigma = 0.0;
           end
           
           out.c_nN1 = c_nN1;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSDigital(ch,to,tp,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ch.CMSDigitalArg(to,tp,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(ch,to,tp,tenor1,tau,lower,upper)
            lower_value = ch.CMSDigital(to,tp,tenor1,tau,lower);
            upper_value = ch.CMSDigital(to,tp,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        
        function out = CMSSpreadDigitalArg(ch,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           p_nN1 = ch.PV01(to,to+tenor1,tau);
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ch.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ch.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = ch.PV01(to,to+tenor2,tau);
           df_Tn2 = ch.DF(to);
           df_TN2 = ch.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = ch.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = ch.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = ch.H_nTerm(to,tp);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.corr = coVariance/sqrt(variance1*variance2);
           
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
           out.pmu = driftTerm1 - driftTerm2;
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.y1 = y_nN1;
           out.y2 = y_nN2;
           
        end
        
        function out = CMSSpreadDigitalArgDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = ch.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = ch.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = ch.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = ch.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = ch.DF(to);
           df_TN2 = ch.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = ch.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = ch.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = ch.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSSpreadDigital(ch,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ch.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ch.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(ch,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = ch.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = ch.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = ch.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = ch.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function atmSwaption = ATMSwaption(ch,tx,tnr)
            sizeN = ch.numOfFactor;
            tau = 0.25;
            pv01 = ch.PV01(tx,tx+tnr,tau);
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            c_nN = ch.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            if totalVar <=0
                totalVar = 0.0;
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function atmSwaptionNormalVol = ATMSwaptionNormalVol(ch,tx,tnr)
            sizeN = ch.numOfFactor;
            tau = 0.25;
            pv01 = ch.PV01(tx,tx+tnr,tau);
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            c_nN = ch.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            if totalVar <=0
                totalVar = 0.0;
            end
            atmSwaptionNormalVol =  sqrt(totalVar/tx);
        end
        
        function capletOut = Caplet(ch,tx,x,tnr)
        %blackCaplet:a function that computes a blackCaplet price
        %arguments
        %tx =(annualized)time to reset
        %sig= caplet volatility
        %x  = caplet strike
        %tnr=(annualized)time to payment
        %NA =Notional Amount
        
            tau = 0.25;
            p = @ch.DF;
            beta = ch.Beta(tx,tx + tnr);
            sizeN = ch.numOfFactor;
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + beta(i)*beta(j)*variances(i,j);
                end
            end
            sig = sqrt(totalVar);
            d1= (log(p(tx)/p(tx + tnr)*1.0/(1+tnr*x))+(sig^2/2.0))/(sig);
            d2 = d1 -sig;
            capletOut = p(tx)*H_ncdf(d1) - p(tx+tnr)*(1+tnr*x)*H_ncdf(d2);
        end
        
        function capOut = Cap(ch,tx,x,tnr)
            capOut = sum(arrayfun(@(i) ch.Caplet(tnr*i,x,tnr),1:round(tx/tnr)-1));
        end
        
         function atmSwaptionImpVol = ATMSwaptionImpVol(ch,tx,tnr)
            atmSwaption = ch.ATMSwaption(tx,tnr);
            
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;
            tau = 0.25;
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            pv01 = ch.PV01(tx,tx+tnr,tau);
            impVol = BisecBSIV('P',fsr,fsr, ...
                                          0.0,tx,a,b,atmSwaption/pv01,Tol,MaxIter);
            atmSwaptionImpVol =  impVol;
         end
        
         function out = findTindex(ch,t,sigma)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(sigma.tenor)
                alpha = i;
                if t<= sigma.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function instFwdCorrTest = InsFwdCorrTest(ch,tx,tnr1,tnr2)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            
            sigma_f = zeros(sizeN,sizeN);
            
%             tindex = ch.findTindex(tx+1e-6,plambda{1,1});
            tenor = ch.lambda{1,1}.tenor;
            tindex = min(find(tx<=tenor));
            
            for j=1:sizeN
                for k=1:sizeN
                    sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                end
            end
                
            eta = ch.inv_H_tau*sigma_f;
            eta2 = eta*eta';
            
            KappaTau1 = zeros(sizeN,1);
            KappaTau2 = zeros(sizeN,1);
            
            for i=1:sizeN
                KappaTau1(i) = exp(-ch.kappa(i)*tnr1);
                KappaTau2(i) = exp(-ch.kappa(i)*tnr2);
            end
            
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  KappaTau1'*eta2*KappaTau1;
            var2 =  KappaTau2'*eta2*KappaTau2;
            covar =  KappaTau1'*eta2*KappaTau2;
            
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(ch,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                sizeN = ch.numOfFactor;
                plambda =cell(sizeN,1);
                for i=1:sizeN
                    plambda{i,1}= ch.lambda{i,1};
                end

                sigma_f = zeros(sizeN,sizeN);

    %             tindex = ch.findTindex(tx+1e-6,plambda{1,1});
                tenor = ch.lambda{1,1}.tenor;
                tindex = min(find(tx<=tenor));

                for j=1:sizeN
                    for k=1:sizeN
                        sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                    end
                end

                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';


                deltaG_tT = zeros(length(newtnr),sizeN);
                
                for i=1:length(newtnr)
                    G_tT1 = ch.Beta(tx,newtnr(i)+tau);
                    G_tT2 = ch.Beta(tx,newtnr(i));
                    for j=1:sizeN
                        deltaG_tT(i,j) = (G_tT1(j) - G_tT2(j))/tau;
                    end
                end

                
                testLoadingD = deltaG_tT*eta;

                coVar =  deltaG_tT*eta*eta'*deltaG_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';

            end
            
        end
        
        function InstSwapFwdCorrTest = InsSwapFwdCorrTest(ch,tx,tnr1,tnr2)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f = zeros(sizeN,sizeN);
            tenor = ch.lambda{1,1}.tenor;
            tindex = min(find(tx<=tenor));
            
            for j=1:sizeN
                for k=1:sizeN
                    sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                end
            end
                
            eta = ch.inv_H_tau*sigma_f;
            eta2 = eta*eta';
            
            KappaTau1 = zeros(sizeN,1);
            KappaTau2 = zeros(sizeN,1);
            
            tau = 0.25;
            p_nN1 = ch.PV01(tx,tx+tnr1,tau);
            df_Tn1 = ch.DF(tx);
            df_TN1 = ch.DF(tx + tnr1);
            y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
            c_nN1 = ch.C_nN(tx,tx + tnr1,tau,p_nN1,y_nN1);
            
            p_nN2 = ch.PV01(tx,tx+tnr2,tau);
            df_Tn2 = ch.DF(tx);
            df_TN2 = ch.DF(tx + tnr2);
            y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
            c_nN2 = ch.C_nN(tx,tx + tnr2,tau,p_nN2,y_nN2);
            
            for i=1:sizeN
                KappaTau1(i) = c_nN1(i)*exp(-ch.kappa(i)*tnr1);
                KappaTau2(i) = c_nN2(i)*exp(-ch.kappa(i)*tnr2);
            end
            
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  KappaTau1'*eta2*KappaTau1;
            var2 =  KappaTau2'*eta2*KappaTau2;
            covar =  KappaTau1'*eta2*KappaTau2;
            
            InstSwapFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function spreadInfo = SpreadInfo(ch,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  ch.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impCorrel = ImpCorrel(ch,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ch.PV01(tx,tx+tnr2,tau);
            fsr_2  = ch.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ch.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impVariance = ImpVariance(ch,tx,tnr1)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function impCovar = ImpCovar(ch,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ch.PV01(tx,tx+tnr2,tau);
            fsr_2  = ch.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ch.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function out = RebonatoFwdCorr(ch,tx,tnr1,tnr2)
           rho_inf = 0.7;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function out = RebonatoFwdCorr3(ch,tx,tnr1,tnr2)
           rho_inf = 0.4;
           rho_decay = 0.15;
           rho_alpha = 0.1;
           beta = rho_decay*exp(-rho_alpha*min(tnr1-tx,tnr2-tx));
           out =  rho_inf + (1-rho_inf)*exp(-beta*abs(tnr1-tnr2));
        end
        
        function out = TargetFunctionATMSwaptionAndCapBootStrapLevMar(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            capExpiry = params.capExpiry;
            blackCapPrice = params.blackCapPrice;
            blackCapStrike = params.blackCapStrike;
            
            useCap = false;
            if expiry(idxNow) <= capExpiry(length(capExpiry))
                if capExpiry(idxNow) == expiry(idxNow)
                    useCap = true;
                end
            end
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(length(tenor)+ 1,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end
            
            if useCap
                chCapPrice = ch.Cap(capExpiry(idxNow),blackCapStrike(idxNow),0.25);
                out(length(tenor)+1) = chCapPrice/blackCapPrice(idxNow)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionAndCapGlobalLevMar(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            capExpiry = params.capExpiry;
            blackCapPrice = params.blackCapPrice;
            blackCapStrike = params.blackCapStrike;
            chCapPrice = zeros(length(capExpiry),1);
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            sizeOut = length(expiry) * length(tenor);
            sizeOut = sizeOut + length(capExpiry);
            
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
            
            for j=1:length(capExpiry)
                chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                out(length(expiry)*length(tenor) + j) = chCapPrice(j)/blackCapPrice(j) - 1.0;
            end
        end 
        
        function out = TargetFunctionATMSwaptionBootStrapLevMar(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarNormalVol(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(1,length(tenor));
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chATMSwaptionNormalVol(k) = ch.ATMSwaptionNormalVol(expiry(idxNow),tenor(k));
                out(k) = chATMSwaptionNormalVol(k)/blackSwaptionNormalVol(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarNormalVariance(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(1,length(tenor));
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chATMSwaptionNormalVol(k) = ch.ATMSwaptionNormalVol(expiry(idxNow),tenor(k));
                out(k) = (chATMSwaptionNormalVol(k)*chATMSwaptionNormalVol(k)*expiry(idxNow))/(blackSwaptionNormalVol(idxNow,k)*blackSwaptionNormalVol(idxNow,k)*expiry(idxNow))-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarAbs(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = abs(tvar(j));
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionGlobalLevMar(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end 
        
        function out = TargetFunctionATMSwaptionGlobalLevMarKappa(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            for i=1:sizeN
                ch.kappa(i) = tvar(sizeN*alphaSize + i);
            end
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end 

        function out = TargetFunctionATMSwaptionGlobal(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(1,size(tenor,2));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out = out + (chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaptionGlobalImpVol(ch,tvar,params)
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionImpVol = params.blackSwaptionImpVol;
            chSwaptionPrice = zeros(1,size(tenor,2));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chATMSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                    out = out + (chATMSwaptionImpVol(j,k)- blackSwaptionImpVol(j,k))*(chATMSwaptionImpVol(j,k)- blackSwaptionImpVol(j,k))*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaptionGlobalNormalVol(ch,tvar,params)
            
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chATMSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                    out = out + (chATMSwaptionNormalVol(j,k)/blackSwaptionNormalVol(j,k)-1.0)*(chATMSwaptionNormalVol(j,k)/blackSwaptionNormalVol(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = CalibrateToATMSwaption(ch,black,calibrationFlag)
            tic;
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
               for j=1:alphaSize
                   out.lambda{i,1} = ch.lambda{i,1};
               end
            end
            
          % Calculate Black ATM Swaption Prices
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            blackSwaptionImpVol = zeros(expirySize,tenorSize);
            blackSwaptionVega = zeros(expirySize,tenorSize);
            blackSwaptionFsr = zeros(expirySize,tenorSize);
%             blackSwaptionNormalVol = zeros(expirySize,tenorSize);
            
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j)  = black.BlackATMSwaption(expiry(k),tenor(j));
                    blackSwaptionImpVol(k,j) = black.swaptionVol(expiry(k),tenor(j));
                    blackSwaptionFsr(k,j) = black.Fsr(expiry(k),expiry(k) + tenor(j),0.25);
                    blackSwaptionVega(k,j) = black.BlackSwaptionVegaWithVol(expiry(k),tenor(j),blackSwaptionFsr(k,j),blackSwaptionImpVol(k,j));
%                     blackSwaptionNormalVol(k,j) = black.ATMSwaptionNormalVol(expiry(k),tenor(j));
                end
            end
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.blackSwaptionImpVol = blackSwaptionImpVol;
            params.blackSwaptionFsr = blackSwaptionFsr;
            params.blackSwaptionVega = blackSwaptionVega;
%             params.blackSwaptionNormalVol = blackSwaptionNormalVol;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.swaption_calibration_yn = ch.swaption_calibration_yn;
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.blackSwaptionVega = blackSwaptionVega;
            out.blackSwaptionFsr = blackSwaptionFsr;
            
            out.chSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.relVolDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            out.rmseVolTotal = 0.0;
            
            out.impVol2Total = 0.0;
            out.normalVol2Total = 0.0;
            out.normalVarianceTotal = 0.0;
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            tvar = [];
            
            switch calibrationType
                case 'bootstrapLevMar'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMar(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                case 'bootstrapLevMarNormalVol'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMarNormalVol(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                    out.calibrationTime = toc;
                    
                case 'bootstrapLevMarNormalVariance'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMarNormalVariance(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.NormalVarianceDiff(j,k) = (out.chSwaptionNormalVol(j,k)*out.chSwaptionNormalVol(j,k)*expiry(j))/(out.blackSwaptionNormalVol(j,k)*out.blackSwaptionNormalVol(j,k)*expiry(j))-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            out.normalVarianceTotal = out.normalVarianceTotal + out.NormalVarianceDiff(j,k)*out.NormalVarianceDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                    out.normalVarianceTotal = sqrt(out.normalVarianceTotal/N);
                    out.calibrationTime = toc;
                    
                case 'globalLevMar'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMar(ch,tvar,params);
%                     options=optimset('Algorithm','levenberg-marquardt');
                    options=optimset('Algorithm','levenberg-marquardt','MaxFunEvals',700);
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                
                 case 'dummy'
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = ch.lambda{i,1}.quote(j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                    
                    
                case 'globalLevMarKappa'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                    for i=1:sizeN
                        tvar(sizeN*alphaSize + i) = ch.kappa(i);
                        lb(sizeN*alphaSize + i) = 0.0001;
                        ub(sizeN*alphaSize + i) = 10.0;
                    end
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMarKappa(ch,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt');
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    for i=1:sizeN
                        out.kappa(i) = x(sizeN*alphaSize+i);
                        ch.kappa(i)  = x(sizeN*alphaSize+i);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);    
                    
                case 'global'
                    optParams.params = params;
                    optParams.modelParamsSize = sizeN * alphaSize;
                    optParams.numOfEquation = params.expirySize*params.tenorSize + ...  % target cap & swaption
                                              sizeN*(alphaSize -2);   % tikhonov regularization gamma constraint
                    
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                        end
                    end
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionCheyetteATMSwaptionSurfaceGlobal',tvar,x, 500, options,'unc',ch,optParams);
                    toc

%                     targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
% %                     options=optimset('Algorithm','interior-point');
%                      options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                             ch.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                 case 'globalVegaWeight'
                    optParams.params = params;
                    optParams.modelParamsSize = sizeN * alphaSize;
                    optParams.numOfEquation = params.expirySize*params.tenorSize + ...  % target cap & swaption
                                              sizeN*(alphaSize -2);   % tikhonov regularization gamma constraint
                    
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                        end
                    end
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionCheyetteATMSwaptionSurfaceGlobalVegaWeight',tvar,x, 500, options,'unc',ch,optParams);
                    toc

%                     targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
% %                     options=optimset('Algorithm','interior-point');
%                      options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                             ch.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.relVolDiff(j,k) = (out.chSwaptionPrice(j,k) - out.blackATMSwaptionPrice(j,k))/out.blackSwaptionVega(j,k);
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.rmseVolTotal = out.rmseVolTotal + out.relVolDiff(j,k)*out.relVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.rmseVolTotal = sqrt(out.rmseVolTotal/N);
                    
                 case 'globalFmin'
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.01;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                 case 'globalImpVol'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalImpVol(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blaskSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            our.volDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blaskSwaptionImpVol(j,k);
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.vol2Total = out.vol2Total + our.volDiff(j,k)*our.volDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.vol2Total = sqrt(out.vol2Total/N);
                    
                case 'globalNormalVol'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalNormalVol(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blaskSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMSwaptionAndCap(ch,black,calibrationFlag)
            tic;
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
               for j=1:alphaSize
                   out.lambda{i,1} = ch.lambda{i,1};
               end
            end
            
          %% Calculate Black ATM Swaption Prices
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            blackSwaptionImpVol = zeros(expirySize,tenorSize);
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j)  = black.BlackATMSwaption(expiry(k),tenor(j));
                    blackSwaptionImpVol(k,j) = black.swaptionVol(expiry(k),tenor(j));
                end
            end
            
          %% Calculate Black ATM Cap Prices
           capVol = black.capVol;
           capVolData = black.mktData('capVol');
           capVolrawData =  capVolData.params('rawData');
            
           capExpiry = capVolrawData(:,1);
           capExpirySize = length(capExpiry);
            
           blackCapPrice = zeros(capExpirySize,1);
           blackCapImpVol = zeros(capExpirySize,1);
           blackCapStrike = zeros(capExpirySize,1);
           for k=1:capExpirySize
               blackCapStrike(k) = black.Fsr(0.25,capExpiry(k) -0.25,0.25);
               blackCapPrice(k) = black.BlackCap(capExpiry(k),blackCapStrike(k),0.25);
               blackCapImpVol(k) = black.capVol(capExpiry(k));
           end
          
          
            %%
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.blackSwaptionImpVol = blackSwaptionImpVol;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.capExpiry = capExpiry;
            params.blackCapPrice = blackCapPrice;
            params.blackCapStrike = blackCapStrike;
            params.blackCapImpVol = blackCapImpVol;
            params.capExpirySize = capExpirySize;
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.chSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            
            out.blackCapPrice = blackCapPrice;
            out.chCapPrice = zeros(capExpirySize,1);
            out.capRelDiff = zeros(capExpirySize,1);
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            tvar = [];
            
            switch calibrationType
                case 'bootstrapLevMar'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionAndCapBootStrapLevMar(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                    
                    for j=1:capExpirySize
                        out.chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                        out.capRelDiff(j) = out.chCapPrice(j)/out.blackCapPrice(j) -1.0;
                        out.rmseTotal = out.rmseTotal +  out.capRelDiff(j)* out.capRelDiff(j);
                        N=N+1;
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                out.calibrationTime = toc;
                
                case 'globalLevMar'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionAndCapGlobalLevMar(ch,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt');
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                    
                    for j=1:capExpirySize
                        out.chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                        out.capRelDiff(j) = out.chCapPrice(j)/out.blackCapPrice(j) -1.0;
                        out.rmseTotal = out.rmseTotal +  out.capRelDiff(j)* out.capRelDiff(j);
                        N=N+1;
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                   
                otherwise
                    disp('unImplemented')
            end
        end    
    end
    
end



In [ ]:
classdef QuantoSpreadRangeAccrual < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenorL
        tenorS
        typeL
        typeS
        lower
        upper
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
    end
    
    methods
        % constructor
        function rangeAccrual = QuantoSpreadRangeAccrual(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
%                 ragneAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                
                rangeAccrual.typeL = params.params('typeL');
                rangeAccrual.typeS = params.params('typeS');
                
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = zeros(periodCount,1);
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    to = DateDiff(observeDate, valueDate)/365.0;
                    tp = DateDiff(schedule.payDates(i), valueDate)/365.0;
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    type1 = rangeAccrual.typeL;
                    type2 = rangeAccrual.typeS;
                    
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.QuantoCMSSpreadDigitalRange(to,tp,tenor1,tenor2,type1,type2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                rangeProb(i) = rangeProb(i)/observeDays;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
       
        
    end
    
end



In [ ]:
classdef QuantoLGM2F_LGM2F < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        domestic_dH_r;
        domestic_dH_s;
        
        foreign_dH_r;
        foreign_dH_s;
        
        domestic_Alpha_r;
        domestic_Alpha_s;
        
        foreign_Alpha_r;
        foreign_Alpha_s;
        
        %market input
        corr_df;
        corr_dx;
        corr_fx;
        
        %modelParams
        
        corr_drds;
        corr_frfs;
        
        corr_drfr;
        corr_dsfr;
        
        corr_drfs;
        corr_dsfs;
        
        corr_drx;
        corr_dsx;
        
        corr_frx;
        corr_fsx;
        
        fxBlackVol;
        
        numOfFactor;
        modelType;
        freq;
    end
    
    methods
        function quantolgm2f_lgm2f = QuantoLGM2F_LGM2F(params)
            if nargin > 0

                quantolgm2f_lgm2f.zeroCurve = params.foreignModel.zeroCurve;
%                 quantolgm2f_lgm2f.freq = params.foreignModel.freq;
                quantolgm2f_lgm2f.freq = 4;
                
                quantolgm2f_lgm2f.numOfFactor = 2;
                quantolgm2f_lgm2f.modelType = 'quantolgm2f_lgm2f';
                
                
                if params.fxVolCalibrated == true
                    % we do not calibrate fxBlackVol
                    quantolgm2f_lgm2f.corr_df = params.corr_df;
                    quantolgm2f_lgm2f.corr_dx = params.corr_dx;
                    quantolgm2f_lgm2f.corr_fx = params.corr_fx;
                    
                    
                    
                    out = quantolgm2f_lgm2f.synchronizeModelParams(params);
                    
                    quantolgm2f_lgm2f.corr_drds.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.corr_drds.quote = out.corr_drds_Values;
                    
                    quantolgm2f_lgm2f.corr_frfs.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.corr_frfs.quote = out.corr_frfs_Values;
                    
                    quantolgm2f_lgm2f.corr_drfr.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.corr_drfr.quote = out.corr_drfr_Values;
                    
                    quantolgm2f_lgm2f.corr_dsfr.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.corr_dsfr.quote = out.corr_dsfr_Values;
                    
                    quantolgm2f_lgm2f.corr_drfs.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.corr_drfs.quote = out.corr_drfs_Values;
                    
                    quantolgm2f_lgm2f.corr_dsfs.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.corr_dsfs.quote = out.corr_dsfs_Values;
                    
                    quantolgm2f_lgm2f.corr_drx.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.corr_drx.quote = out.corr_drx_Values;
                    
                    quantolgm2f_lgm2f.corr_dsx.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.corr_dsx.quote = out.corr_dsx_Values;
                    
                    quantolgm2f_lgm2f.corr_frx.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.corr_frx.quote = out.corr_frx_Values;
                    
                    quantolgm2f_lgm2f.corr_fsx.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.corr_fsx.quote = out.corr_fsx_Values;
                    
                    quantolgm2f_lgm2f.domestic_Alpha_r.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.domestic_Alpha_r.quote = out.domestic_Alpha_r_Values;
                    
                    quantolgm2f_lgm2f.domestic_Alpha_s.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.domestic_Alpha_s.quote = out.domestic_Alpha_s_Values;

                    quantolgm2f_lgm2f.foreign_Alpha_r.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.foreign_Alpha_r.quote = out.foreign_Alpha_r_Values;
                    
                    quantolgm2f_lgm2f.foreign_Alpha_s.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.foreign_Alpha_s.quote = out.foreign_Alpha_s_Values;

                    quantolgm2f_lgm2f.domestic_dH_r.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.domestic_dH_r.quote = out.domestic_dH_r_Values;

                    quantolgm2f_lgm2f.domestic_dH_s.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.domestic_dH_s.quote = out.domestic_dH_s_Values;
                    
                    quantolgm2f_lgm2f.foreign_dH_r.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.foreign_dH_r.quote = out.foreign_dH_r_Values;
                    
                    quantolgm2f_lgm2f.foreign_dH_s.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.foreign_dH_s.quote = out.foreign_dH_s_Values;
                    
                    quantolgm2f_lgm2f.fxBlackVol.tenor = out.sigmaTimes;
                    quantolgm2f_lgm2f.fxBlackVol.quote = out.fxBlackVolValues;
                    
                else
                    
                    % black vol calibration start
                    paramsIn.fxImpVol = params.fxBlackVol;
                    paramsIn.fxBlackVol = paramsIn.fxImpVol;
                    paramsIn.domestic_dH_r = params.domesticModel.dH1;
                    paramsIn.domestic_dH_s = params.domesticModel.dH2;
                                        
                    paramsIn.domestic_Alpha_r = params.domesticModel.Alpha1;
                    paramsIn.domestic_Alpha_s = params.domesticModel.Alpha2;
                    
                    paramsIn.foreign_dH_r = params.foreignModel.dH1;
                    paramsIn.foreign_Alpha_r = params.foreignModel.Alpha1;
                    
                    paramsIn.foreign_dH_s = params.foreignModel.dH2;
                    paramsIn.foreign_Alpha_s = params.foreignModel.Alpha2;
                    
                    paramsIn.corr_df = params.corr_df;
                    paramsIn.corr_dx = params.corr_dx;
                    paramsIn.corr_fx = params.corr_fx;
                    
                    paramsIn.corr_drds = params.corr_drds;
                    paramsIn.corr_frfs = params.corr_frfs;
                    
                    paramsIn.corr_drfr = params.corr_drfr;
                    paramsIn.corr_dsfr = params.corr_dsfr;
                    
                    paramsIn.corr_drfs = params.corr_drfs;
                    paramsIn.corr_dsfs = params.corr_dsfs;
                    
                    paramsIn.corr_drx = params.corr_drx;
                    paramsIn.corr_dsx = params.corr_dsx;
                    
                    paramsIn.corr_frx = params.corr_frx;
                    paramsIn.corr_fsx = params.corr_fsx;
                    
                    % BSLGM2FLGM2F Calibration Start
                    out = quantolgm2f_lgm2f.CalibrateBSLGM2FLGM2FVol(paramsIn);
                    re =  out.re;
                    rmseTotal = out.rmseTotal;
                    
                    % BSLGM2FLGM2F Calibration End
                    
                    params.fxBlackVol = out.fxBlackVol;
                    params.fxVolCalibrated =  true;
                    quantolgm2f_lgm2f = QuantoLGM2F_LGM2F(params); 
                    
                end
                %end
                
                
            end
        end
        
        function out =  synchronizeModelParams(quantolgm2f_lgm2f,params)
            fxBlackVolTenor = params.fxBlackVol.tenor;
            fxBlackVolQuote = params.fxBlackVol.quote;

            % synchronize model parameters time-steps start
            corr_drds_Tenor = params.corr_drds.tenor;
            corr_drds_Quote = params.corr_drds.quote;
            
            corr_frfs_Tenor = params.corr_frfs.tenor;
            corr_frfs_Quote = params.corr_frfs.quote;
            
            corr_drfr_Tenor = params.corr_drfr.tenor;
            corr_drfr_Quote = params.corr_drfr.quote;
            
            corr_dsfr_Tenor = params.corr_dsfr.tenor;
            corr_dsfr_Quote = params.corr_dsfr.quote;
            
            corr_drfs_Tenor = params.corr_drfs.tenor;
            corr_drfs_Quote = params.corr_drfs.quote;
            
            corr_dsfs_Tenor = params.corr_dsfs.tenor;
            corr_dsfs_Quote = params.corr_dsfs.quote;
            
            corr_drx_Tenor = params.corr_drx.tenor;
            corr_drx_Quote = params.corr_drx.quote;
            
            corr_dsx_Tenor = params.corr_dsx.tenor;
            corr_dsx_Quote = params.corr_dsx.quote;
            
            corr_frx_Tenor = params.corr_frx.tenor;
            corr_frx_Quote = params.corr_frx.quote;
            
            corr_fsx_Tenor = params.corr_fsx.tenor;
            corr_fsx_Quote = params.corr_fsx.quote;
            
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;

            foreign_Alpha_r_Tenor = params.foreignModel.Alpha1.tenor;
            foreign_Alpha_r_Quote = params.foreignModel.Alpha1.quote;
            
            foreign_Alpha_s_Tenor = params.foreignModel.Alpha2.tenor;
            foreign_Alpha_s_Quote = params.foreignModel.Alpha2.quote;
            
            foreign_dH_r_Tenor = params.foreignModel.dH1.tenor;
            foreign_dH_r_Quote = params.foreignModel.dH1.quote;
            
            foreign_dH_s_Tenor = params.foreignModel.dH2.tenor;
            foreign_dH_s_Quote = params.foreignModel.dH2.quote;
 
            sigmaTimes = unique(union([corr_drds_Tenor(:);corr_frfs_Tenor(:);corr_drfr_Tenor(:);corr_dsfr_Tenor(:);...
                         corr_drfs_Tenor(:);corr_dsfs_Tenor(:);...
                         corr_drx_Tenor(:);corr_dsx_Tenor(:);corr_frx_Tenor(:);corr_fsx_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:);foreign_Alpha_r_Tenor(:);foreign_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:);domestic_dH_s_Tenor(:);foreign_dH_r_Tenor(:);foreign_dH_s_Tenor(:)],fxBlackVolTenor));


            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            corr_frfs_Values = zeros(length(sigmaTimes),1);
            
            corr_drfr_Values = zeros(length(sigmaTimes),1);
            corr_dsfr_Values = zeros(length(sigmaTimes),1);
            
            corr_drfs_Values = zeros(length(sigmaTimes),1);
            corr_dsfs_Values = zeros(length(sigmaTimes),1);
            
            corr_drx_Values = zeros(length(sigmaTimes),1);
            corr_dsx_Values = zeros(length(sigmaTimes),1);
            corr_frx_Values = zeros(length(sigmaTimes),1);
            corr_fsx_Values = zeros(length(sigmaTimes),1);

            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            foreign_Alpha_r_Values = zeros(length(sigmaTimes),1);
            foreign_dH_r_Values = zeros(length(sigmaTimes),1);
            
            foreign_Alpha_s_Values = zeros(length(sigmaTimes),1);
            foreign_dH_s_Values = zeros(length(sigmaTimes),1);
            
            fxBlackVolValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                corr_frfs_Values(k) = H_interpolation(corr_frfs_Tenor, corr_frfs_Quote,sigmaTimes(k),0);
                
                corr_drfr_Values(k) = H_interpolation(corr_drfr_Tenor, corr_drfr_Quote,sigmaTimes(k),0);
                corr_dsfr_Values(k) = H_interpolation(corr_dsfr_Tenor, corr_dsfr_Quote,sigmaTimes(k),0);
                
                corr_drfs_Values(k) = H_interpolation(corr_drfs_Tenor, corr_drfs_Quote,sigmaTimes(k),0);
                corr_dsfs_Values(k) = H_interpolation(corr_dsfs_Tenor, corr_dsfs_Quote,sigmaTimes(k),0);
                
                corr_drx_Values(k) = H_interpolation(corr_drx_Tenor, corr_drx_Quote,sigmaTimes(k),0);
                corr_dsx_Values(k) = H_interpolation(corr_dsx_Tenor, corr_dsx_Quote,sigmaTimes(k),0);
                corr_frx_Values(k) = H_interpolation(corr_frx_Tenor, corr_frx_Quote,sigmaTimes(k),0);
                corr_fsx_Values(k) = H_interpolation(corr_fsx_Tenor, corr_fsx_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_Alpha_r_Values(k) = H_interpolation(foreign_Alpha_r_Tenor, foreign_Alpha_r_Quote,sigmaTimes(k),0);
                foreign_Alpha_s_Values(k) = H_interpolation(foreign_Alpha_s_Tenor, foreign_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_dH_r_Values(k) = H_interpolation(foreign_dH_r_Tenor, foreign_dH_r_Quote,sigmaTimes(k),0);
                foreign_dH_s_Values(k) = H_interpolation(foreign_dH_s_Tenor, foreign_dH_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                fxBlackVolValues(k) = H_interpolation(fxBlackVolTenor, fxBlackVolQuote,sigmaTimes(k),0);
            end
            
            out.sigmaTimes = sigmaTimes;
            
            out.corr_drds_Values = corr_drds_Values;
            out.corr_frfs_Values = corr_frfs_Values;
            
            out.corr_drfr_Values = corr_drfr_Values;
            out.corr_dsfr_Values = corr_dsfr_Values;
            
            out.corr_drfs_Values = corr_drfs_Values;
            out.corr_dsfs_Values = corr_dsfs_Values;
            
            out.corr_drx_Values = corr_drx_Values;
            out.corr_dsx_Values = corr_dsx_Values;
            out.corr_frx_Values = corr_frx_Values;
            out.corr_fsx_Values = corr_fsx_Values;
            
            out.domestic_Alpha_r_Values = domestic_Alpha_r_Values;
            out.domestic_Alpha_s_Values = domestic_Alpha_s_Values;
            
            out.foreign_Alpha_r_Values = foreign_Alpha_r_Values;
            out.foreign_Alpha_s_Values = foreign_Alpha_s_Values;
            
            out.foreign_dH_r_Values = foreign_dH_r_Values;
            out.foreign_dH_s_Values = foreign_dH_s_Values;
            
            out.domestic_dH_r_Values = domestic_dH_r_Values;
            out.domestic_dH_s_Values = domestic_dH_s_Values;
            
            out.fxBlackVolValues = fxBlackVolValues;
            
        end
        
        function out =  Libor_SimpleDate(quantolgm2f_lgm2f,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           
           % foreign Libor in domestic Terminal measure
           
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = quantolgm2f_lgm2f.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = quantolgm2f_lgm2f.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
        function out =  CMS_SimpleDate(quantolgm2f_lgm2f,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMT
%            tau = 0.25;
           tau = 1.0/(quantolgm2f_lgm2f.freq);
            % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           
           if abs(tenor- 0.25) < 1e-8
                tau = tenor;
           end
           
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tNTime = tNDate.DateDiff(valueDate);
           tN     =  tNTime/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           addMonthUnit = round(12*tau);
           cnt = round((tN-to)/tau);
           
           stateSize = size(modelStatesT,2);
           
           annuity = zeros(1,stateSize);
           df_Tp = zeros(1,stateSize);
           
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           for i=1:cnt
                tpDate = toDate.AddDate(i*addMonthUnit,'month');
                tpTime = tpDate.DateDiff(valueDate);
                df_Tp = quantolgm2f_lgm2f.discountFactor(observeTime,tpTime,numMatTime,modelStatesT);
                annuity = annuity + tau * df_Tp;
            end
           
           df_Tn = quantolgm2f_lgm2f.discountFactor(observeTime,observeTime,numMatTime,modelStatesT);
           df_TN = quantolgm2f_lgm2f.discountFactor(observeTime,tNTime,numMatTime,modelStatesT);
           
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           for i=1:stateSize
                cms_PCA_SimpleOut(i) = (df_Tn(i) - df_TN(i))/annuity(i);
           end
           
           out = cms_PCA_SimpleOut;

            
         end
        
         function out =  CMS_PSA_SimpleDate(quantolgm2f_lgm2f,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMT
%            tau = 0.25;
           tau = 1.0/(quantolgm2f_lgm2f.freq);
            % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           
           if abs(tenor- 0.25) < 1e-8
                tau = tenor;
           end
           
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = quantolgm2f_lgm2f.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = quantolgm2f_lgm2f.DF(to);
           df_TN = quantolgm2f_lgm2f.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = quantolgm2f_lgm2f.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = quantolgm2f_lgm2f.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
%            h_nTerm = quantolgm2f_lgm2f.H_nTermDate(valueDate,toDate,numMatDate);
           
           %h_nTerm is replace by i_Term & i_Tn_tilda in the following
           
           h_nTerm(1) = 0.0;
           h_nTerm(2) = 0.0;
           
           varCovar = quantolgm2f_lgm2f.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:2
               for j=1:2
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           % additional drift correction term from foreign Term USD to
           % domestic Term measure KRW
           i_Term = I_Term(quantolgm2f_lgm2f,0,to,numMatTime/365.0);
           i_Tn_tilda = I_Tn_tilda(quantolgm2f_lgm2f,0,to,to);
           j_x_tilda = J_x_tilda(quantolgm2f_lgm2f,0,to,to);
           
           additionalDrift = 0;
           for i=1:2
                additionalDrift = additionalDrift + c_nN(i) * (i_Term(i) + i_Tn_tilda(i) + j_x_tilda(i));
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + additionalDrift + c_nN(1)*modelStatesT(1,i)+c_nN(2)*modelStatesT(2,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
         end
         
         function out = Foreign_HtT_r(quantolgm2f_lgm2f,t,T)
            out =  quantolgm2f_lgm2f.H(T,quantolgm2f_lgm2f.foreign_dH) - quantolgm2f_lgm2f.H(t,quantolgm2f_lgm2f.foreign_dH); 
         end
         
         function out = Domestic_HtT_r(quantolgm2f_lgm2f,t,T)
            out =  quantolgm2f_lgm2f.H(T,quantolgm2f_lgm2f.domestic_dH) - quantolgm2f_lgm2f.H(t,quantolgm2f_lgm2f.domestic_dH); 
         end
         
         function out = H(quantolgm2f_lgm2f,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
         end

        
        function out = fwdZcVarianceForeign(quantolgm2f_lgm2f, from, to, T1, T2)
            
            localVariance = zeros(6,1);
            measureChange = zeros(4,1);
            quantoChange = zeros(4,1);
            sigmaTimes = quantolgm2f_lgm2f.domestic_Alpha_r.tenor;
            
            foreign_Alpha_r_Values = quantolgm2f_lgm2f.foreign_Alpha_r.quote;
            foreign_Alpha_s_Values = quantolgm2f_lgm2f.foreign_Alpha_s.quote;
            
            foreign_dH_r_Values = quantolgm2f_lgm2f.foreign_dH_r.quote;
            foreign_dH_s_Values = quantolgm2f_lgm2f.foreign_dH_s.quote;
            
            domestic_Alpha_r_Values = quantolgm2f_lgm2f.domestic_Alpha_r.quote;
            domestic_Alpha_s_Values = quantolgm2f_lgm2f.domestic_Alpha_s.quote;
            
            domestic_dH_r_Values = quantolgm2f_lgm2f.domestic_dH_r.quote;
            domestic_dH_s_Values = quantolgm2f_lgm2f.domestic_dH_s.quote;
            
            fxValues = quantolgm2f_lgm2f.fxBlackVol.quote;
            
            correl_1 = ones(length(sigmaTimes),1);
            
            corr_frfs_Values = quantolgm2f_lgm2f.corr_frfs.quote; 
            
            corr_frx_Values = quantolgm2f_lgm2f.corr_frx.quote;
            corr_fsx_Values = quantolgm2f_lgm2f.corr_fsx.quote;
            
            corr_drfr_Values = quantolgm2f_lgm2f.corr_drfr.quote;
            corr_dsfr_Values = quantolgm2f_lgm2f.corr_dsfr.quote;
            
            corr_drfs_Values = quantolgm2f_lgm2f.corr_drfs.quote;
            corr_dsfs_Values = quantolgm2f_lgm2f.corr_dsfs.quote;
            
            
            localVariance(1) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,T1,T1,sigmaTimes,foreign_dH_r_Values,foreign_dH_r_Values,...
                                                                        foreign_Alpha_r_Values,foreign_Alpha_r_Values,correl_1);
                                                                    
            localVariance(2) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,to,to,sigmaTimes,foreign_dH_r_Values,foreign_dH_r_Values,...
                                                                        foreign_Alpha_r_Values,foreign_Alpha_r_Values,correl_1);
                                                                    
            localVariance(3) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,T1,T1,sigmaTimes,foreign_dH_s_Values,foreign_dH_s_Values,...
                                                                        foreign_Alpha_s_Values,foreign_Alpha_s_Values,correl_1);
                                                                    
            localVariance(4) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,to,to,sigmaTimes,foreign_dH_s_Values,foreign_dH_s_Values,...
                                                                        foreign_Alpha_s_Values,foreign_Alpha_s_Values,correl_1);
                                                                    
            localVariance(5) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,T1,T1,sigmaTimes,foreign_dH_r_Values,foreign_dH_s_Values,...
                                                                        foreign_Alpha_r_Values,foreign_Alpha_s_Values,corr_frfs_Values);
                                                                    
            localVariance(6) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,to,to,sigmaTimes,foreign_dH_r_Values,foreign_dH_s_Values,...
                                                                        foreign_Alpha_r_Values,foreign_Alpha_s_Values,corr_frfs_Values);
            
            quantoChange(1) = -1.0*quantolgm2f_lgm2f.GeneralVariance_BSLGM(from,to,T1,sigmaTimes,foreign_dH_r_Values,foreign_Alpha_r_Values,fxValues,corr_frx_Values);
            
            quantoChange(2) = -1.0*quantolgm2f_lgm2f.GeneralVariance_BSLGM(from,to,to,sigmaTimes,foreign_dH_r_Values,foreign_Alpha_r_Values,fxValues,corr_frx_Values);
            
            
            quantoChange(3) = -1.0*quantolgm2f_lgm2f.GeneralVariance_BSLGM(from,to,T1,sigmaTimes,foreign_dH_s_Values,foreign_Alpha_s_Values,fxValues,corr_fsx_Values);
            
            quantoChange(4) = -1.0*quantolgm2f_lgm2f.GeneralVariance_BSLGM(from,to,to,sigmaTimes,foreign_dH_s_Values,foreign_Alpha_s_Values,fxValues,corr_fsx_Values);
            
            
            measureChange(1) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,T1,T2,sigmaTimes,domestic_dH_r_Values,foreign_dH_r_Values,domestic_Alpha_r_Values, ...
                                                                                                    foreign_Alpha_r_Values,corr_drfr_Values);
            
            measureChange(2) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,to,T2,sigmaTimes,domestic_dH_r_Values,foreign_dH_r_Values,domestic_Alpha_r_Values, ...
                                                                                                    foreign_Alpha_r_Values,corr_drfr_Values);
                                                                                                
            measureChange(3) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,T1,T2,sigmaTimes,domestic_dH_s_Values,foreign_dH_r_Values,domestic_Alpha_s_Values, ...
                                                                                                    foreign_Alpha_r_Values,corr_dsfr_Values);
            
            measureChange(4) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,to,T2,sigmaTimes,domestic_dH_s_Values,foreign_dH_r_Values,domestic_Alpha_s_Values, ...
                                                                                                    foreign_Alpha_r_Values,corr_dsfr_Values);
            
                                                                                                
            measureChange(5) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,T1,T2,sigmaTimes,domestic_dH_r_Values,foreign_dH_s_Values,domestic_Alpha_r_Values, ...
                                                                                                    foreign_Alpha_s_Values,corr_drfs_Values);
            
            measureChange(6) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,to,T2,sigmaTimes,domestic_dH_r_Values,foreign_dH_s_Values,domestic_Alpha_r_Values, ...
                                                                                                    foreign_Alpha_s_Values,corr_drfs_Values);
                                                                                                
            measureChange(7) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,T1,T2,sigmaTimes,domestic_dH_s_Values,foreign_dH_s_Values,domestic_Alpha_s_Values, ...
                                                                                                    foreign_Alpha_s_Values,corr_dsfs_Values);
            
            measureChange(8) = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(from,to,to,T2,sigmaTimes,domestic_dH_s_Values,foreign_dH_s_Values,domestic_Alpha_s_Values, ...
                                                                                                    foreign_Alpha_s_Values,corr_dsfs_Values);                                                                                    
            
            value = localVariance(1) - localVariance(2);
            value = value +    (localVariance(3) - localVariance(4));
            value = value +2.0*(localVariance(5) - localVariance(6));
            
            value = value -2.0 * -1.0 * (quantoChange(1) - quantoChange(2));
            value = value -2.0 * -1.0 * (quantoChange(3) - quantoChange(4));
            
            
            value = value -2.0 * 1.0 * (measureChange(1) - measureChange(2) + measureChange(3) - measureChange(4));
            value = value -2.0 * 1.0 * (measureChange(5) - measureChange(6) + measureChange(7) - measureChange(8));
            
            out = value;
            
        end
        
        function beta = Beta(quantolgm2f_lgm2f,t,T)
            beta =[];
            beta(1) = quantolgm2f_lgm2f.H(T,quantolgm2f_lgm2f.foreign_dH_r)-quantolgm2f_lgm2f.H(t,quantolgm2f_lgm2f.foreign_dH_r);
            beta(2) = quantolgm2f_lgm2f.H(T,quantolgm2f_lgm2f.foreign_dH_s)-quantolgm2f_lgm2f.H(t,quantolgm2f_lgm2f.foreign_dH_s);
            
        end
        
        function beta = BetaDate(quantolgm2f_lgm2f,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            beta(1) = quantolgm2f_lgm2f.H(T,quantolgm2f_lgm2f.foreign_dH_r)-quantolgm2f_lgm2f.H(t,quantolgm2f_lgm2f.foreign_dH_r);
            beta(2) = quantolgm2f_lgm2f.H(T,quantolgm2f_lgm2f.foreign_dH_s)-quantolgm2f_lgm2f.H(t,quantolgm2f_lgm2f.foreign_dH_s);
            
        end
        
        function c_nN = C_nN(quantolgm2f_lgm2f,tn,tN,tau,pnN,ynN)
            p = @quantolgm2f_lgm2f.DF;
            c_nN=[0.0;0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = quantolgm2f_lgm2f.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
             
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
            
        end
        
        function c_nN = C_nNDate(quantolgm2f_lgm2f,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @quantolgm2f_lgm2f.DF;
            c_nN=[0.0;0.0];
            beta = [0.0;0.0];
            df = 0.0;
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = quantolgm2f_lgm2f.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(1) = c_nN(1) +deltaT * beta(1) * df;
                c_nN(2) = c_nN(2) +deltaT * beta(2) * df;
                prevTime = tp;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
            
        end
        
        function g_nN = G_nN(quantolgm2f_lgm2f,tn,tN,tau,pnN)
            p = @quantolgm2f_lgm2f.DF;
            g_nN=[0.0;0.0];
            
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = quantolgm2f_lgm2f.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
            
        end
        
        function g_nN = G_nNDate(quantolgm2f_lgm2f,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @quantolgm2f_lgm2f.DF;
            g_nN=[0.0;0.0];
            
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = quantolgm2f_lgm2f.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(1) = g_nN(1) + deltaT * beta(1) * df;
                g_nN(2) = g_nN(2) + deltaT * beta(2) * df;
                prevTime = tp;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
        end
        
        function h_nTerm = H_nTerm(quantolgm2f_lgm2f,tn,tTerm)
           h_nTerm =[];
           beta = quantolgm2f_lgm2f.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        function h_nTerm = H_nTermDate(quantolgm2f_lgm2f,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = quantolgm2f_lgm2f.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
           
        end
        
        function i_Term = I_Term(quantolgm2f_lgm2f,from,to,tTerm)

            sigmaTimes = quantolgm2f_lgm2f.foreign_Alpha_r.tenor;
            
            foreign_dH_r_Values = quantolgm2f_lgm2f.foreign_dH_r.quote;
            foreign_dH_s_Values = quantolgm2f_lgm2f.foreign_dH_s.quote;
            
            foreign_Alpha_r_Values = quantolgm2f_lgm2f.foreign_Alpha_r.quote;
            foreign_Alpha_s_Values = quantolgm2f_lgm2f.foreign_Alpha_s.quote;
            
            domestic_dH_r_Values = quantolgm2f_lgm2f.domestic_dH_r.quote;
            domestic_Alpha_r_Values = quantolgm2f_lgm2f.domestic_Alpha_r.quote;
            corr_drfr_Values = quantolgm2f_lgm2f.corr_drfr.quote;
            corr_drfs_Values = quantolgm2f_lgm2f.corr_drfs.quote;
            
            domestic_dH_s_Values = quantolgm2f_lgm2f.domestic_dH_s.quote;
            domestic_Alpha_s_Values = quantolgm2f_lgm2f.domestic_Alpha_s.quote;
            corr_dsfr_Values = quantolgm2f_lgm2f.corr_dsfr.quote;
            corr_dsfs_Values = quantolgm2f_lgm2f.corr_dsfs.quote;
            
            variance_drfr = quantolgm2f_lgm2f.GeneralVariance_BSLGM(from,to,tTerm,sigmaTimes,domestic_dH_r_Values,domestic_Alpha_r_Values,...
                                                         foreign_Alpha_r_Values,corr_drfr_Values);
            
            variance_dsfr = quantolgm2f_lgm2f.GeneralVariance_BSLGM(from,to,tTerm,sigmaTimes,domestic_dH_s_Values,domestic_Alpha_s_Values,...
                 foreign_Alpha_r_Values,corr_dsfr_Values);
            
            
            variance_drfs = quantolgm2f_lgm2f.GeneralVariance_BSLGM(from,to,tTerm,sigmaTimes,domestic_dH_r_Values,domestic_Alpha_r_Values,...
                                                         foreign_Alpha_s_Values,corr_drfs_Values);
            
            variance_dsfs = quantolgm2f_lgm2f.GeneralVariance_BSLGM(from,to,tTerm,sigmaTimes,domestic_dH_s_Values,domestic_Alpha_s_Values,...
                 foreign_Alpha_s_Values,corr_dsfs_Values); 

            % we add minus 1 so that we don't need additional minus in
            % drift adjust term
            i_Term = [0.0;0.0];
            i_Term(1) = -1.0 * (variance_drfr + variance_dsfr);
            i_Term(2) = -1.0 * (variance_drfs + variance_dsfs);
            
        end
        
        function i_Tn_tilda = I_Tn_tilda(quantolgm2f_lgm2f,from,to,tn)

            sigmaTimes = quantolgm2f_lgm2f.foreign_Alpha_r.tenor;
            foreign_dH_r_Values = quantolgm2f_lgm2f.foreign_dH_r.quote;
            foreign_dH_s_Values = quantolgm2f_lgm2f.foreign_dH_s.quote;
            
            foreign_Alpha_r_Values = quantolgm2f_lgm2f.foreign_Alpha_r.quote;
            foreign_Alpha_s_Values = quantolgm2f_lgm2f.foreign_Alpha_s.quote;
            
            corr_frfr_Values = ones(length(sigmaTimes),1);
            corr_fsfs_Values = ones(length(sigmaTimes),1);
            
            
            variance_frfr = quantolgm2f_lgm2f.GeneralVariance_BSLGM(from,to,tn,sigmaTimes,foreign_dH_r_Values,...
                                                                        foreign_Alpha_r_Values,foreign_Alpha_r_Values,corr_frfr_Values);
            
            variance_fsfs = quantolgm2f_lgm2f.GeneralVariance_BSLGM(from,to,tn,sigmaTimes,foreign_dH_s_Values,...
                                                                        foreign_Alpha_s_Values,foreign_Alpha_s_Values,corr_fsfs_Values);
                                                                    

           % we don't need minus 1
           i_Tn_tilda = [0.0;0.0];
           i_Tn_tilda(1) = 1.0 * variance_frfr; %variance_frfs = 0
           i_Tn_tilda(2) = 1.0 * variance_fsfs; %variance_fsfr = 0
           
        end
        
        function j_x_tilda = J_x_tilda(quantolgm2f_lgm2f,from,to,tn)


            sigmaTimes = quantolgm2f_lgm2f.foreign_Alpha_r.tenor;
            foreign_Alpha_r_Values = quantolgm2f_lgm2f.foreign_Alpha_r.quote;
            foreign_Alpha_s_Values = quantolgm2f_lgm2f.foreign_Alpha_s.quote;
            
            fxBlackVol_Values = quantolgm2f_lgm2f.fxBlackVol.quote;
            corr_frx_Values = quantolgm2f_lgm2f.corr_frx.quote;
            corr_fsx_Values = quantolgm2f_lgm2f.corr_fsx.quote;
            
            variance_frx = quantolgm2f_lgm2f.LocalVariance_Alpha2(from,to,sigmaTimes,...
                                                                        foreign_Alpha_r_Values,fxBlackVol_Values,corr_frx_Values);
            
            variance_fsx = quantolgm2f_lgm2f.LocalVariance_Alpha2(from,to,sigmaTimes,...
                                                                        foreign_Alpha_s_Values,fxBlackVol_Values,corr_fsx_Values);
                                                                    
           
           % we need minus 1
           j_x_tilda = [0.0;0.0];
           j_x_tilda(1) = -1.0 * variance_frx;
           j_x_tilda(2) = -1.0 * variance_fsx;
        end
        

        
        function out = discountFactor(quantolgm2f_lgm2f, startTime, endTime,numMatTime,modelStatesT)
           
            zcT = quantolgm2f_lgm2f.DF(endTime/365.0);
           
            %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = quantolgm2f_lgm2f.DF(startTime/365.0);
           beta = quantolgm2f_lgm2f.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = quantolgm2f_lgm2f.fwdZcVarianceForeign(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           % quantolgm2f_lgm2f is one factor model
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i)- beta(2)*modelStatesT(2,i));
           end
           
           out = discountFactorOut;
           
        end

        function variances = LocalVariance(quantolgm2f_lgm2f,from,to)

            sig(1) = quantolgm2f_lgm2f.foreign_Alpha_r;
            sig(2) = quantolgm2f_lgm2f.foreign_Alpha_s;
            corr_rs = quantolgm2f_lgm2f.corr_frfs;
            corr(1,1) = 1.0;
            corr(2,1) = corr_rs.quote(1);
            corr(1,2) = corr(2,1);
            corr(2,2) = 1.0;
            corr = 1.0;
            
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = zeros(2,2);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                
                correl =ones(2,2);
                if (i< alphaSize) correl(2,1) = corr_rs.quote(i);else correl(2,1) = corr_rs.quote(alphaSize);end;
                correl(1,2) = correl(2,1);
                for j=1:2
                    for k=1:2
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end

        function out = LocalDrift(quantolgm2f_lgm2f,from,to)
            drift = eye(4,4);
%             drift(3,3) =  exp(-1.0*quantolgm2f_lgm2f.foreign_meanR_r*(to-from));
            out = drift;
        end
        
        function variances = LocalCovariance(quantolgm2f_lgm2f,from,to)

            sig(1) = quantolgm2f_lgm2f.domestic_Alpha_r;
            sig(2) = quantolgm2f_lgm2f.domestic_Alpha_s;
            
            sig(3) = quantolgm2f_lgm2f.foreign_Alpha_r;
            sig(4) = quantolgm2f_lgm2f.foreign_Alpha_s;
            
            corr_drds = quantolgm2f_lgm2f.corr_drds;
            corr_drfr = quantolgm2f_lgm2f.corr_drfr;
            corr_dsfr = quantolgm2f_lgm2f.corr_dsfr;
            
            corr_frfs = quantolgm2f_lgm2f.corr_frfs;
            corr_drfs = quantolgm2f_lgm2f.corr_drfs;
            corr_dsfs = quantolgm2f_lgm2f.corr_dsfs;
            
            correl = zeros(4,4);
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
  
           lastU  = from;
           
           variances = zeros(4,4);
           correl =ones(4,4);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end
                if (i< alphaSize) vol(3) = sig(3).quote(i);else vol(3) = sig(3).quote(alphaSize);end
                if (i< alphaSize) vol(4) = sig(4).quote(i);else vol(4) = sig(4).quote(alphaSize);end
                
                if (i< alphaSize) correl(1,2) = corr_drds.quote(i);else correl(1,2) = corr_drds.quote(alphaSize);end
                if (i< alphaSize) correl(1,3) = corr_drfr.quote(i);else correl(1,3) = corr_drfr.quote(alphaSize);end
                if (i< alphaSize) correl(1,4) = corr_drfs.quote(i);else correl(1,4) = corr_drfs.quote(alphaSize);end
                
                if (i< alphaSize) correl(2,3) = corr_dsfr.quote(i);else correl(2,3) = corr_dsfr.quote(alphaSize);end
                if (i< alphaSize) correl(2,4) = corr_dsfs.quote(i);else correl(2,4) = corr_dsfs.quote(alphaSize);end
                
                if (i< alphaSize) correl(3,4) = corr_frfs.quote(i);else correl(3,4) = corr_frfs.quote(alphaSize);end
                
                correl(2,1) = correl(1,2);
                correl(3,1) = correl(1,3);
                correl(4,1) = correl(1,4);
                
                correl(3,2) = correl(2,3);
                correl(4,2) = correl(2,4);
                
                correl(4,3) = correl(3,4);
                
                for j=1:4
                    for k=1:4

                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           

           
        end
        
        function out = LocalVariance_H2_Alpha2(quantolgm2f_lgm2f,from,to,sigmaTimes,dH1Values, dH2Values, sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1.quote = dH1Values;
           dH1.tenor = sigmaTimes;
           
           dH2.quote = dH2Values;
           dH2.tenor = sigmaTimes;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH1Value = dH1.quote(i);else dH1Value = dH1.quote(sigmaSize);end;
                if (i< sigmaSize) dH2Value = dH2.quote(i);else dH2Value = dH2.quote(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + quantolgm2f_lgm2f.H(lastU,dH1)*quantolgm2f_lgm2f.H(lastU,dH2)*correl*vol1*vol2*(U - lastU);
                variances = variances + dH1Value*quantolgm2f_lgm2f.H(lastU,dH2)*correl*vol1*vol2*0.5*(U - lastU)*(U - lastU);
                variances = variances + quantolgm2f_lgm2f.H(lastU,dH1)*dH2Value*correl*vol1*vol2*0.5*(U - lastU)*(U - lastU);
                variances = variances + dH1Value*dH2Value*correl*vol1*vol2*1.0/3.0*(U - lastU)*(U - lastU)*(U - lastU);
                
                
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = LocalVariance_H_Alpha2(quantolgm2f_lgm2f,from,to,sigmaTimes,dH1Values,sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1.quote = dH1Values;
           dH1.tenor = sigmaTimes;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH = dH1.quote(i);else dH = dH1.quote(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + quantolgm2f_lgm2f.H(lastU,dH1) * correl * vol1 * vol2 * (U - lastU);
                variances = variances + 0.5 * dH * correl * vol1 * vol2 * (U - lastU) * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = LocalVariance_Alpha2(quantolgm2f_lgm2f,from,to,sigmaTimes,sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + correl* vol1 * vol2 * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        

        
        function out = GeneralVariance_LGMLGM(quantolgm2f_lgm2f,a,b,T1,T2,sigmaTimes,dH1Values,dH2Values,Alpha1,Alpha2,corrValues)
            % Variance in [a,b] of (H(T1)-H(s))(H(T2)-H(s))alpha1(s)alpha2(s)
            dH1.quote = dH1Values;
            dH1.tenor = sigmaTimes;
            
            dH2.quote = dH2Values;
            dH2.tenor = sigmaTimes;
            
            variances = zeros(4,1);
            variances(1) = quantolgm2f_lgm2f.LocalVariance_Alpha2(a,b,sigmaTimes, Alpha1,Alpha2,corrValues);
            variances(1) = variances(1) * quantolgm2f_lgm2f.H(T1,dH1) * quantolgm2f_lgm2f.H(T2,dH2);
            
            variances(2) = quantolgm2f_lgm2f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH1Values,Alpha1,Alpha2,corrValues);
            variances(2) = variances(2) * -1.0* quantolgm2f_lgm2f.H(T2,dH2);
            
            variances(3) = quantolgm2f_lgm2f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH2Values,Alpha1,Alpha2,corrValues);
            variances(3) = variances(3) * -1.0* quantolgm2f_lgm2f.H(T1,dH1);
            
            variances(4) = quantolgm2f_lgm2f.LocalVariance_H2_Alpha2(a,b,sigmaTimes,dH1Values,dH2Values,Alpha1,Alpha2,corrValues);
            
            value = variances(1) + variances(2) + variances(3) + variances(4);
            out = value;
        end
        
        function out = GeneralVariance_BSLGM(quantolgm2f_lgm2f,a,b,T1,sigmaTimes,dH1Values,Alpha1,Alpha2,corrValues)
            % Variance in [a,b] of (H(T1)-H(s))alpha1(s)alpha2(s)
            dH1.quote = dH1Values;
            dH1.tenor = sigmaTimes;
            
            variances = zeros(2,1);
            variances(1) = quantolgm2f_lgm2f.LocalVariance_Alpha2(a,b,sigmaTimes, Alpha1,Alpha2,corrValues);
            variances(1) = variances(1) * quantolgm2f_lgm2f.H(T1,dH1);
            
            variances(2) = quantolgm2f_lgm2f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH1Values,Alpha1,Alpha2,corrValues);
            variances(2) = variances(2) * -1.0;
            
            value = variances(1) + variances(2);
            out = value;
        end
        
        function out = CMSDigitalArgDate(quantolgm2f_lgm2f,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           to = toDate.DateDiff(valueDate)/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           tauIn = tau;
           if abs(tenor1- 0.25) < 1e-8
                tauIn = tenor1;
           end
           
           p_nN1 = quantolgm2f_lgm2f.PV01Date(valueDate,toDate,tN1Date,tauIn);
           
           df_Tn1 = quantolgm2f_lgm2f.DF(to);
           df_TN1 = quantolgm2f_lgm2f.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = quantolgm2f_lgm2f.C_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1,y_nN1);
           g_nN1 = quantolgm2f_lgm2f.G_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1);
%            h_nTerm1 = quantolgm2f_lgm2f.H_nTermDate(valueDate,toDate,tpDate);
           % h_nTerm1 is replaced with i_Term & i_Tn_tilda as in the
           % following discussion
           
           h_nTerm1(1) = 0.0;
           h_nTerm1(2) = 0.0;
           
           varCovar = quantolgm2f_lgm2f.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:2
               for j=1:2
                    driftTerm = driftTerm + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           tpTime = tpDate.DateDiff(valueDate);
           % additional drift correction term from foreign Term measure USD to KRW domestic Term 
           i_Term = I_Term(quantolgm2f_lgm2f,0,to,tpTime/365.0);
           i_Tn_tilda = I_Tn_tilda(quantolgm2f_lgm2f,0,to,to);
           j_x_tilda = J_x_tilda(quantolgm2f_lgm2f,0,to,to);
           
           additionalDrift = 0;
           for i=1:2
                additionalDrift = additionalDrift+ c_nN1(i) *(i_Term(i) + i_Tn_tilda(i) + j_x_tilda(i));
           end
           
           variance = 0.0;
           for i=1:2
               for j=1:2
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.fwdCMS = y_nN1;
           out.mu = y_nN1 + driftTerm + additionalDrift;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
        end
                       
        function out = CMSSpreadDigitalArgDate(quantolgm2f_lgm2f,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS Spread rate > strike
            %CMS 1
           to = toDate.DateDiff(valueDate)/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           tauIn = tau;
%            if abs(tenor1- 0.25) < 1e-8
%                 tauIn = tenor1;
%            end
           
           p_nN1 = quantolgm2f_lgm2f.PV01Date(valueDate,toDate,tN1Date,tauIn);
           
           df_Tn1 = quantolgm2f_lgm2f.DF(to);
           df_TN1 = quantolgm2f_lgm2f.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = quantolgm2f_lgm2f.C_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1,y_nN1);
           g_nN1 = quantolgm2f_lgm2f.G_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1);

           %CMS 2
           if tenor2 < 1 && tenor2 >= 1.0/12.0
               tenorMonth2 = round(12*tenor2);
                tN2Date = toDate.AddDate(tenorMonth2,'month');
           elseif tenor2 >= 1
               tN2Date = toDate.AddDate(tenor2,'year');
           else
               disp('unimplemented')
           end
           
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
%            tauIn = tau;
%            if abs(tenor1- 0.25) < 1e-8
%                 tauIn = tenor1;
%            end
           
           p_nN2 = quantolgm2f_lgm2f.PV01Date(valueDate,toDate,tN2Date,tauIn);
           
           df_Tn2 = quantolgm2f_lgm2f.DF(to);
           df_TN2 = quantolgm2f_lgm2f.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = quantolgm2f_lgm2f.C_nNDate(valueDate,toDate,tN2Date,tauIn,p_nN2,y_nN2);
           g_nN2 = quantolgm2f_lgm2f.G_nNDate(valueDate,toDate,tN2Date,tauIn,p_nN2);
           
           % h_nTerm1 is replaced with i_Term & i_Tn_tilda as in the
           % following discussion
           
           h_nTerm1(1) = 0.0;
           h_nTerm1(2) = 0.0;
           
           h_nTerm2(1) = 0.0;
           h_nTerm2(2) = 0.0;
           
           varCovar = quantolgm2f_lgm2f.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           tpTime = tpDate.DateDiff(valueDate);
           % additional drift correction term from foreign Term measure USD to KRW domestic Term 
           i_Term = I_Term(quantolgm2f_lgm2f,0,to,tpTime/365.0);
           i_Tn_tilda = I_Tn_tilda(quantolgm2f_lgm2f,0,to,to);
           j_x_tilda = J_x_tilda(quantolgm2f_lgm2f,0,to,to);
           
           additionalDrift1 = 0;
           additionalDrift2 = 0;
           
           for i=1:2
                additionalDrift1 = additionalDrift1+ c_nN1(i) *(i_Term(i) + i_Tn_tilda(i) + j_x_tilda(i));
                additionalDrift2 = additionalDrift2+ c_nN2(i) *(i_Term(i) + i_Tn_tilda(i) + j_x_tilda(i));
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + c_nN1(i) * c_nN1(j) * varCovar(i,j);
                    variance2 = variance2 + c_nN2(i) * c_nN2(j) * varCovar(i,j);
                    coVariance = coVariance + c_nN1(i) * c_nN2(j) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = y_nN1 + driftTerm1 + additionalDrift1 - (y_nN2 + driftTerm2 + additionalDrift2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1 + additionalDrift1;
           out.mu2 = y_nN2 + driftTerm2 + additionalDrift2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out =computeBSLGM2FLGM2FVariance(quantolgm2f_lgm2f,a,b,T,corr_drds,corr_frfs,...
                                  corr_drfr,corr_dsfr,corr_drfs,corr_dsfs,corr_drx,corr_dsx,corr_frx,corr_fsx, ...
                                  fxBlackVol, domestic_dH_r,domestic_dH_s, domestic_Alpha_r,domestic_Alpha_s,...
                                  foreign_dH_r,foreign_dH_s, foreign_Alpha_r,foreign_Alpha_s)
                 
          % synchronize model parameters time-steps start
            corr_drds_Tenor = corr_drds.tenor;
            corr_drds_Quote = corr_drds.quote;
            
            corr_frfs_Tenor = corr_frfs.tenor;
            corr_frfs_Quote = corr_frfs.quote;
            
            corr_drfr_Tenor = corr_drfr.tenor;
            corr_drfr_Quote = corr_drfr.quote;
            
            corr_dsfr_Tenor = corr_dsfr.tenor;
            corr_dsfr_Quote = corr_dsfr.quote;
            
            corr_drfs_Tenor = corr_drfs.tenor;
            corr_drfs_Quote = corr_drfs.quote;
            
            corr_dsfs_Tenor = corr_dsfs.tenor;
            corr_dsfs_Quote = corr_dsfs.quote;
            
            corr_drx_Tenor = corr_drx.tenor;
            corr_drx_Quote = corr_drx.quote;
            
            corr_dsx_Tenor = corr_dsx.tenor;
            corr_dsx_Quote = corr_dsx.quote;
            
            corr_frx_Tenor = corr_frx.tenor;
            corr_frx_Quote = corr_frx.quote;
            
            corr_fsx_Tenor = corr_fsx.tenor;
            corr_fsx_Quote = corr_fsx.quote;
            
            fxBlackVolTenor = fxBlackVol.tenor;
            fxBlackVolQuote = fxBlackVol.quote;

            domestic_dH_r_Tenor = domestic_dH_r.tenor;
            domestic_dH_r_Quote = domestic_dH_r.quote;

            domestic_dH_s_Tenor = domestic_dH_s.tenor;
            domestic_dH_s_Quote = domestic_dH_s.quote;
            
            domestic_Alpha_r_Tenor = domestic_Alpha_r.tenor;
            domestic_Alpha_r_Quote = domestic_Alpha_r.quote;

            domestic_Alpha_s_Tenor = domestic_Alpha_s.tenor;
            domestic_Alpha_s_Quote = domestic_Alpha_s.quote;

            foreign_dH_r_Tenor = foreign_dH_r.tenor;
            foreign_dH_r_Quote = foreign_dH_r.quote;
            
            foreign_dH_s_Tenor = foreign_dH_s.tenor;
            foreign_dH_s_Quote = foreign_dH_s.quote;

            foreign_Alpha_r_Tenor = foreign_Alpha_r.tenor;
            foreign_Alpha_r_Quote = foreign_Alpha_r.quote;

            foreign_Alpha_s_Tenor = foreign_Alpha_s.tenor;
            foreign_Alpha_s_Quote = foreign_Alpha_s.quote;



            sigmaTimes = unique(union([corr_drds_Tenor(:);corr_frfs_Tenor(:);corr_drfr_Tenor(:);corr_dsfr_Tenor(:);...
                         corr_drfs_Tenor(:);corr_dsfs_Tenor(:);...
                         corr_drx_Tenor(:);corr_dsx_Tenor(:);corr_frx_Tenor(:);corr_fsx_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:);foreign_Alpha_r_Tenor(:);foreign_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:);domestic_dH_s_Tenor(:);foreign_dH_r_Tenor(:);foreign_dH_s_Tenor(:)],fxBlackVolTenor));


            sigmaTimes = sort(sigmaTimes,'ascend');

            corr_drds_Values = zeros(length(sigmaTimes),1);
            corr_frfs_Values = zeros(length(sigmaTimes),1);
            
            corr_drfr_Values = zeros(length(sigmaTimes),1);
            corr_dsfr_Values = zeros(length(sigmaTimes),1);
            
            corr_drfs_Values = zeros(length(sigmaTimes),1);
            corr_dsfs_Values = zeros(length(sigmaTimes),1);
            
            corr_drx_Values = zeros(length(sigmaTimes),1);
            corr_dsx_Values = zeros(length(sigmaTimes),1);
            
            corr_frx_Values = zeros(length(sigmaTimes),1);
            corr_fsx_Values = zeros(length(sigmaTimes),1);
            

            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            foreign_Alpha_r_Values = zeros(length(sigmaTimes),1);
            foreign_Alpha_s_Values = zeros(length(sigmaTimes),1);
            
            foreign_dH_r_Values = zeros(length(sigmaTimes),1);
            foreign_dH_s_Values = zeros(length(sigmaTimes),1);
            
            fxBlackVolValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                corr_frfs_Values(k) = H_interpolation(corr_frfs_Tenor, corr_frfs_Quote,sigmaTimes(k),0);
                
                corr_drfr_Values(k) = H_interpolation(corr_drfr_Tenor, corr_drfr_Quote,sigmaTimes(k),0);
                corr_dsfr_Values(k) = H_interpolation(corr_dsfr_Tenor, corr_dsfr_Quote,sigmaTimes(k),0);
                
                corr_drfs_Values(k) = H_interpolation(corr_drfs_Tenor, corr_drfs_Quote,sigmaTimes(k),0);
                corr_dsfs_Values(k) = H_interpolation(corr_dsfs_Tenor, corr_dsfs_Quote,sigmaTimes(k),0);
                
                corr_drx_Values(k) = H_interpolation(corr_drx_Tenor, corr_drx_Quote,sigmaTimes(k),0);
                corr_dsx_Values(k) = H_interpolation(corr_dsx_Tenor, corr_dsx_Quote,sigmaTimes(k),0);
                corr_frx_Values(k) = H_interpolation(corr_frx_Tenor, corr_frx_Quote,sigmaTimes(k),0);
                corr_fsx_Values(k) = H_interpolation(corr_fsx_Tenor, corr_fsx_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_Alpha_r_Values(k) = H_interpolation(foreign_Alpha_r_Tenor, foreign_Alpha_r_Quote,sigmaTimes(k),0);
                foreign_Alpha_s_Values(k) = H_interpolation(foreign_Alpha_s_Tenor, foreign_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_dH_r_Values(k) = H_interpolation(foreign_dH_r_Tenor, foreign_dH_r_Quote,sigmaTimes(k),0);
                foreign_dH_s_Values(k) = H_interpolation(foreign_dH_s_Tenor, foreign_dH_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                fxBlackVolValues(k) = H_interpolation(fxBlackVolTenor, fxBlackVolQuote,sigmaTimes(k),0);
            end
            
            % synchronize model parameters time-steps end
            % dummy correlation
            correl_1 = ones(length(sigmaTimes),1);
             
            var_x_x = quantolgm2f_lgm2f.LocalVariance_Alpha2(a,b,sigmaTimes,fxBlackVolValues,fxBlackVolValues,correl_1);
            
            var_fr_fr = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,foreign_dH_r_Values,foreign_dH_r_Values, ...
                                        foreign_Alpha_r_Values,foreign_Alpha_r_Values,correl_1);
                                    
            var_fs_fs = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,foreign_dH_s_Values,foreign_dH_s_Values, ...
                                        foreign_Alpha_s_Values,foreign_Alpha_s_Values,correl_1);
                                                     
            var_dr_dr = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_r_Values,domestic_dH_r_Values, ...
                                                         domestic_Alpha_r_Values,domestic_Alpha_r_Values,correl_1);
                                                     
            var_ds_ds = quantolgm2f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_s_Values,domestic_dH_s_Values, ...
                                                         domestic_Alpha_s_Values,domestic_Alpha_s_Values,correl_1);
           % 1st= sign in techNote , 2nd = HW bond vol? -1,1
            covar_fr_x =  1.0*-1.0*quantolgm2f_lgm2f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
                                            foreign_dH_r_Values,foreign_Alpha_r_Values,fxBlackVolValues,corr_frx_Values);
            
            covar_fs_x =  1.0*-1.0*quantolgm2f_lgm2f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
                                            foreign_dH_s_Values,foreign_Alpha_s_Values,fxBlackVolValues,corr_fsx_Values);

                                        
                                        
            covar_dr_x = -1.0*-1.0*quantolgm2f_lgm2f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
                                            domestic_dH_r_Values,domestic_Alpha_r_Values,fxBlackVolValues,corr_drx_Values);
                                        
            covar_ds_x = -1.0*-1.0*quantolgm2f_lgm2f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
                                            domestic_dH_s_Values,domestic_Alpha_s_Values,fxBlackVolValues,corr_dsx_Values);
                                        
            covar_dr_fr =  -1.0*1.0*quantolgm2f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_r_Values,foreign_dH_r_Values,...
                                            domestic_Alpha_r_Values,foreign_Alpha_r_Values,corr_drfr_Values);
                                        
            covar_ds_fr =  -1.0*1.0*quantolgm2f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_s_Values,foreign_dH_r_Values,...
                                            domestic_Alpha_s_Values,foreign_Alpha_r_Values,corr_dsfr_Values);
                                        
            covar_dr_fs =  -1.0*1.0*quantolgm2f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_r_Values,foreign_dH_s_Values,...
                                            domestic_Alpha_r_Values,foreign_Alpha_s_Values,corr_drfs_Values);
                                        
            covar_ds_fs =  -1.0*1.0*quantolgm2f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_s_Values,foreign_dH_s_Values,...
                                            domestic_Alpha_s_Values,foreign_Alpha_s_Values,corr_dsfs_Values);
                            
            covar_dr_ds =  1.0*1.0*0.0;  % corr_drds is zero
            covar_fr_fs =  1.0*1.0*0.0;  % corr_frfs is zero
                                                         
            value = var_x_x + var_fr_fr + var_fs_fs + var_dr_dr + var_ds_ds + ...
2.0*(covar_fr_x + covar_fs_x + covar_dr_x + covar_ds_x + covar_dr_fr + covar_ds_fr + covar_dr_fr + covar_ds_fr + covar_dr_ds + covar_fr_fs);                                         
            
            out = value;                                         
                                                     
            
        end
        
        function out = FindFXBlackVol(quantolgm2f_lgm2f,index,params)
            
            expiry = params.fxImpVol.tenor(index);
            impVol = params.fxImpVol.quote(index);
            blackVariance = impVol*impVol*expiry;
            optParams.blackVariance = blackVariance;
            
            optParams.params = params;
            optParams.index = index;
            modelParamSize = 1;
            tvar = zeros(modelParamSize,1);
            lb = zeros(modelParamSize,1);
            ub = zeros(modelParamSize,1);
            
            tvar(1) = params.fxImpVol.quote(index);
            lb(1)= 0.0001;
            ub(1)= 10;
            
            %options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
            options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
            numOfEquation =1;
            x=  zeros(numOfEquation,1);
            [ret, popt, info, covar]=levmar('TargetFunctionBSLGM2FLGM2FVariance',tvar,x, 200, options,'unc',quantolgm2f_lgm2f,optParams);
            
            out.fxBlackVol_index = min(max(popt(1),lb(1)),ub(1));
            out.re = TargetFunctionBSLGM2FLGM2FVariance(popt,quantolgm2f_lgm2f,optParams);
%             params.fxBlackVol.quote(index) = popt(1);

        end
        
        function out = CalibrateBSLGM2FLGM2FVol(quantolgm2f_lgm2f, paramsIn)
 
                expirySize = length(paramsIn.fxImpVol.tenor);
                out.fxBlackVol.quote = zeros(expirySize,1);
                out.fxBlackVol.tenor = paramsIn.fxImpVol.tenor;
                out.re =  zeros(expirySize,1);
                out.rmseTotal = 0;
                for i=1:expirySize
                    index = i;
                    BootstrapOut = quantolgm2f_lgm2f.FindFXBlackVol(index,paramsIn);
                    out.fxBlackVol.quote(i) = BootstrapOut.fxBlackVol_index;
                    params.fxBlackVol.quote(i) = BootstrapOut.fxBlackVol_index;
                    out.re(i) = BootstrapOut.re;
                    out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                end
                out.rmseTotal = out.rmseTotal/expirySize;
                
                out.rmseTotal = sqrt(out.rmseTotal);
                 
                
        end
        
        
    end
    
end



In [ ]:
classdef QuantoLGM1F_LGM2F < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        domestic_dH_r;
        domestic_dH_s;
        
        foreign_dH_r;
        
        domestic_Alpha_r;
        domestic_Alpha_s;
        
        foreign_Alpha_r;
        
        %market input
        corr_df;
        corr_dx;
        corr_fx;
        
        %modelParams
        
        corr_drds;
        
        corr_drfr;
        corr_dsfr;
        
        corr_drx;
        corr_dsx;
        
        corr_frx;
        
        
        fxBlackVol;
        
        numOfFactor;
        modelType;
        freq;
    end
    
    methods
        function quantolgm1f_lgm2f = QuantoLGM1F_LGM2F(params)
            if nargin > 0

                quantolgm1f_lgm2f.zeroCurve = params.foreignModel.zeroCurve;
                quantolgm1f_lgm2f.freq = params.foreignModel.freq;
                
                quantolgm1f_lgm2f.numOfFactor = 1;
                quantolgm1f_lgm2f.modelType = 'quantolgm1f_lgm2f';
                
                
                if params.fxVolCalibrated == true
                    % we do not calibrate fxBlackVol
                    quantolgm1f_lgm2f.corr_df = params.corr_df;
                    quantolgm1f_lgm2f.corr_dx = params.corr_dx;
                    quantolgm1f_lgm2f.corr_fx = params.corr_fx;
                    
                    
                    
                    out = quantolgm1f_lgm2f.synchronizeModelParams(params);
                    
                    quantolgm1f_lgm2f.corr_drds.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.corr_drds.quote = out.corr_drds_Values;
                    
                    quantolgm1f_lgm2f.corr_drfr.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.corr_drfr.quote = out.corr_drfr_Values;
                    
                    quantolgm1f_lgm2f.corr_dsfr.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.corr_dsfr.quote = out.corr_dsfr_Values;
                    
                    quantolgm1f_lgm2f.corr_drx.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.corr_drx.quote = out.corr_drx_Values;
                    
                    quantolgm1f_lgm2f.corr_dsx.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.corr_dsx.quote = out.corr_dsx_Values;
                    
                    quantolgm1f_lgm2f.corr_frx.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.corr_frx.quote = out.corr_frx_Values;
                    
                    quantolgm1f_lgm2f.domestic_Alpha_r.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.domestic_Alpha_r.quote = out.domestic_Alpha_r_Values;
                    
                    quantolgm1f_lgm2f.domestic_Alpha_s.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.domestic_Alpha_s.quote = out.domestic_Alpha_s_Values;

                    quantolgm1f_lgm2f.foreign_Alpha_r.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.foreign_Alpha_r.quote = out.foreign_Alpha_r_Values;

                    quantolgm1f_lgm2f.domestic_dH_r.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.domestic_dH_r.quote = out.domestic_dH_r_Values;

                    quantolgm1f_lgm2f.domestic_dH_s.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.domestic_dH_s.quote = out.domestic_dH_s_Values;
                    
                    quantolgm1f_lgm2f.foreign_dH_r.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.foreign_dH_r.quote = out.foreign_dH_r_Values;
                    
                    quantolgm1f_lgm2f.fxBlackVol.tenor = out.sigmaTimes;
                    quantolgm1f_lgm2f.fxBlackVol.quote = out.fxBlackVolValues;
                    
                else
                    
                    % black vol calibration start
                    paramsIn.fxImpVol = params.fxBlackVol;
                    paramsIn.fxBlackVol = paramsIn.fxImpVol;
                    paramsIn.domestic_dH_r = params.domesticModel.dH1;
                    paramsIn.domestic_dH_s = params.domesticModel.dH2;
                                        
                    paramsIn.domestic_Alpha_r = params.domesticModel.Alpha1;
                    paramsIn.domestic_Alpha_s = params.domesticModel.Alpha2;
                    
                    paramsIn.foreign_dH_r = params.foreignModel.dH1;
                    paramsIn.foreign_Alpha_r = params.foreignModel.Alpha1;
                    
                    paramsIn.corr_df = params.corr_df;
                    paramsIn.corr_dx = params.corr_dx;
                    paramsIn.corr_fx = params.corr_fx;
                    
                    paramsIn.corr_drds = params.corr_drds;
                    
                    paramsIn.corr_drfr = params.corr_drfr;
                    paramsIn.corr_dsfr = params.corr_dsfr;

                    paramsIn.corr_drx = params.corr_drx;
                    paramsIn.corr_dsx = params.corr_dsx;
                    
                    paramsIn.corr_frx = params.corr_frx;
                    
                    
                    % BSLGM1FLGM2F Calibration Start
                    out = quantolgm1f_lgm2f.CalibrateBSLGM1FLGM2FVol(paramsIn);
                    re =  out.re;
                    rmseTotal = out.rmseTotal;
                    
                    % BSLGM1FLGM2F Calibration End
                    
                    params.fxBlackVol = out.fxBlackVol;
                    params.fxVolCalibrated =  true;
                    quantolgm1f_lgm2f = QuantoLGM1F_LGM2F(params); 
                    
                end
                %end
                
                
            end
        end
        
        function out =  synchronizeModelParams(quantolgm1f_lgm2f,params)
            fxBlackVolTenor = params.fxBlackVol.tenor;
            fxBlackVolQuote = params.fxBlackVol.quote;

            % synchronize model parameters time-steps start
            corr_drds_Tenor = params.corr_drds.tenor;
            corr_drds_Quote = params.corr_drds.quote;
            
            corr_drfr_Tenor = params.corr_drfr.tenor;
            corr_drfr_Quote = params.corr_drfr.quote;
            
            corr_dsfr_Tenor = params.corr_dsfr.tenor;
            corr_dsfr_Quote = params.corr_dsfr.quote;
            
            corr_drx_Tenor = params.corr_drx.tenor;
            corr_drx_Quote = params.corr_drx.quote;
            
            corr_dsx_Tenor = params.corr_dsx.tenor;
            corr_dsx_Quote = params.corr_dsx.quote;
            
            corr_frx_Tenor = params.corr_frx.tenor;
            corr_frx_Quote = params.corr_frx.quote;
            
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;

            foreign_Alpha_r_Tenor = params.foreignModel.Alpha1.tenor;
            foreign_Alpha_r_Quote = params.foreignModel.Alpha1.quote;
            
            foreign_dH_r_Tenor = params.foreignModel.dH1.tenor;
            foreign_dH_r_Quote = params.foreignModel.dH1.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);corr_drfr_Tenor(:);corr_dsfr_Tenor(:);...
                         corr_drx_Tenor(:);corr_dsx_Tenor(:);corr_frx_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:);foreign_Alpha_r_Tenor(:); ...
                         domestic_dH_r_Tenor(:);domestic_dH_s_Tenor(:);foreign_dH_r_Tenor(:)],fxBlackVolTenor));


            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            corr_drfr_Values = zeros(length(sigmaTimes),1);
            corr_dsfr_Values = zeros(length(sigmaTimes),1);
            corr_drx_Values = zeros(length(sigmaTimes),1);
            corr_dsx_Values = zeros(length(sigmaTimes),1);
            corr_frx_Values = zeros(length(sigmaTimes),1);

            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            foreign_Alpha_r_Values = zeros(length(sigmaTimes),1);
            foreign_dH_r_Values = zeros(length(sigmaTimes),1);
            
            fxBlackVolValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                corr_drfr_Values(k) = H_interpolation(corr_drfr_Tenor, corr_drfr_Quote,sigmaTimes(k),0);
                corr_dsfr_Values(k) = H_interpolation(corr_dsfr_Tenor, corr_dsfr_Quote,sigmaTimes(k),0);
                corr_drx_Values(k) = H_interpolation(corr_drx_Tenor, corr_drx_Quote,sigmaTimes(k),0);
                corr_dsx_Values(k) = H_interpolation(corr_dsx_Tenor, corr_dsx_Quote,sigmaTimes(k),0);
                corr_frx_Values(k) = H_interpolation(corr_frx_Tenor, corr_frx_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_Alpha_r_Values(k) = H_interpolation(foreign_Alpha_r_Tenor, foreign_Alpha_r_Quote,sigmaTimes(k),0);
                foreign_dH_r_Values(k) = H_interpolation(foreign_dH_r_Tenor, foreign_dH_r_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                fxBlackVolValues(k) = H_interpolation(fxBlackVolTenor, fxBlackVolQuote,sigmaTimes(k),0);
            end
            
            out.sigmaTimes = sigmaTimes;
            
            out.corr_drds_Values = corr_drds_Values;
            out.corr_drfr_Values = corr_drfr_Values;
            out.corr_dsfr_Values = corr_dsfr_Values;
            out.corr_drx_Values = corr_drx_Values;
            out.corr_dsx_Values = corr_dsx_Values;
            out.corr_frx_Values = corr_frx_Values;
            
            out.domestic_Alpha_r_Values = domestic_Alpha_r_Values;
            out.domestic_Alpha_s_Values = domestic_Alpha_s_Values;
            
            out.foreign_Alpha_r_Values = foreign_Alpha_r_Values;
            out.foreign_dH_r_Values = foreign_dH_r_Values;
            
            out.domestic_dH_r_Values = domestic_dH_r_Values;
            out.domestic_dH_s_Values = domestic_dH_s_Values;
            
            out.fxBlackVolValues = fxBlackVolValues;
            
        end
        
        function out =  Libor_SimpleDate(quantolgm1f_lgm2f,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           
           % foreign Libor in domestic Terminal measure
           
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = quantolgm1f_lgm2f.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = quantolgm1f_lgm2f.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
        function out =  CMS_SimpleDate(quantolgm1f_lgm2f,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMT
%            tau = 0.25;
           tau = 1.0/(quantolgm1f_lgm2f.freq);
            % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           
           if abs(tenor- 0.25) < 1e-8
                tau = tenor;
           end
           
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tNTime = tNDate.DateDiff(valueDate);
           tN     =  tNTime/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           addMonthUnit = round(12*tau);
           cnt = round((tN-to)/tau);
           
           stateSize = size(modelStatesT,2);
           
           annuity = zeros(1,stateSize);
           df_Tp = zeros(1,stateSize);
           
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           for i=1:cnt
                tpDate = toDate.AddDate(i*addMonthUnit,'month');
                tpTime = tpDate.DateDiff(valueDate);
                df_Tp = quantolgm1f_lgm2f.discountFactor(observeTime,tpTime,numMatTime,modelStatesT);
                annuity = annuity + tau * df_Tp;
            end
           
           df_Tn = quantolgm1f_lgm2f.discountFactor(observeTime,observeTime,numMatTime,modelStatesT);
           df_TN = quantolgm1f_lgm2f.discountFactor(observeTime,tNTime,numMatTime,modelStatesT);
           
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           for i=1:stateSize
                cms_PCA_SimpleOut(i) = (df_Tn(i) - df_TN(i))/annuity(i);
           end
           
           out = cms_PCA_SimpleOut;

            
         end
        
         function out =  CMS_PSA_SimpleDate(quantolgm1f_lgm2f,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMT
%            tau = 0.25;
           tau = 1.0/(quantolgm1f_lgm2f.freq);
            % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           
           if abs(tenor- 0.25) < 1e-8
                tau = tenor;
           end
           
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = quantolgm1f_lgm2f.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = quantolgm1f_lgm2f.DF(to);
           df_TN = quantolgm1f_lgm2f.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = quantolgm1f_lgm2f.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = quantolgm1f_lgm2f.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
%            h_nTerm = quantolgm1f_lgm2f.H_nTermDate(valueDate,toDate,numMatDate);
           
           %h_nTerm is replace by i_Term & i_Tn_tilda in the following
           
           h_nTerm(1) = 0.0;
           
           varCovar = quantolgm1f_lgm2f.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:1
               for j=1:1
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           % additional drift correction term from foreign Term USD to
           % domestic Term measure KRW
           i_Term = I_Term(quantolgm1f_lgm2f,0,to,numMatTime/365.0);
           i_Tn_tilda = I_Tn_tilda(quantolgm1f_lgm2f,0,to,to);
           j_x_tilda = J_x_tilda(quantolgm1f_lgm2f,0,to,to);
           
%            additionalDrift = 0;
           additionalDrift = c_nN(1) * (i_Term + i_Tn_tilda + j_x_tilda);
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + additionalDrift + c_nN(1)*modelStatesT(1,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
         end
         
         function out = Foreign_HtT_r(quantolgm1f_lgm2f,t,T)
            out =  quantolgm1f_lgm2f.H(T,quantolgm1f_lgm2f.foreign_dH) - quantolgm1f_lgm2f.H(t,quantolgm1f_lgm2f.foreign_dH); 
         end
         
         function out = Domestic_HtT_r(quantolgm1f_lgm2f,t,T)
            out =  quantolgm1f_lgm2f.H(T,quantolgm1f_lgm2f.domestic_dH) - quantolgm1f_lgm2f.H(t,quantolgm1f_lgm2f.domestic_dH); 
         end
         
         function out = H(quantolgm1f_lgm2f,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
         end

        
        function out = fwdZcVarianceForeign(quantolgm1f_lgm2f, from, to, T1, T2)
            
            localVariance = zeros(2,1);
            measureChange = zeros(4,1);
            quantoChange = zeros(2,1);
            sigmaTimes = quantolgm1f_lgm2f.domestic_Alpha_r.tenor;
            
            foreign_Alpha_r_Values = quantolgm1f_lgm2f.foreign_Alpha_r.quote;
            foreign_dH_r_Values = quantolgm1f_lgm2f.foreign_dH_r.quote;
            
            domestic_Alpha_r_Values = quantolgm1f_lgm2f.domestic_Alpha_r.quote;
            domestic_Alpha_s_Values = quantolgm1f_lgm2f.domestic_Alpha_s.quote;
            
            domestic_dH_r_Values = quantolgm1f_lgm2f.domestic_dH_r.quote;
            domestic_dH_s_Values = quantolgm1f_lgm2f.domestic_dH_s.quote;
            
            fxValues = quantolgm1f_lgm2f.fxBlackVol.quote;
            
            correl_1 = ones(length(sigmaTimes),1);
            
            corr_frx_Values = quantolgm1f_lgm2f.corr_frx.quote;
            
            corr_drfr_Values = quantolgm1f_lgm2f.corr_drfr.quote;
            corr_dsfr_Values = quantolgm1f_lgm2f.corr_dsfr.quote;
            
            
            localVariance(1) = quantolgm1f_lgm2f.GeneralVariance_LGMLGM(from,to,T1,T1,sigmaTimes,foreign_dH_r_Values,foreign_dH_r_Values,...
                                                                        foreign_Alpha_r_Values,foreign_Alpha_r_Values,correl_1);
                                                                    
            localVariance(2) = quantolgm1f_lgm2f.GeneralVariance_LGMLGM(from,to,to,to,sigmaTimes,foreign_dH_r_Values,foreign_dH_r_Values,...
                                                                        foreign_Alpha_r_Values,foreign_Alpha_r_Values,correl_1);
            
%             localVariance(1) = quantolgm1f_lgm2f.GeneralVariance_HWHW(from,to,T1,T1,MRS_r,MRS_r,sigmaTimes,foreign_vol_r_Values,foreign_vol_r_Values,correl_1);
%             localVariance(2) = quantolgm1f_lgm2f.GeneralVariance_HWHW(from,to,to,to,MRS_r,MRS_r,sigmaTimes,foreign_vol_r_Values,foreign_vol_r_Values,correl_1);
            
            quantoChange(1) = -1.0*quantolgm1f_lgm2f.GeneralVariance_BSLGM(from,to,T1,sigmaTimes,foreign_dH_r_Values,foreign_Alpha_r_Values,fxValues,corr_frx_Values);
            
            quantoChange(2) = -1.0*quantolgm1f_lgm2f.GeneralVariance_BSLGM(from,to,to,sigmaTimes,foreign_dH_r_Values,foreign_Alpha_r_Values,fxValues,corr_frx_Values);
            
%             quantoChange(1) = -1.0*quantolgm1f_lgm2f.GeneralVariance_BSHW(from,to,T1,MRS_r,sigmaTimes,foreign_vol_r_Values,fxValues,corr_frx_Values);
%             
%             quantoChange(2) = -1.0*quantolgm1f_lgm2f.GeneralVariance_BSHW(from,to,to,MRS_r,sigmaTimes,foreign_vol_r_Values,fxValues,corr_frx_Values);
            
            measureChange(1) = quantolgm1f_lgm2f.GeneralVariance_LGMLGM(from,to,T1,T2,sigmaTimes,domestic_dH_r_Values,foreign_dH_r_Values,domestic_Alpha_r_Values, ...
                                                                                                    foreign_Alpha_r_Values,corr_drfr_Values);
            
            measureChange(2) = quantolgm1f_lgm2f.GeneralVariance_LGMLGM(from,to,to,T2,sigmaTimes,domestic_dH_r_Values,foreign_dH_r_Values,domestic_Alpha_r_Values, ...
                                                                                                    foreign_Alpha_r_Values,corr_drfr_Values);
                                                                                                
            measureChange(3) = quantolgm1f_lgm2f.GeneralVariance_LGMLGM(from,to,T1,T2,sigmaTimes,domestic_dH_s_Values,foreign_dH_r_Values,domestic_Alpha_s_Values, ...
                                                                                                    foreign_Alpha_r_Values,corr_dsfr_Values);
            
            measureChange(4) = quantolgm1f_lgm2f.GeneralVariance_LGMLGM(from,to,to,T2,sigmaTimes,domestic_dH_s_Values,foreign_dH_r_Values,domestic_Alpha_s_Values, ...
                                                                                                    foreign_Alpha_r_Values,corr_dsfr_Values);
                                                                                                
%             measureChange(1) = quantolgm1f_lgm2f.GeneralVariance_HWLGM(from,to,T1,T2,MRS_r,sigmaTimes,domestic_dH_r_Values,domestic_Alpha_r_Values, ...
%                                                                                                     foreign_vol_r_Values,corr_drfr_Values);
%             
%             measureChange(2) = quantolgm1f_lgm2f.GeneralVariance_HWLGM(from,to,to,T2,MRS_r,sigmaTimes,domestic_dH_r_Values,domestic_Alpha_r_Values, ...
%                                                                                                     foreign_vol_r_Values,corr_drfr_Values);
%                                                                                                 
%             measureChange(3) = quantolgm1f_lgm2f.GeneralVariance_HWLGM(from,to,T1,T2,MRS_r,sigmaTimes,domestic_dH_s_Values,domestic_Alpha_s_Values, ...
%                                                                                                     foreign_vol_r_Values,corr_dsfr_Values);
%             
%             measureChange(4) = quantolgm1f_lgm2f.GeneralVariance_HWLGM(from,to,to,T2,MRS_r,sigmaTimes,domestic_dH_s_Values,domestic_Alpha_s_Values, ...
%                                                                                                     foreign_vol_r_Values,corr_dsfr_Values);
            
                                                                                                
            
            value = localVariance(1) - localVariance(2);
            value = value -2.0 * -1.0 * (quantoChange(1) - quantoChange(2));
            
            value = value -2.0 * 1.0 * (measureChange(1) - measureChange(2) + measureChange(3) - measureChange(4));
            
            
            out = value;
            
        end
        
        function beta = Beta(quantolgm1f_lgm2f,t,T)
            beta =[];
            beta(1) = quantolgm1f_lgm2f.H(T,quantolgm1f_lgm2f.foreign_dH_r)-quantolgm1f_lgm2f.H(t,quantolgm1f_lgm2f.foreign_dH_r);
            
        end
        
        function beta = BetaDate(quantolgm1f_lgm2f,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            beta(1) = quantolgm1f_lgm2f.H(T,quantolgm1f_lgm2f.foreign_dH_r)-quantolgm1f_lgm2f.H(t,quantolgm1f_lgm2f.foreign_dH_r);
            
        end
        
        function c_nN = C_nN(quantolgm1f_lgm2f,tn,tN,tau,pnN,ynN)
            p = @quantolgm1f_lgm2f.DF;
            c_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = quantolgm1f_lgm2f.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
        function c_nN = C_nNDate(quantolgm1f_lgm2f,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @quantolgm1f_lgm2f.DF;
            c_nN=[0.0];
            beta = [0.0];
            df = 0.0;
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = quantolgm1f_lgm2f.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(1) = c_nN(1) +deltaT * beta(1) * df;
                prevTime = tp;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
        end
        
        function g_nN = G_nN(quantolgm1f_lgm2f,tn,tN,tau,pnN)
            p = @quantolgm1f_lgm2f.DF;
            g_nN=[0.0];
            
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = quantolgm1f_lgm2f.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function g_nN = G_nNDate(quantolgm1f_lgm2f,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @quantolgm1f_lgm2f.DF;
            g_nN=[0.0];
            
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = quantolgm1f_lgm2f.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(1) = g_nN(1) + deltaT * beta(1) * df;
                prevTime = tp;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function h_nTerm = H_nTerm(quantolgm1f_lgm2f,tn,tTerm)
           h_nTerm =[];
           beta = quantolgm1f_lgm2f.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function h_nTerm = H_nTermDate(quantolgm1f_lgm2f,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = quantolgm1f_lgm2f.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function i_Term = I_Term(quantolgm1f_lgm2f,from,to,tTerm)

            sigmaTimes = quantolgm1f_lgm2f.foreign_Alpha_r.tenor;
            
            foreign_dH_r_Values = quantolgm1f_lgm2f.foreign_dH_r.quote;
            foreign_Alpha_r_Values = quantolgm1f_lgm2f.foreign_Alpha_r.quote;
            
            domestic_dH_r_Values = quantolgm1f_lgm2f.domestic_dH_r.quote;
            domestic_Alpha_r_Values = quantolgm1f_lgm2f.domestic_Alpha_r.quote;
            corr_drfr_Values = quantolgm1f_lgm2f.corr_drfr.quote;
            
            domestic_dH_s_Values = quantolgm1f_lgm2f.domestic_dH_s.quote;
            domestic_Alpha_s_Values = quantolgm1f_lgm2f.domestic_Alpha_s.quote;
            corr_dsfr_Values = quantolgm1f_lgm2f.corr_dsfr.quote;
            
            variance_drfr = quantolgm1f_lgm2f.GeneralVariance_BSLGM(from,to,tTerm,sigmaTimes,domestic_dH_r_Values,domestic_Alpha_r_Values,...
                                                         foreign_Alpha_r_Values,corr_drfr_Values);
            
            variance_dsfr = quantolgm1f_lgm2f.GeneralVariance_BSLGM(from,to,tTerm,sigmaTimes,domestic_dH_s_Values,domestic_Alpha_s_Values,...
                 foreign_Alpha_r_Values,corr_dsfr_Values);
            
%             variance_drfr = quantolgm1f_lgm2f.GeneralVariance_HWLGM_type2(from,to,to,tTerm,MRS1,sigmaTimes,domestic_dH_r_Values,domestic_Alpha_r_Values,...
%                                                          foreign_vol_r_Values,corr_drfr_Values);
%             
%             variance_dsfr = quantolgm1f_lgm2f.GeneralVariance_HWLGM_type2(from,to,to,tTerm,MRS1,sigmaTimes,domestic_dH_s_Values,domestic_Alpha_s_Values,...
%                  foreign_vol_r_Values,corr_dsfr_Values);
            
            % we add minus 1 so that we don't need additional minus in
            % drift adjust term
            i_Term = -1.0 * (variance_drfr + variance_dsfr);
           
        end
        
        function i_Tn_tilda = I_Tn_tilda(quantolgm1f_lgm2f,from,to,tn)

            sigmaTimes = quantolgm1f_lgm2f.foreign_Alpha_r.tenor;
            foreign_dH_r_Values = quantolgm1f_lgm2f.foreign_dH_r.quote;
            foreign_Alpha_r_Values = quantolgm1f_lgm2f.foreign_Alpha_r.quote;
            corr_frfr_Values = ones(length(sigmaTimes),1);
            
            variance_frfr = quantolgm1f_lgm2f.GeneralVariance_BSLGM(from,to,tn,sigmaTimes,foreign_dH_r_Values,...
                                                                        foreign_Alpha_r_Values,foreign_Alpha_r_Values,corr_frfr_Values);
                                                                    
%             variance_frfr = quantolgm1f_lgm2f.GeneralVariance_HWHW_type2(from,to,tn,tn,MRS1,MRS1,sigmaTimes,...
%                                                                         foreign_vol_r_Values,foreign_vol_r_Values,corr_frfr_Values);

           % we don't need minus 1
           i_Tn_tilda = 1.0 * variance_frfr;
           
        end
        
%         function i_Tn_tilda = I_Tn_tilda_old(quantolgm1f_lgm2f,from,to,tn)
% 
%             sigmaTimes = quantolgm1f_lgm2f.foreign_vol_r.tenor;
%             sigmaSize =  length(sigmaTimes);
%             
%             for i=1:sigmaSize
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:sigmaSize
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%            
%            MRS2 = quantolgm1f_lgm2f.foreign_meanR_r;
%            
%            newExpScale = zeros(2,1);
%            lastExpScale = zeros(2,1);
%            variances = zeros(2,1);
%             
%            lastU  = from;
%            
%            lastExpScale(1) = exp(MRS2*lastU);
%            lastExpScale(2) = exp((MRS2 + MRS2)*lastU);
%            
%            sigmaValues1 = quantolgm1f_lgm2f.foreign_vol_r.quote;
%            sigmaValues2 = quantolgm1f_lgm2f.foreign_vol_r.quote;
%            corr12 = 1.0;
% 
%            
%            for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end;
%                 if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
%                 if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
%                 
%                 newExpScale(1) = exp(MRS2*U);
%                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(1)-lastExpScale(1));
%                 lastExpScale(1) = newExpScale(1);
%                 
%                 newExpScale(2) = exp((MRS2+MRS2)*U);
%                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/(MRS2+MRS2)) * (newExpScale(2)-lastExpScale(2));
%                 lastExpScale(2) = newExpScale(2);
%                 
%                 lastU = U;
%            end
%            
%            value = exp(-1.0*MRS2*tn)*(1.0/ MRS2)*variances(1);
%            value = value + -1.0*exp(-1.0*(MRS2 + MRS2)*tn)*(1.0/ MRS2)*variances(2);
%            % we don't need minus 1
%            i_Tn_tilda = 1.0 * value;
%            
%         end
%         
        
        
        function j_x_tilda = J_x_tilda(quantolgm1f_lgm2f,from,to,tn)


            sigmaTimes = quantolgm1f_lgm2f.foreign_Alpha_r.tenor;
%             foreign_dH_r_Values = quantolgm1f_lgm2f.foreign_dH_r.quote;
            foreign_Alpha_r_Values = quantolgm1f_lgm2f.foreign_Alpha_r.quote;
            fxBlackVol_Values = quantolgm1f_lgm2f.fxBlackVol.quote;
            corr_frx_Values = quantolgm1f_lgm2f.corr_frx.quote;
            
            variance_frx = quantolgm1f_lgm2f.LocalVariance_Alpha2(from,to,sigmaTimes,...
                                                                        foreign_Alpha_r_Values,fxBlackVol_Values,corr_frx_Values);
                                                                    
%             variance_frx = quantolgm1f_lgm2f.GeneralVariance_BSHW_type2(from,to,tn,MRS1,sigmaTimes,...
%                                                                         foreign_vol_r_Values,fxBlackVol_Values,corr_frx_Values);
           
           % we need minus 1
           j_x_tilda = -1.0 * variance_frx;
           
        end
        
%         function j_x_tilda = J_x_tilda_old(quantolgm1f_lgm2f,from,to,tn)
% 
%             sigmaTimes = quantolgm1f_lgm2f.foreign_vol_r.tenor;
%             sigmaSize =  length(sigmaTimes);
%             
%             for i=1:sigmaSize
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:sigmaSize
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%            
%             
%            newExpScale = zeros(1,1);
%            lastExpScale = zeros(1,1);
%            
%            lastU  = from;
%            MRS2 = quantolgm1f_lgm2f.foreign_meanR_r;
%            lastExpScale(1) = exp(MRS2*lastU);
%            variances = 0.0;
%            
%            sigmaValues1 = quantolgm1f_lgm2f.fxBlackVol.quote;
%            sigmaValues2 = quantolgm1f_lgm2f.foreign_vol_r.quote;
%            corrValues12 = quantolgm1f_lgm2f.corr_frx.quote;
%            
%            for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end;
%                 if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
%                 if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
%                 if (i< sigmaSize) corr12 = corrValues12(i);else corr12 = corrValues12(sigmaSize);end;
%                 
%                 newExpScale(1) = exp(MRS2*U);
%                 variances = variances + corr12 * vol1 * vol2 * (1.0/MRS2) * (newExpScale(1)-lastExpScale(1));
%                 lastExpScale(1) = newExpScale(1);
%                 
%                 lastU = U;
%            end
%            
%            % we need minus 1
%            j_x_tilda = -1.0 * exp(-1.0*MRS2*tn) * (1.0/MRS2) * variances;
%            
%         end
        
        function out = discountFactor(quantolgm1f_lgm2f, startTime, endTime,numMatTime,modelStatesT)
           
            zcT = quantolgm1f_lgm2f.DF(endTime/365.0);
           
            %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = quantolgm1f_lgm2f.DF(startTime/365.0);
           beta = quantolgm1f_lgm2f.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = quantolgm1f_lgm2f.fwdZcVarianceForeign(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           % quantolgm1f_lgm2f is one factor model
           for i=1:stateSize
%                discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
           end
           
           out = discountFactorOut;
           
        end
        
%         function out = discountPayoff(quantolgm1f_lgm2f,eventTime,numMatTime ...
%                 ,modelStatesT,cashflow)
%             
%             dfNumMat = quantolgm1f_lgm2f.GetChildModel('d').DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             domesticModelStatesT = modelStatesT(1,:);
%             discountT = quantolgm1f_lgm2f.GetChildModel('d').discountFactor(eventTime, numMatTime,numMatTime,domesticModelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end

        function variances = LocalVariance(quantolgm1f_lgm2f,from,to)

            sig(1) = quantolgm1f_lgm2f.foreign_Alpha_r;
            sig(2) = quantolgm1f_lgm2f.foreign_Alpha_r;
            corr = 1.0;
            
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = zeros(2,2);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                
                correl =ones(2,2);
                correl(1,2) = corr;
                correl(2,1) = corr;
                for j=1:2
                    for k=1:2
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
%         function variances = LocalVariance(quantolgm1f_lgm2f,from,to)
% 
%             sig(1) = quantolgm1f_lgm2f.foreign_vol_r;
%             meanR(1) = quantolgm1f_lgm2f.foreign_meanR_r;
%             
%             alphaSize = length(sig(1).quote);
%             
%             for i=1:length(sig(1).tenor)
%                 if from<= sig(1).tenor(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:length(sig(1).tenor)
%                 if to<= sig(1).tenor(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%             
%             tiny = zeros(1,1);
%             lastExpScale = zeros(1,1);
%             newExpScale  = zeros(1,1);
%             
%             for i=1:1
%                 for j=1:1
%                     if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end
%                     lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
%                 end
%             end
%   
%            lastU  = from;
%            
%            variances = zeros(1,1);
%            
%            for i=startIdx:endIdx
%                 if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
%                 if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
%                 
%                 correl =ones(1,1);
%                 for j=1:1
%                     for k=1:1
%                         newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
%                         if(tiny(j,k)~=1)
%                             scale =newExpScale(j,k)-lastExpScale(j,k);
%                         else
%                             scale = U - lastU;
%                         end
%                         variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
%                                        * scale;
%                         lastExpScale(j,k) = newExpScale(j,k);
%                     end
%                 end
%                 lastU = U;
%            end
%            
%            for j=1:1
%                 for k=1:1
%                     if(tiny(j,k)~= 1)
%                         variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
%                     end
%                 end
%            end
%            
%         end
        
        function out = LocalDrift(quantolgm1f_lgm2f,from,to)
            drift = eye(3,3);
%             drift(3,3) =  exp(-1.0*quantolgm1f_lgm2f.foreign_meanR_r*(to-from));
            out = drift;
        end
        
        function variances = LocalCovariance(quantolgm1f_lgm2f,from,to)

            sig(1) = quantolgm1f_lgm2f.domestic_Alpha_r;
            sig(2) = quantolgm1f_lgm2f.domestic_Alpha_s;
            
            sig(3) = quantolgm1f_lgm2f.foreign_Alpha_r;
            
            corr_drds = quantolgm1f_lgm2f.corr_drds;
            corr_drfr = quantolgm1f_lgm2f.corr_drfr;
            corr_dsfr = quantolgm1f_lgm2f.corr_dsfr;
            
            correl = zeros(3,3);
            
%             meanR = zeros(3,1);
%             
%             meanR(1) = 0.0;
%             meanR(2) = 0.0;
%             meanR(3) = quantolgm1f_lgm2f.foreign_meanR_r;
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
%             tiny = zeros(3,3);
%             lastExpScale = zeros(3,3);
%             newExpScale  = zeros(3,3);
%             
%             for i=1:3
%                 for j=1:3
%                     if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end
%                     lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
%                 end
%             end
            
  
           lastU  = from;
           
           variances = zeros(3,3);
           correl =ones(3,3);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end
                if (i< alphaSize) vol(3) = sig(3).quote(i);else vol(3) = sig(3).quote(alphaSize);end
                
                if (i< alphaSize) correl(1,2) = corr_drds.quote(i);else correl(1,2) = corr_drds.quote(alphaSize);end
                if (i< alphaSize) correl(1,3) = corr_drfr.quote(i);else correl(1,3) = corr_drfr.quote(alphaSize);end
                if (i< alphaSize) correl(2,3) = corr_dsfr.quote(i);else correl(2,3) = corr_dsfr.quote(alphaSize);end
                
                correl(2,1) = correl(1,2);
                correl(3,1) = correl(1,3);
                correl(3,2) = correl(2,3);
                
                for j=1:3
                    for k=1:3
%                         newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
%                         if(tiny(j,k)~=1)
%                             scale =newExpScale(j,k)-lastExpScale(j,k);
%                         else
%                             scale = U - lastU;
%                         end
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
%                         lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
%            for j=1:3
%                 for k=1:3
%                     if(tiny(j,k)~= 1)
%                         variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
%                     end
%                 end
%             end
           
        end
        
        function out = LocalVariance_H2_Alpha2(quantolgm1f_lgm2f,from,to,sigmaTimes,dH1Values, dH2Values, sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1.quote = dH1Values;
           dH1.tenor = sigmaTimes;
           
           dH2.quote = dH2Values;
           dH2.tenor = sigmaTimes;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH1Value = dH1.quote(i);else dH1Value = dH1.quote(sigmaSize);end;
                if (i< sigmaSize) dH2Value = dH2.quote(i);else dH2Value = dH2.quote(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + quantolgm1f_lgm2f.H(lastU,dH1)*quantolgm1f_lgm2f.H(lastU,dH2)*correl*vol1*vol2*(U - lastU);
                variances = variances + dH1Value*quantolgm1f_lgm2f.H(lastU,dH2)*correl*vol1*vol2*0.5*(U - lastU)*(U - lastU);
                variances = variances + quantolgm1f_lgm2f.H(lastU,dH1)*dH2Value*correl*vol1*vol2*0.5*(U - lastU)*(U - lastU);
                variances = variances + dH1Value*dH2Value*correl*vol1*vol2*1.0/3.0*(U - lastU)*(U - lastU)*(U - lastU);
                
                
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = LocalVariance_H_Alpha2(quantolgm1f_lgm2f,from,to,sigmaTimes,dH1Values,sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1.quote = dH1Values;
           dH1.tenor = sigmaTimes;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH = dH1.quote(i);else dH = dH1.quote(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + quantolgm1f_lgm2f.H(lastU,dH1) * correl * vol1 * vol2 * (U - lastU);
                variances = variances + 0.5 * dH * correl * vol1 * vol2 * (U - lastU) * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = LocalVariance_Alpha2(quantolgm1f_lgm2f,from,to,sigmaTimes,sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + correl* vol1 * vol2 * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        
%         function out = GeneralVariance_HWHW(quantolgm1f_lgm2f,from,to,T1,T2,MRS1,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
%             % Variance in [from,to] of 1/a*(1-e^{-a(T1-s)})*1/b*(1-e^{-b(T2-s)})sigma1(s)sigma2(s)
%             sigmaSize = length(sigmaValue1);
%             
%             for i=1:length(sigmaTimes)
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:length(sigmaTimes)
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%             
%             
%             newExpScale = zeros(4,1);
%             lastExpScale = zeros(4,1);
%             variances = zeros(4,1);
%             
%             lastU  = from;
%             lastExpScale(2) = exp(MRS1*lastU);
%             lastExpScale(3) = exp(MRS2*lastU);
%             lastExpScale(4) = exp((MRS1 + MRS2)*lastU);
%             
%             for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end;
%                 if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end;
%                 if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end;
%                 if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end;
%                 
%                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
%                 
%                 newExpScale(2) = exp(MRS1*U);
%                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
%                 lastExpScale(2) = newExpScale(2);
%                 
%                 newExpScale(3) = exp(MRS2*U);
%                 variances(3) = variances(3) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(3)-lastExpScale(3));
%                 lastExpScale(3) = newExpScale(3);
%                 
%                 newExpScale(4) = exp((MRS1+MRS2)*U);
%                 variances(4) = variances(4) + corr12 * sig1 * sig2 * (1.0/(MRS1+MRS2)) * (newExpScale(4)-lastExpScale(4));
%                 lastExpScale(4) = newExpScale(4);
%                 
%                 lastU = U;
%                 
%             end
%             
%             value = (1.0/ MRS1)*(1.0/MRS2)*variances(1);
%             value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/ MRS1)*(1.0/MRS2)*variances(2);
%             value = value + -1.0*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(3);
%             value = value +  1.0*exp(-1.0*MRS1*T1)*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(4);
% 
%             out  = value;
%         end
        
%         function out = GeneralVariance_HWHW_type2(quantolgm1f_lgm2f,from,to,T1,T2,MRS1,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
%             % Variance in [a,b] of 1/a*(1-e^{-a(T1-s)})*e^{-b(T2-s)}*sigma1(s)sigma2(s)
%             sigmaSize = length(sigmaValue1);
%             
%             for i=1:length(sigmaTimes)
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:length(sigmaTimes)
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%             
%             
%             newExpScale = zeros(4,1);
%             lastExpScale = zeros(4,1);
%             variances = zeros(4,1);
%             
%             lastU  = from;
%             lastExpScale(2) = exp(MRS1*lastU);
%             lastExpScale(3) = exp(MRS2*lastU);
%             lastExpScale(4) = exp((MRS1 + MRS2)*lastU);
%             
%             for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end;
%                 if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end;
%                 if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end;
%                 if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end;
%                 
% %                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
%                 
% %                 newExpScale(2) = exp(MRS1*U);
% %                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
% %                 lastExpScale(2) = newExpScale(2);
%                 
%                 newExpScale(3) = exp(MRS2*U);
%                 variances(3) = variances(3) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(3)-lastExpScale(3));
%                 lastExpScale(3) = newExpScale(3);
%                 
%                 newExpScale(4) = exp((MRS1+MRS2)*U);
%                 variances(4) = variances(4) + corr12 * sig1 * sig2 * (1.0/(MRS1+MRS2)) * (newExpScale(4)-lastExpScale(4));
%                 lastExpScale(4) = newExpScale(4);
%                 
%                 lastU = U;
%                 
%             end
%             
% %             value = (1.0/ MRS1)*(1.0/MRS2)*variances(1);
% %             value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/ MRS1)*(1.0/MRS2)*variances(2);
%             value =  +1.0*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*variances(3);
%             value = value +  -1.0*exp(-1.0*MRS1*T1)*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*variances(4);
% 
%             out  = value;
%         end
        
%         function out = GeneralVariance_HWLGM(quantolgm1f_lgm2f,from,to,T1,T2,MRS1,sigmaTimes,dH1Values,sigmaValue1,sigmaValue2,corrValue)
%             % Variance in [from,to] of 1/a*(1-e^{-a(T1-s)})*(H(T2)-H(s))*sigma1(s)sigma2(s)
%             
%             dH1.quote = dH1Values;
%             dH1.tenor = sigmaTimes;
%             
%             sigmaSize = length(sigmaValue1);
%             
%             for i=1:length(sigmaTimes)
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:length(sigmaTimes)
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%             
%             
%             newExpScale = zeros(4,1);
%             lastExpScale = zeros(4,1);
%             variances = zeros(4,1);
%             
%             lastU  = from;
%             lastExpScale(2) = exp(MRS1*lastU);
%             lastExpScale(3) = exp(MRS1*lastU);
%             lastExpScale(4) = exp(MRS1*lastU);
%             
%             for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end
%                 if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
%                 if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
%                 if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
%                 if (i< sigmaSize) dH = dH1Values(i);else dH = dH1Values(sigmaSize);end
%                 
%                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
%                 
%                 newExpScale(2) = exp(MRS1*U);
%                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
%                 lastExpScale(2) = newExpScale(2);
%                 
%                 newExpScale(3) = exp(MRS1*U);
%                 variances(3) = variances(3) + quantolgm1f_lgm2f.H(lastU,dH1) * corr12 * sig1 * sig2 * (U - lastU);
%                 variances(3) = variances(3) + dH * corr12 * sig1 * sig2 * 0.5 * (U - lastU) * (U - lastU);
%                 lastExpScale(3) = newExpScale(3);
%                 
%                 newExpScale(4) = exp(MRS1*U);
%                 variances(4) = variances(4) + quantolgm1f_lgm2f.H(lastU,dH1) * corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
%                 variances(4) = variances(4) + dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (U*newExpScale(4)-lastU*lastExpScale(4));
%                 variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
%                 variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * lastU * (newExpScale(4)-lastExpScale(4));
%                 lastExpScale(4) = newExpScale(4);
%                 
%                 lastU = U;
%                 
%             end
%             
%             value = quantolgm1f_lgm2f.H(T2,dH1)*(1.0/MRS1)*variances(1);
%             value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/MRS1)*quantolgm1f_lgm2f.H(T2,dH1)*variances(2);
%             value = value + -1.0*(1.0/MRS1)*variances(3);
%             value = value +  1.0*exp(-1.0*MRS1*T1)*(1.0/MRS1)*variances(4);
% 
%             out  = value;
%         end
        
%         function out = GeneralVariance_HWLGM_type2(quantolgm1f_lgm2f,from,to,T1,T2,MRS1,sigmaTimes,dH1Values,sigmaValue1,sigmaValue2,corrValue)
%         % Variance in [a,b] of e^{-a(T1-s)}*(H(T2)-H(s))*sigma1(s)sigma2(s)
%             
%             dH1.quote = dH1Values;
%             dH1.tenor = sigmaTimes;
%             
%             sigmaSize = length(sigmaValue1);
%             
%             for i=1:length(sigmaTimes)
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:length(sigmaTimes)
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%             
%             
%             newExpScale = zeros(4,1);
%             lastExpScale = zeros(4,1);
%             variances = zeros(4,1);
%             
%             lastU  = from;
%             lastExpScale(2) = exp(MRS1*lastU);
%             lastExpScale(3) = exp(MRS1*lastU);
%             lastExpScale(4) = exp(MRS1*lastU);
%             
%             for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end
%                 if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
%                 if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
%                 if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
%                 if (i< sigmaSize) dH = dH1Values(i);else dH = dH1Values(sigmaSize);end
%                 
% %                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
%                 
%                 newExpScale(2) = exp(MRS1*U);
%                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
%                 lastExpScale(2) = newExpScale(2);
%                 
% %                 newExpScale(3) = exp(MRS1*U);
% %                 variances(3) = variances(3) + quantolgm1f_lgm2f.H(lastU,dH1) * corr12 * sig1 * sig2 * (U - lastU);
% %                 variances(3) = variances(3) + dH * corr12 * sig1 * sig2 * 0.5 * (U - lastU) * (U - lastU);
% %                 lastExpScale(3) = newExpScale(3);
%                 
%                 newExpScale(4) = exp(MRS1*U);
%                 variances(4) = variances(4) + quantolgm1f_lgm2f.H(lastU,dH1) * corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
%                 variances(4) = variances(4) + dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (U*newExpScale(4)-lastU*lastExpScale(4));
%                 variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
%                 variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * lastU * (newExpScale(4)-lastExpScale(4));
%                 lastExpScale(4) = newExpScale(4);
%                 
%                 lastU = U;
%                 
%             end
%             
% %             value = quantolgm1f_lgm2f.H(T2,dH1)*(1.0/MRS1)*variances(1);
% %             value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/MRS1)*quantolgm1f_lgm2f.H(T2,dH1)*variances(2);
%             value = +1.0*exp(-1.0*MRS1*T1)*quantolgm1f_lgm2f.H(T2,dH1)*variances(2);
%             
% %             value = value + -1.0*(1.0/MRS1)*variances(3);
%             value = value +  -1.0*exp(-1.0*MRS1*T1)*variances(4);
% 
%             out  = value;
%         end
        
%         function out = GeneralVariance_BSHW(quantolgm1f_lgm2f,from,to,T2,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
%             % Variance in [from,to] of 1/a*(1-e^{-a(T1-s)})sigma1(s)sigma2(s)
%             
%             sigmaSize = length(sigmaValue1);
%             
%             for i=1:length(sigmaTimes)
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:length(sigmaTimes)
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%             
%             
%             newExpScale = zeros(2,1);
%             lastExpScale = zeros(2,1);
%             variances = zeros(2,1);
%             
%             lastU  = from;
%             lastExpScale(2) = exp(MRS2*lastU);
%             
%             for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end
%                 if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
%                 if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
%                 if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
%                 
%                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
%                 
%                 newExpScale(2) = exp(MRS2*U);
%                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(2)-lastExpScale(2));
%                 lastExpScale(2) = newExpScale(2);
%                 
%                 lastU = U;
%                 
%             end
%             
%             value = (1.0/MRS2)*variances(1);
%             value = value + -1.0*exp(-1.0*MRS2*T2)*(1.0/MRS2)*variances(2);
%             
%             out  = value;
%         end
        
%         function out = GeneralVariance_BSHW_type2(quantolgm1f_lgm2f,from,to,T2,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
%             % Variance in [a,b] of (e^{-a(T1-s)})sigma1(s)sigma2(s)
%             
%             sigmaSize = length(sigmaValue1);
%             
%             for i=1:length(sigmaTimes)
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:length(sigmaTimes)
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%             
%             
%             newExpScale = zeros(2,1);
%             lastExpScale = zeros(2,1);
%             variances = zeros(2,1);
%             
%             lastU  = from;
%             lastExpScale(2) = exp(MRS2*lastU);
%             
%             for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end
%                 if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
%                 if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
%                 if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
%                 
% %                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
%                 
%                 newExpScale(2) = exp(MRS2*U);
%                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(2)-lastExpScale(2));
%                 lastExpScale(2) = newExpScale(2);
%                 
%                 lastU = U;
%                 
%             end
%             
% %             value = (1.0/MRS2)*variances(1);
%             value = +1.0*exp(-1.0*MRS2*T2)*variances(2);
%             
%             out  = value;
%         end
        
        function out = GeneralVariance_LGMLGM(quantolgm1f_lgm2f,a,b,T1,T2,sigmaTimes,dH1Values,dH2Values,Alpha1,Alpha2,corrValues)
            % Variance in [a,b] of (H(T1)-H(s))(H(T2)-H(s))alpha1(s)alpha2(s)
            dH1.quote = dH1Values;
            dH1.tenor = sigmaTimes;
            
            dH2.quote = dH2Values;
            dH2.tenor = sigmaTimes;
            
            variances = zeros(4,1);
            variances(1) = quantolgm1f_lgm2f.LocalVariance_Alpha2(a,b,sigmaTimes, Alpha1,Alpha2,corrValues);
            variances(1) = variances(1) * quantolgm1f_lgm2f.H(T1,dH1) * quantolgm1f_lgm2f.H(T2,dH2);
            
            variances(2) = quantolgm1f_lgm2f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH1Values,Alpha1,Alpha2,corrValues);
            variances(2) = variances(2) * -1.0* quantolgm1f_lgm2f.H(T2,dH2);
            
            variances(3) = quantolgm1f_lgm2f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH2Values,Alpha1,Alpha2,corrValues);
            variances(3) = variances(3) * -1.0* quantolgm1f_lgm2f.H(T1,dH1);
            
            variances(4) = quantolgm1f_lgm2f.LocalVariance_H2_Alpha2(a,b,sigmaTimes,dH1Values,dH2Values,Alpha1,Alpha2,corrValues);
            
            value = variances(1) + variances(2) + variances(3) + variances(4);
            out = value;
        end
        
        function out = GeneralVariance_BSLGM(quantolgm1f_lgm2f,a,b,T1,sigmaTimes,dH1Values,Alpha1,Alpha2,corrValues)
            % Variance in [a,b] of (H(T1)-H(s))alpha1(s)alpha2(s)
            dH1.quote = dH1Values;
            dH1.tenor = sigmaTimes;
            
            variances = zeros(2,1);
            variances(1) = quantolgm1f_lgm2f.LocalVariance_Alpha2(a,b,sigmaTimes, Alpha1,Alpha2,corrValues);
            variances(1) = variances(1) * quantolgm1f_lgm2f.H(T1,dH1);
            
            variances(2) = quantolgm1f_lgm2f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH1Values,Alpha1,Alpha2,corrValues);
            variances(2) = variances(2) * -1.0;
            
            value = variances(1) + variances(2);
            out = value;
        end
        
        function out = CMSDigitalArgDate(quantolgm1f_lgm2f,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           to = toDate.DateDiff(valueDate)/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           tauIn = tau;
           if abs(tenor1- 0.25) < 1e-8
                tauIn = tenor1;
           end
           
           p_nN1 = quantolgm1f_lgm2f.PV01Date(valueDate,toDate,tN1Date,tauIn);
           
           df_Tn1 = quantolgm1f_lgm2f.DF(to);
           df_TN1 = quantolgm1f_lgm2f.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = quantolgm1f_lgm2f.C_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1,y_nN1);
           g_nN1 = quantolgm1f_lgm2f.G_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1);
%            h_nTerm1 = quantolgm1f_lgm2f.H_nTermDate(valueDate,toDate,tpDate);
           % h_nTerm1 is replaced with i_Term & i_Tn_tilda as in the
           % following discussion
           
           h_nTerm1(1) = 0.0;
           
           varCovar = quantolgm1f_lgm2f.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           tpTime = tpDate.DateDiff(valueDate);
           % additional drift correction term from foreign Term measure USD to KRW domestic Term 
           i_Term = I_Term(quantolgm1f_lgm2f,0,to,tpTime/365.0);
           i_Tn_tilda = I_Tn_tilda(quantolgm1f_lgm2f,0,to,to);
           j_x_tilda = J_x_tilda(quantolgm1f_lgm2f,0,to,to);
           
%            additionalDrift = 0;
           additionalDrift = c_nN1(1) *(i_Term + i_Tn_tilda + j_x_tilda);
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.fwdCMS = y_nN1;
           out.mu = y_nN1 + driftTerm + additionalDrift;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
        end
        
        function out =computeBSLGM1FLGM2FVariance(quantolgm1f_lgm2f,a,b,T,corr_drds,corr_drfr,corr_dsfr,corr_drx,corr_dsx,corr_frx, ...
                                  fxBlackVol, domestic_dH_r,domestic_dH_s, domestic_Alpha_r,domestic_Alpha_s,...
                                  foreign_dH_r, foreign_Alpha_r)
                 
          % synchronize model parameters time-steps start
            corr_drds_Tenor = corr_drds.tenor;
            corr_drds_Quote = corr_drds.quote;
            
            corr_drfr_Tenor = corr_drfr.tenor;
            corr_drfr_Quote = corr_drfr.quote;
            
            corr_dsfr_Tenor = corr_dsfr.tenor;
            corr_dsfr_Quote = corr_dsfr.quote;
            
            corr_drx_Tenor = corr_drx.tenor;
            corr_drx_Quote = corr_drx.quote;
            
            corr_dsx_Tenor = corr_dsx.tenor;
            corr_dsx_Quote = corr_dsx.quote;
            
            corr_frx_Tenor = corr_frx.tenor;
            corr_frx_Quote = corr_frx.quote;
            
            
            fxBlackVolTenor = fxBlackVol.tenor;
            fxBlackVolQuote = fxBlackVol.quote;

            domestic_dH_r_Tenor = domestic_dH_r.tenor;
            domestic_dH_r_Quote = domestic_dH_r.quote;

            domestic_dH_s_Tenor = domestic_dH_s.tenor;
            domestic_dH_s_Quote = domestic_dH_s.quote;
            
            domestic_Alpha_r_Tenor = domestic_Alpha_r.tenor;
            domestic_Alpha_r_Quote = domestic_Alpha_r.quote;

            domestic_Alpha_s_Tenor = domestic_Alpha_s.tenor;
            domestic_Alpha_s_Quote = domestic_Alpha_s.quote;

            foreign_dH_r_Tenor = foreign_dH_r.tenor;
            foreign_dH_r_Quote = foreign_dH_r.quote;

            foreign_Alpha_r_Tenor = foreign_Alpha_r.tenor;
            foreign_Alpha_r_Quote = foreign_Alpha_r.quote;

            


            sigmaTimes = unique(union([corr_drds_Tenor(:);corr_drfr_Tenor(:);corr_dsfr_Tenor(:);...
                         corr_drx_Tenor(:);corr_dsx_Tenor(:);corr_frx_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:);foreign_Alpha_r_Tenor(:); ...
                         domestic_dH_r_Tenor(:);domestic_dH_s_Tenor(:);foreign_dH_r_Tenor(:)],fxBlackVolTenor));


            sigmaTimes = sort(sigmaTimes,'ascend');

            corr_drds_Values = zeros(length(sigmaTimes),1);
            corr_drfr_Values = zeros(length(sigmaTimes),1);
            corr_dsfr_Values = zeros(length(sigmaTimes),1);
            corr_drx_Values = zeros(length(sigmaTimes),1);
            corr_dsx_Values = zeros(length(sigmaTimes),1);
            corr_frx_Values = zeros(length(sigmaTimes),1);
            

            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            foreign_Alpha_r_Values = zeros(length(sigmaTimes),1);
            foreign_dH_r_Values = zeros(length(sigmaTimes),1);
            
            fxBlackVolValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                corr_drfr_Values(k) = H_interpolation(corr_drfr_Tenor, corr_drfr_Quote,sigmaTimes(k),0);
                corr_dsfr_Values(k) = H_interpolation(corr_dsfr_Tenor, corr_dsfr_Quote,sigmaTimes(k),0);
                corr_drx_Values(k) = H_interpolation(corr_drx_Tenor, corr_drx_Quote,sigmaTimes(k),0);
                corr_dsx_Values(k) = H_interpolation(corr_dsx_Tenor, corr_dsx_Quote,sigmaTimes(k),0);
                corr_frx_Values(k) = H_interpolation(corr_frx_Tenor, corr_frx_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_Alpha_r_Values(k) = H_interpolation(foreign_Alpha_r_Tenor, foreign_Alpha_r_Quote,sigmaTimes(k),0);
                foreign_dH_r_Values(k) = H_interpolation(foreign_dH_r_Tenor, foreign_dH_r_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                fxBlackVolValues(k) = H_interpolation(fxBlackVolTenor, fxBlackVolQuote,sigmaTimes(k),0);
            end
            
            % synchronize model parameters time-steps end
            % dummy correlation
            correl_1 = ones(length(sigmaTimes),1);
             
            var_x_x = quantolgm1f_lgm2f.LocalVariance_Alpha2(a,b,sigmaTimes,fxBlackVolValues,fxBlackVolValues,correl_1);
            
            var_fr_fr = quantolgm1f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,foreign_dH_r_Values,foreign_dH_r_Values, ...
                                        foreign_Alpha_r_Values,foreign_Alpha_r_Values,correl_1);
                                    
%             var_fr_fr = quantolgm1f_lgm2f.GeneralVariance_HWHW(a,b,T,T,foreign_meanR_r,foreign_meanR_r,sigmaTimes, ...
%                                         foreign_vol_r_Values,foreign_vol_r_Values,correl_1);
                                                     
            var_dr_dr = quantolgm1f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_r_Values,domestic_dH_r_Values, ...
                                                         domestic_Alpha_r_Values,domestic_Alpha_r_Values,correl_1);
                                                     
            var_ds_ds = quantolgm1f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_s_Values,domestic_dH_s_Values, ...
                                                         domestic_Alpha_s_Values,domestic_Alpha_s_Values,correl_1);
           % 1st= sign in techNote , 2nd = HW bond vol? -1,1
            covar_fr_x =  1.0*-1.0*quantolgm1f_lgm2f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
                                            foreign_dH_r_Values,foreign_Alpha_r_Values,fxBlackVolValues,corr_frx_Values);
                                        
%             covar_fr_x =  1.0*-1.0*quantolgm1f_lgm2f.GeneralVariance_BSHW(a,b,T,foreign_meanR_r,sigmaTimes, ...
%                                             foreign_vol_r_Values,fxBlackVolValues,corr_frx_Values);
                                        
            covar_dr_x = -1.0*-1.0*quantolgm1f_lgm2f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
                                            domestic_dH_r_Values,domestic_Alpha_r_Values,fxBlackVolValues,corr_drx_Values);
                                        
            covar_ds_x = -1.0*-1.0*quantolgm1f_lgm2f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
                                            domestic_dH_s_Values,domestic_Alpha_s_Values,fxBlackVolValues,corr_dsx_Values);
                                        
            covar_dr_fr =  -1.0*1.0*quantolgm1f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_r_Values,foreign_dH_r_Values,...
                                            domestic_Alpha_r_Values,foreign_Alpha_r_Values,corr_drfr_Values);
                                        
            covar_ds_fr =  -1.0*1.0*quantolgm1f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_s_Values,foreign_dH_r_Values,...
                                            domestic_Alpha_s_Values,foreign_Alpha_r_Values,corr_dsfr_Values);
                                        
%             covar_dr_fr =  -1.0*1.0*quantolgm1f_lgm2f.GeneralVariance_HWLGM(a,b,T,T,foreign_meanR_r,sigmaTimes,...
%                                             domestic_dH_r_Values,domestic_Alpha_r_Values,foreign_vol_r_Values,corr_drfr_Values);
%                                         
%             covar_ds_fr =  -1.0*1.0*quantolgm1f_lgm2f.GeneralVariance_HWLGM(a,b,T,T,foreign_meanR_r,sigmaTimes,...
%                                             domestic_dH_s_Values,domestic_Alpha_s_Values,foreign_vol_r_Values,corr_dsfr_Values);
                                        
            covar_dr_ds =  1.0*1.0*0.0;  % corr_drds is zero
                                                         
            value = var_x_x + var_fr_fr + var_dr_dr + var_ds_ds + 2.0*(covar_fr_x + covar_dr_x + covar_ds_x + covar_dr_fr + covar_ds_fr + covar_dr_ds);                                         
            
            out = value;                                         
                                                     
            
        end
        
        function out = FindFXBlackVol(quantolgm1f_lgm2f,index,params)
            
            expiry = params.fxImpVol.tenor(index);
            impVol = params.fxImpVol.quote(index);
            blackVariance = impVol*impVol*expiry;
            optParams.blackVariance = blackVariance;
            
            optParams.params = params;
            optParams.index = index;
            modelParamSize = 1;
            tvar = zeros(modelParamSize,1);
            lb = zeros(modelParamSize,1);
            ub = zeros(modelParamSize,1);
            
            tvar(1) = params.fxImpVol.quote(index);
            lb(1)= 0.0001;
            ub(1)= 10;
            
            %options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
            options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
            numOfEquation =1;
            x=  zeros(numOfEquation,1);
            [ret, popt, info, covar]=levmar('TargetFunctionBSLGM1FLGM2FVariance',tvar,x, 200, options,'unc',quantolgm1f_lgm2f,optParams);
            
            out.fxBlackVol_index = popt(1);
            out.re = TargetFunctionBSLGM1FLGM2FVariance(popt,quantolgm1f_lgm2f,optParams);
%             params.fxBlackVol.quote(index) = popt(1);

        end
        
        function out = CalibrateBSLGM1FLGM2FVol(quantolgm1f_lgm2f, paramsIn)
 
                expirySize = length(paramsIn.fxImpVol.tenor);
                out.fxBlackVol.quote = zeros(expirySize,1);
                out.fxBlackVol.tenor = paramsIn.fxImpVol.tenor;
                out.re =  zeros(expirySize,1);
                out.rmseTotal = 0;
                for i=1:expirySize
                    index = i;
                    BootstrapOut = quantolgm1f_lgm2f.FindFXBlackVol(index,paramsIn);
                    out.fxBlackVol.quote(i) = BootstrapOut.fxBlackVol_index;
                    params.fxBlackVol.quote(i) = BootstrapOut.fxBlackVol_index;
                    out.re(i) = BootstrapOut.re;
                    out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                end
                out.rmseTotal = out.rmseTotal/expirySize;
                
                out.rmseTotal = sqrt(out.rmseTotal);
                 
                
        end
        
        
    end
    
end



In [ ]:
classdef QuantoLGM1F_LGM1F_KTB < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        numOfFactor;
        numOfFactor_d;
        numOfFactor_f;
        
        corr_df;
        
        fxBlackVol;
        domesticModel;
        foreignModel;
        
        domestic_Alpha;
        foreign_Alpha;
        domestic_dH;
        foreign_dH;
    end
    
    methods
        function quantolgm1f_lgm1f = QuantoLGM1F_LGM1F_KTB(params)
            if nargin > 0
                
                quantolgm1f_lgm1f.domesticModel = params('domesticModel');
                quantolgm1f_lgm1f.foreignModel  = params('foreignModel');
                
                quantolgm1f_lgm1f.zeroCurve = params.zeroCurve;
                
                quantolgm1f_lgm1f.numOfFactor_d = quantolgm1f_lgm1f.domesticModel.modelParams('numOfFactor');
                quantolgm1f_lgm1f.numOfFactor_f = quantolgm1f_lgm1f.foreignModel.modelParams('numOfFactor');
                
                quantolgm1f_lgm1f.numOfFactor = quantolgm1f_lgm1f.numOfFactor_d + quantolgm1f_lgm1f.numOfFactor_f;
%                 lgm1F.numOfFactor = ch.numOfFactor_d + ch.numOfFactor_f + 1;
                
                sizeN = quantolgm1f_lgm1f.numOfFactor;
                
                fxBlackVol.tenor = lgm1F.domesticModel.Alpha1.tenor;
                fxBlackVol.quote = zeros(length(fxBlackVol.tenor),1);
                
                
                % synchronize model parameters time-steps start
                domestic_AlphaTenor = quantolgm1f_lgm1f.GetChildModel('d').Alpha1.tenor;
                domestic_AlphaQuote = quantolgm1f_lgm1f.GetChildModel('d').Alpha1.quote;

                foreign_AlphaTenor = quantolgm1f_lgm1f.GetChildModel('f').Alpha1.tenor;
                foreign_AlphaQuote = quantolgm1f_lgm1f.GetChildModel('f').Alpha1.quote;

                domestic_dHTenor = quantolgm1f_lgm1f.GetChildModel('d').dH1.tenor;
                domestic_dHQuote = quantolgm1f_lgm1f.GetChildModel('d').dH1.quote;

                foreign_dHTenor = quantolgm1f_lgm1f.GetChildModel('f').dH1.tenor;
                foreign_dHQuote = quantolgm1f_lgm1f.GetChildModel('f').dH1.quote;

                sigmaTimes = sort(union(domestic_AlphaTenor,foreign_AlphaTenor,domestic_dHTenor,foreign_dHTenor),'ascend');

                domestic_AlphaValues = zeros(length(sigmaTimes),1);
                foreign_AlphaValues = zeros(length(sigmaTimes),1);
                domestic_dHValues = zeros(length(sigmaTimes),1);
                foreign_dHValues = zeros(length(sigmaTimes),1);


                for k=1:length(sigmaTimes)
                    domestic_AlphaValues(k) = H_interpolation(domestic_AlphaTenor, domestic_AlphaQuote,sigmaTimes(k),0);
                    foreign_AlphaValues(k) = H_interpolation(foreign_AlphaTenor, foreign_AlphaQuote,sigmaTimes(k),0);
                    domestic_dHValues(k) = H_interpolation(domestic_dHTenor, domestic_dHQuote,sigmaTimes(k),0);
                    foreign_dHValues(k) = H_interpolation(foreign_dHTenor, foreign_dHQuote,sigmaTimes(k),0);
                end
                
                domestic_Alpha.tenor = sigmaTimes;
                domestic_Alpha.quote = domestic_AlphaValues;
                
                foreign_Alpha.tenor = sigmaTimes;
                foreign_Alpha.quote = foreign_AlphaValues;
                
                domestic_dH.tenor = sigmaTimes;
                domestic_dH.quote = domestic_dHValues;
                
                foreign_dH.tenor = sigmaTimes;
                foreign_dH.quote = foreign_dHValues;
                
                %end
                
            end
        end
        
        function out = GetChildModel(quantolgm1f_lgm1f,type)
            if type == 'd'
                out = quantolgm1f_lgm1f.domesticModel;
            elseif type == 'f'
                out = quantolgm1f_lgm1f.foreignModel;
            end
        
        end
        
        function out =  Libor_SimpleDate(quantolgm1f_lgm1f,type,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           % domestic Libor
           if type == 'd'
               domesticModelStatesT = modelStatesT(1,:);
               out = quantolgm1f_lgm1f.GetChildModel('d').Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,domesticModelStatesT);
               return; 
           end
           
           % foreign Libor in domestic Terminal measure
           
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = quantolgm1f_lgm1f.discountFactorForeign(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = quantolgm1f_lgm1f.discountFactorForeign(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
%          function out =  CMS_PSA_SimpleDate(lgm1F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
%            %CMS
%            tau = 0.25;
%            to      = observeTime/365.0;
%            toDate = valueDate.AddDate(observeTime,'day');
%            tNDate = toDate.AddDate(tenor,'year');
%            tN   =  tNDate.DateDiff(valueDate)/365.0;
%            
%            numMatDate = valueDate.AddDate(numMatTime,'day');
%            
%            p_nN = lgm1F.PV01Date(valueDate,toDate,tNDate,tau);
%            
%            df_Tn = lgm1F.DF(to);
%            df_TN = lgm1F.DF(tN);
%            y_nN = (df_Tn - df_TN)/p_nN;
%            
%            c_nN = lgm1F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
%            g_nN = lgm1F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
%            h_nTerm = lgm1F.H_nTermDate(valueDate,toDate,numMatDate);
%            varCovar = lgm1F.LocalVariance(0,to);
%            
%            stateSize = size(modelStatesT,2);
%            cms_PCA_SimpleOut = zeros(1,stateSize);
%            
%            driftTerm = 0;
%            for i=1:1
%                for j=1:1
%                    driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
%                end
%            end
%            
%            for i=1:stateSize
%                cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i); 
%            end
%            
%            out = cms_PCA_SimpleOut;
%            
%             
%          end
         
        
         function out = fwdZcVarianceForeign(quantolgm1f_lgm1f, a, b, T1, T2)
            
            localVariance = zeros(3,1);
            measureChange = zeros(2,1);
            quantoChange = zeros(1,1);
            sigmaTimes = quantolgm1f_lgm1f.domestic_Alpha.tenor;
            foreign_AlphaValues = quantolgm1f_lgm1f.foreign_Alpha.quote;
            domestic_AlphaValues = quantolgm1f_lgm1f.domestic_Alpha.quote;
            foreign_dHValues = quantolgm1f_lgm1f.foreign_dH.quote;
            domestic_dHValues = quantolgm1f_lgm1f.domestic_dH.quote;
            
            localVariance(1) =  quantolgm1f_lgm1f.GetChildModel('f').HtT_r(a,T1) * ...
                                quantolgm1f_lgm1f.GetChildModel('f').HtT_r(a,T1) * ...
                                LocalVariance_Alpha2(a,b,sigmaTimes,foreign_AlphaValues,foreign_AlphaValues);
            
            localVariance(2) =  quantolgm1f_lgm1f.GetChildModel('f').HtT_r(a,b) * ...
                                quantolgm1f_lgm1f.GetChildModel('f').HtT_r(a,b) * ...
                                LocalVariance_Alpha2(a,b,sigmaTimes,foreign_AlphaValues,foreign_AlphaValues);

            localVariance(3) =  (quantolgm1f_lgm1f.GetChildModel('f').HtT_r(a,T1) - ...
                                quantolgm1f_lgm1f.GetChildModel('f').HtT_r(a,b)) * ...
                                LocalVariance_H_Alpha2(a,b,sigmaTimes,foreign_dHValues,foreign_AlphaValues,foreign_AlphaValues);
            
            
            measureChange(1) = (quantolgm1f_lgm1f.GetChildModel('f').HtT_r(a,T1) - ...
                                quantolgm1f_lgm1f.GetChildModel('f').HtT_r(a,b)) * ...
                                quantolgm1f_lgm1f.GetChildModel('d').HtT_r(a,T2) * ...
                                LocalVariance_Alpha2(a,b,sigmaTimes,foreign_AlphaValues,domestic_AlphaValues);
                            
            measureChange(2) = (quantolgm1f_lgm1f.GetChildModel('f').HtT_r(a,T1) - ...
                                quantolgm1f_lgm1f.GetChildModel('f').HtT_r(a,b)) * ...
                                LocalVariance_Alpha2(a,b,sigmaTimes,domestic_dHValues,foreign_AlphaValues,domestic_AlphaValues);
                                
            
            quantoChange(0) = 0.0;
            
            value = localVariance(1) - localVariance(2) - 2.0*localVariance(3);
            value = value -2.0 * quantolgm1f_lgm1f.corr_df * (measureChange(1) - measureChange(2));
            
            out = value;
            
         end
         
        function out = discountFactorForeign(quantolgm1f_lgm1f, startTime, endTime,numMatTime,modelStatesT)
           
            zcT = quantolgm1f_lgm1f.GetChildModel('f').DF(endTime/365.0);
           
            %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = quantolgm1f_lgm1f.GetChildModel('f').DF(startTime/365.0);
           beta = quantolgm1f_lgm1f.GetChildModel('f').Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = quantolgm1f_lgm1f.fwdZcVarianceForeign(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           for i=1:stateSize
%                discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(2,i));
           end
           
           out = discountFactorOut;
           
        end
        
        function out = discountPayoff(quantolgm1f_lgm1f,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = quantolgm1f_lgm1f.GetChildModel('d').DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            domesticModelStatesT = modelStatesT(1,:);
            discountT = quantolgm1f_lgm1f.GetChildModel('d').discountFactor(eventTime, numMatTime,numMatTime,domesticModelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(quantolgm1f_lgm1f,fromTime, toTime)
            
            out = eye(2,2);
        end
        
        function out = stateLocalVariance(quantolgm1f_lgm1f,fromTime, toTime)
            out = quantolgm1f_lgm1f.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(quantolgm1f_lgm1f,from,to)

            sig(1) = quantolgm1f_lgm1f.domestic_Alpha;
            sig(2) = quantolgm1f_lgm1f.foreign_Alpha;
            corr = quantolgm1f_lgm1f.corr_df;
            
            
            alphaSize = length(sig(1));
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i);
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i);
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = zeros(2,2);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                
                correl =ones(2,2);
                correl(1,2) = corr;
                correl(2,1) = corr;
                for j=1:2
                    for k=1:2
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = LocalVariance_H_Alpha2(quantolgm1f_lgm1f,from,to,sigmaTimes,dH1,sigmaValues1,sigmaValues2)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i);
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i);
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH = dH1.quote(i);else dH = dH1.quote(sigmaSize);end;
                
                variances = variances + quantolgm1f_lgm1f.H(lastU,dH1) * vol1 * vol2 * (U - lastU);
                variances = variances + dH * vol1 * vol2 * (U - lastU) * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = LocalVariance_Alpha2(quantolgm1f_lgm1f,from,to,sigmaTimes,sigmaValues1,sigmaValues2)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i);
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i);
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                
                variances = variances + vol1 * vol2 * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = dAlpha_Test(lgm1F,t,Alpha1)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha1.tenor)
                alpha = Alpha1.quote(i);
                if t<= Alpha1.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function out = Alpha_Test(lgm1F,t,Alpha1)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha1.tenor)
                alpha = Alpha1.quote(i) * Alpha1.quote(i);
                if t<= Alpha1.tenor(i)
                    sum = sum + alpha*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + alpha*(Alpha1.tenor(i)-prev_t);
                end
                prev_t = Alpha1.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + alpha * (t - Alpha1.tenor(i));
            end
            out = sum;
        end
        
        function out = dH_Test(lgm1F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = dH.quote(i);
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = dH.quote(i);
            end
            out = sum;
        end
        
        function out = H_Test(lgm1F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = sum + dH.quote(i)*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + dH.quote(i)*(dH.tenor(i)-prev_t);
                end
                prev_t = dH.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + dH.quote(i) * (t - dH.tenor(i));
            end
            out = sum;
        end
        
        function out = H(lgm1F,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
        end


        function beta = Beta(lgm1F,t,T)
            beta =[];
            beta(1) = lgm1F.H(T,lgm1F.dH1)-lgm1F.H(t,lgm1F.dH1);
        end
        
        function c_Test = C_Test(lgm1F,tx, tnr,tau)
            pv01_1 = lgm1F.PV01(tx,tx+tnr,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr,tau,pv01_1,fsr_1);
            c_Test = c_nN_1;
        end
        
        function c_nN = C_nN(lgm1F,tn,tN,tau,pnN,ynN)
            p = @lgm1F.DF;
            c_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm1F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
        function g_nN = G_nN(lgm1F,tn,tN,tau,pnN)
            p = @lgm1F.DF;
            g_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm1F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function h_nTerm = H_nTerm(lgm1F,tn,tTerm)
           h_nTerm =[];
           beta = lgm1F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function out = CMSDigitalArg(lgm1F,to,tp,tenor,tau)
           % probability that CMS rate > strike
           p_nN = lgm1F.PV01(to,to+tenor,tau);
           df_Tn = lgm1F.DF(to);
           df_TN = lgm1F.DF(to + tenor);
           y_nN = (df_Tn - df_TN)/p_nN;
           c_nN = lgm1F.C_nN(to,to + tenor,tau,p_nN,y_nN);
           g_nN = lgm1F.G_nN(to,to + tenor,tau,p_nN);
           h_nTerm = lgm1F.H_nTerm(to,tp);
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN(i) * (h_nTerm(j) + g_nN(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN(i) * c_nN(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigital(lgm1F,to,tp,tenor,tau,strike)
           cmsDigitalArg= lgm1F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(lgm1F,to,tp,tenor,tau,lower,upper)
            lower_value = lgm1F.CMSDigital(to,tp,tenor,tau,lower);
            upper_value = lgm1F.CMSDigital(to,tp,tenor,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(lgm1F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm1F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm1F.DF(to);
           df_TN1 = lgm1F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm1F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm1F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm1F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm1F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm1F.DF(to);
           df_TN2 = lgm1F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm1F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm1F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm1F.H_nTerm(to,tp);
           
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:1
               for j=1:1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:1
               for j=1:1
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(lgm1F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(lgm1F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = lgm1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSCMSSpreadCovar(lgm1F,c_nN,c_nN1,c_nN2,varCovar,varCovarS)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           covar = 0.0;
           for i=1:1
               for j=1:1
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           out = covar;
           
        end
        
        function out = CMSCMSSpreadDualDigital(lgm1F,to,tp,tenor,tenor1,tenor2,tau,strike,strikeS)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = lgm1F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           c_nN = cmsDigitalArg.c_nN;
           varCovar = cmsDigitalArg.varCovar;
           
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           varCovarS = cmsSpreadDigitalArg.varCovarS;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:1
               for j=1:1
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - strike)/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSCMSSpreadDualDigitalRange(lgm1F,to,tp,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,upperS);
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = CMSDigitalRangeAccrual(lgm1F,matu,tenor,tau,coupon,lower,upper,tbd)
           periodSum = @(Ta,Tb,tbd) sum(arrayfun(@(i) ...
               1/tbd*CMSDigitalRange(lgm1F,Ta+1/tbd*i,Tb,tenor,tau,lower,upper),...
               1:round((Tb-Ta)*tbd)));
           p = @lgm1F.DF;
           dcf = 1.0;
           out.cashflowNpv = zeros(1,round(matu));
           out.price = 0.0;
           for i=1:round(matu)
               out.cashflowNpv(i) = periodSum((i-1)*0.25,i*0.25,tbd)*coupon*p(i*0.25)*NA;
               out.price = out.price + out.cashflowNpv(i);
           end
           out.cashflowNpv(i+1) = p(matu)*NA;
           out.price = out.price + p(matu)*NA;
        end
        
        function atmSwaption = ATMSwaption(lgm1F,tx,tnr)
            tau = 0.25;
            pv01 = lgm1F.PV01(tx,tx+tnr,tau);
            fsr  = lgm1F.Fsr(tx,tx+tnr,tau);
            c_nN = lgm1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = lgm1F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:1
                for j=1:1
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function cap = Cap(lgm1F,cap_strike,dt,Ti)
            idx_mat = round(Ti/dt);
            
            HN = 0.0;
            Hn = 0.0;
            sAlpha = 0.0;
            vol = 0.0;
            d1 = 0.0;
            
            p = @lgm1F.DF;
            model_cap = 0;
            for i=1:idx_mat-1
                Hn = lgm1F.H(i*dt,lgm1F.dH1);
                HN = lgm1F.H((i+1)*dt,lgm1F.dH1);
                sAlpha = lgm1F.Alpha_Test(i*dt,lgm1F.Alpha1);
                vol = (HN-Hn)*sqrt(sAlpha);
                d1 = log(p(i*dt)/(p((i+1)*dt)*(1 + cap_strike*dt)))/vol + 0.5*vol;
                model_cap = model_cap + p(i*dt)*H_ncdf(d1)-p((i+1)*dt)*(1 + cap_strike*dt)*H_ncdf(d1-vol);
            end
            cap = model_cap;
        end
        
%         function atmSwaption = modelSwaptionExpiryTenor(lgm1F,tx,tnr)
%             tau = 0.25;
%             pv01 = lgm1F.PV01(tx,tx+tnr,tau);
%             fsr  = lgm1F.Fsr(tx,tx+tnr,tau);
%             c_nN = lgm1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
%             variances = lgm1F.LocalVariance(0,tx);
%             totalVar = 0.0;
%             for i=1:1
%                 for j=1:1
%                     totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
%                 end
%             end
%             atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
%         end
        
        function impCorrel = ImpCorrel(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function spreadInfo = SpreadInfo(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  ck2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impVariance = ImpVariance(lgm1F,tx,tnr1)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function instFwdCorr = InsFwdCorr(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = (lgm1F.H_Test(tx + tnr1 +0.001,lgm1F.dH1)-lgm1F.H_Test(tx + tnr1,lgm1F.dH1))/0.001;
            dH_tT1(2) = (lgm1F.H_Test(tx + tnr1 +0.001,lgm1F.dH2)-lgm1F.H_Test(tx + tnr1,lgm1F.dH2))/0.001;
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = (lgm1F.H_Test(tx + tnr2 +0.001,lgm1F.dH1)-lgm1F.H_Test(tx + tnr2,lgm1F.dH1))/0.001;
            dH_tT2(2) = (lgm1F.H_Test(tx + tnr2 +0.001,lgm1F.dH2)-lgm1F.H_Test(tx + tnr2,lgm1F.dH2))/0.001;
            
            instVariances = (lgm1F.LocalVariance(0,tx+0.001)-lgm1F.LocalVariance(0,tx))/0.001;
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + dH_tT1(j)*dH_tT1(k)*instVariances(j,k);
                    var2 = var2 + dH_tT2(j)*dH_tT2(k)*instVariances(j,k);
                    covar = covar + dH_tT1(j)*dH_tT2(k)*instVariances(j,k);
                end
            end
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTest = InsFwdCorrTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH1);
            dH_tT1(2) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH2);
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH1);
            dH_tT2(2) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH2);
            
            dAlpha_t = zeros(2,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
            dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:1
                var1 = var1 + dH_tT1(j)*dH_tT1(j)*dAlpha_t(j)*dAlpha_t(j);
                var2 = var2 + dH_tT2(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
                covar = covar + dH_tT1(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCovarTestMatrix = InsFwdCovarTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCovarTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
                eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;

                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
            end
            
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
                eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;
                
                testLoadingD = dH_tT*eta;
                
                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';
                
            end
            
        end
        
        function deltaHMatrix = DeltaHTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                deltaHMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                
                dH_tT = zeros(length(newtnr),2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                deltaHMatrix = dH_tT;
            end
            
        end
        
        function etaMatrix = EtaTest2(lgm1F,tx)
            eta =  zeros(1,1);
            eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
%             eta(1,2) = 0;
%             eta(2,1) = 0;
            
            etaMatrix = eta;
            
        end
        
        function instFwdCovarTest = InsFwdCovarTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(1,1);
            dH_tT1(1) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH1);
%             dH_tT1(2) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH2);
            
            dH_tT2 = zeros(1,1);
            dH_tT2(1) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH1);
%             dH_tT2(2) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            for j=1:1
                covar = covar + dH_tT1(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdCovarTest = covar;
        end
        
        function instFwdSwapCovarTest = InsSwapFwdCovarTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(1,1);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
%             C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(1,1);
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
%             C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            for j=1:1
                covar = covar + C_tT1(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdSwapCovarTest = covar;
        end
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(1,1);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
%             C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(1,1);
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
%             C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for j=1:1
                var1 = var1 + C_tT1(j)*C_tT1(j)*dAlpha_t(j)*dAlpha_t(j);
                var2 = var2 + C_tT2(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
                covar = covar + C_tT1(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function out = RebonatoFwdCorr2(lgm1F,tx,tnr1,tnr2)
           tau = 0.25; 
           rho_inf = 0.35;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function out = RebonatoFwdCorr(lgm1F,tx,tnr1,tnr2)
           rho_inf = 0.35;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function coVariance = CoVariance(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            totalVar = zeros(1,1);
            for j=1:1
                for k=1:1
                    totalVar(1,1) = totalVar(1,1) + c_nN_1(j)*c_nN_2(k)*variances(j,k);
%                     totalVar(1,2) = totalVar(1,2) + c_nN_1(j)*c_nN_2(k)*variances(j,k);
%                     totalVar(2,1) = totalVar(2,1) + c_nN_2(j)*c_nN_1(k)*variances(j,k);
%                     totalVar(2,2) = totalVar(2,2) + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                end
            end
            coVariance =  totalVar;
        end
      

    end
    
end



In [ ]:
classdef QuantoLGM1F_LGM1F < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        numOfFactor;
        numOfFactor_d;
        numOfFactor_f;
        
        corr_df;
        
        fxBlackVol;
        domesticModel;
        foreignModel;
        
    end
    
    methods
        function lgm1F = QuantoLGM1F_LGM1F(params)
            if nargin > 0
                
                lgm1F.domesticModel = params('domesticModel');
                lgm1F.foreignModel  = params('foreignModel');
                
                lgm1F.zeroCurve = params.zeroCurve;
                
                lgm1F.numOfFactor_d = lgm1F.domesticModel.modelParams('numOfFactor');
                lgm1F.numOfFactor_f = lgm1F.foreignModel.modelParams('numOfFactor');
                
                lgm1F.numOfFactor = lgm1F.numOfFactor_d + lgm1F.numOfFactor_f;
%                 lgm1F.numOfFactor = ch.numOfFactor_d + ch.numOfFactor_f + 1;
                
                sizeN = lgm1F.numOfFactor;
                
                fxBlackVol.tenor = lgm1F.domesticModel.Alpha1.tenor;
                fxBlackVol.quote = zeros(length(fxBlackVol.tenor),1);
                
            end
        end
        
        function out = GetChildModel(lgm1F,type)
            if type == 'd'
                out = lgm1F.domesticModel;
            elseif type == 'f'
                out = lgm1F.foreignModel;
            end
        
        end
        
        function out = PiecewiseConst(lgm1F,T,inputTenor,inputQuote)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha1.tenor)
                alpha = Alpha1.quote(i);
                if t<= Alpha1.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function out = SimpleSRPrOverHedge(lgm1F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(lgm1F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(lgm1F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(lgm1F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN
                for i =1:pathSize
                    outVec(i) = lgm1F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm1F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  Libor_SimpleDate(lgm1F,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           %CMS
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = lgm1F.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = lgm1F.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
         function out =  CMS_PSA_SimpleDate(lgm1F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = lgm1F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = lgm1F.DF(to);
           df_TN = lgm1F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = lgm1F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = lgm1F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = lgm1F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = lgm1F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:1
               for j=1:1
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
         end
         
         function out = HtT_r(lgm1F,t,T)
            out =  lgm1F.H(T,lgm1F.dH1) - lgm1F.H(t,lgm1F.dH1); 
        end
        
         function out = fwdZcVariance(lgm1F, a, b, T1, T2)
            varMatrix = lgm1F.LocalVariance(a,b);
            
            localVariance = zeros(1,1);
            measureChange = zeros(1,1);
            
            localVariance(1) = (lgm1F.HtT_r(a,T1)*lgm1F.HtT_r(a,T1)-lgm1F.HtT_r(a,b)*lgm1F.HtT_r(a,b))*varMatrix(1,1);

            measureChange(1) = (lgm1F.HtT_r(a,T1)-lgm1F.HtT_r(a,b))*lgm1F.HtT_r(a,T2)*varMatrix(1,1);
           
            value = localVariance(1);
            measureChangeTerm = measureChange(1);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
         end
         
        function out = discountFactor(lgm1F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm1F.DF(endTime/365.0);
           
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = lgm1F.DF(startTime/365.0);
           beta = lgm1F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm1F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoff(lgm1F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm1F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm1F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(lgm1F,fromTime, toTime)
            
            out = eye(1,1);
        end
        
        function out = stateLocalVariance(lgm1F,fromTime, toTime)
            out = lgm1F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(lgm1F,from,to)

            sig(1) = lgm1F.Alpha1;
            alphaSize = length(lgm1F.Alpha1.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i);
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i);
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = zeros(1,1);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                correl =ones(1,1);
                for j=1:1
                    for k=1:1
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = dAlpha_Test(lgm1F,t,Alpha1)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha1.tenor)
                alpha = Alpha1.quote(i);
                if t<= Alpha1.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function out = Alpha_Test(lgm1F,t,Alpha1)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha1.tenor)
                alpha = Alpha1.quote(i) * Alpha1.quote(i);
                if t<= Alpha1.tenor(i)
                    sum = sum + alpha*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + alpha*(Alpha1.tenor(i)-prev_t);
                end
                prev_t = Alpha1.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + alpha * (t - Alpha1.tenor(i));
            end
            out = sum;
        end
        
        function out = dH_Test(lgm1F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = dH.quote(i);
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = dH.quote(i);
            end
            out = sum;
        end
        
        function out = H_Test(lgm1F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = sum + dH.quote(i)*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + dH.quote(i)*(dH.tenor(i)-prev_t);
                end
                prev_t = dH.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + dH.quote(i) * (t - dH.tenor(i));
            end
            out = sum;
        end
        
        function out = H(lgm1F,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
        end


        function beta = Beta(lgm1F,t,T)
            beta =[];
            beta(1) = lgm1F.H(T,lgm1F.dH1)-lgm1F.H(t,lgm1F.dH1);
        end
        
        function c_Test = C_Test(lgm1F,tx, tnr,tau)
            pv01_1 = lgm1F.PV01(tx,tx+tnr,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr,tau,pv01_1,fsr_1);
            c_Test = c_nN_1;
        end
        
        function c_nN = C_nN(lgm1F,tn,tN,tau,pnN,ynN)
            p = @lgm1F.DF;
            c_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm1F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
        function g_nN = G_nN(lgm1F,tn,tN,tau,pnN)
            p = @lgm1F.DF;
            g_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm1F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function h_nTerm = H_nTerm(lgm1F,tn,tTerm)
           h_nTerm =[];
           beta = lgm1F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function out = CMSDigitalArg(lgm1F,to,tp,tenor,tau)
           % probability that CMS rate > strike
           p_nN = lgm1F.PV01(to,to+tenor,tau);
           df_Tn = lgm1F.DF(to);
           df_TN = lgm1F.DF(to + tenor);
           y_nN = (df_Tn - df_TN)/p_nN;
           c_nN = lgm1F.C_nN(to,to + tenor,tau,p_nN,y_nN);
           g_nN = lgm1F.G_nN(to,to + tenor,tau,p_nN);
           h_nTerm = lgm1F.H_nTerm(to,tp);
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN(i) * (h_nTerm(j) + g_nN(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN(i) * c_nN(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigital(lgm1F,to,tp,tenor,tau,strike)
           cmsDigitalArg= lgm1F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(lgm1F,to,tp,tenor,tau,lower,upper)
            lower_value = lgm1F.CMSDigital(to,tp,tenor,tau,lower);
            upper_value = lgm1F.CMSDigital(to,tp,tenor,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(lgm1F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm1F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm1F.DF(to);
           df_TN1 = lgm1F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm1F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm1F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm1F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm1F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm1F.DF(to);
           df_TN2 = lgm1F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm1F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm1F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm1F.H_nTerm(to,tp);
           
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:1
               for j=1:1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:1
               for j=1:1
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(lgm1F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(lgm1F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = lgm1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSCMSSpreadCovar(lgm1F,c_nN,c_nN1,c_nN2,varCovar,varCovarS)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           covar = 0.0;
           for i=1:1
               for j=1:1
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           out = covar;
           
        end
        
        function out = CMSCMSSpreadDualDigital(lgm1F,to,tp,tenor,tenor1,tenor2,tau,strike,strikeS)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = lgm1F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           c_nN = cmsDigitalArg.c_nN;
           varCovar = cmsDigitalArg.varCovar;
           
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           varCovarS = cmsSpreadDigitalArg.varCovarS;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:1
               for j=1:1
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - strike)/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSCMSSpreadDualDigitalRange(lgm1F,to,tp,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,upperS);
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = CMSDigitalRangeAccrual(lgm1F,matu,tenor,tau,coupon,lower,upper,tbd)
           periodSum = @(Ta,Tb,tbd) sum(arrayfun(@(i) ...
               1/tbd*CMSDigitalRange(lgm1F,Ta+1/tbd*i,Tb,tenor,tau,lower,upper),...
               1:round((Tb-Ta)*tbd)));
           p = @lgm1F.DF;
           dcf = 1.0;
           out.cashflowNpv = zeros(1,round(matu));
           out.price = 0.0;
           for i=1:round(matu)
               out.cashflowNpv(i) = periodSum((i-1)*0.25,i*0.25,tbd)*coupon*p(i*0.25)*NA;
               out.price = out.price + out.cashflowNpv(i);
           end
           out.cashflowNpv(i+1) = p(matu)*NA;
           out.price = out.price + p(matu)*NA;
        end
        
        function atmSwaption = ATMSwaption(lgm1F,tx,tnr)
            tau = 0.25;
            pv01 = lgm1F.PV01(tx,tx+tnr,tau);
            fsr  = lgm1F.Fsr(tx,tx+tnr,tau);
            c_nN = lgm1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = lgm1F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:1
                for j=1:1
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function cap = Cap(lgm1F,cap_strike,dt,Ti)
            idx_mat = round(Ti/dt);
            
            HN = 0.0;
            Hn = 0.0;
            sAlpha = 0.0;
            vol = 0.0;
            d1 = 0.0;
            
            p = @lgm1F.DF;
            model_cap = 0;
            for i=1:idx_mat-1
                Hn = lgm1F.H(i*dt,lgm1F.dH1);
                HN = lgm1F.H((i+1)*dt,lgm1F.dH1);
                sAlpha = lgm1F.Alpha_Test(i*dt,lgm1F.Alpha1);
                vol = (HN-Hn)*sqrt(sAlpha);
                d1 = log(p(i*dt)/(p((i+1)*dt)*(1 + cap_strike*dt)))/vol + 0.5*vol;
                model_cap = model_cap + p(i*dt)*H_ncdf(d1)-p((i+1)*dt)*(1 + cap_strike*dt)*H_ncdf(d1-vol);
            end
            cap = model_cap;
        end
        
%         function atmSwaption = modelSwaptionExpiryTenor(lgm1F,tx,tnr)
%             tau = 0.25;
%             pv01 = lgm1F.PV01(tx,tx+tnr,tau);
%             fsr  = lgm1F.Fsr(tx,tx+tnr,tau);
%             c_nN = lgm1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
%             variances = lgm1F.LocalVariance(0,tx);
%             totalVar = 0.0;
%             for i=1:1
%                 for j=1:1
%                     totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
%                 end
%             end
%             atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
%         end
        
        function impCorrel = ImpCorrel(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function spreadInfo = SpreadInfo(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  ck2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impVariance = ImpVariance(lgm1F,tx,tnr1)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function instFwdCorr = InsFwdCorr(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = (lgm1F.H_Test(tx + tnr1 +0.001,lgm1F.dH1)-lgm1F.H_Test(tx + tnr1,lgm1F.dH1))/0.001;
            dH_tT1(2) = (lgm1F.H_Test(tx + tnr1 +0.001,lgm1F.dH2)-lgm1F.H_Test(tx + tnr1,lgm1F.dH2))/0.001;
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = (lgm1F.H_Test(tx + tnr2 +0.001,lgm1F.dH1)-lgm1F.H_Test(tx + tnr2,lgm1F.dH1))/0.001;
            dH_tT2(2) = (lgm1F.H_Test(tx + tnr2 +0.001,lgm1F.dH2)-lgm1F.H_Test(tx + tnr2,lgm1F.dH2))/0.001;
            
            instVariances = (lgm1F.LocalVariance(0,tx+0.001)-lgm1F.LocalVariance(0,tx))/0.001;
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + dH_tT1(j)*dH_tT1(k)*instVariances(j,k);
                    var2 = var2 + dH_tT2(j)*dH_tT2(k)*instVariances(j,k);
                    covar = covar + dH_tT1(j)*dH_tT2(k)*instVariances(j,k);
                end
            end
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTest = InsFwdCorrTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH1);
            dH_tT1(2) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH2);
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH1);
            dH_tT2(2) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH2);
            
            dAlpha_t = zeros(2,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
            dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:1
                var1 = var1 + dH_tT1(j)*dH_tT1(j)*dAlpha_t(j)*dAlpha_t(j);
                var2 = var2 + dH_tT2(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
                covar = covar + dH_tT1(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCovarTestMatrix = InsFwdCovarTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCovarTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
                eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;

                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
            end
            
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
                eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;
                
                testLoadingD = dH_tT*eta;
                
                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';
                
            end
            
        end
        
        function deltaHMatrix = DeltaHTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                deltaHMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                
                dH_tT = zeros(length(newtnr),2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                deltaHMatrix = dH_tT;
            end
            
        end
        
        function etaMatrix = EtaTest2(lgm1F,tx)
            eta =  zeros(1,1);
            eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
%             eta(1,2) = 0;
%             eta(2,1) = 0;
            
            etaMatrix = eta;
            
        end
        
        function instFwdCovarTest = InsFwdCovarTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(1,1);
            dH_tT1(1) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH1);
%             dH_tT1(2) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH2);
            
            dH_tT2 = zeros(1,1);
            dH_tT2(1) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH1);
%             dH_tT2(2) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            for j=1:1
                covar = covar + dH_tT1(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdCovarTest = covar;
        end
        
        function instFwdSwapCovarTest = InsSwapFwdCovarTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(1,1);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
%             C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(1,1);
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
%             C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            for j=1:1
                covar = covar + C_tT1(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdSwapCovarTest = covar;
        end
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(1,1);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
%             C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(1,1);
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
%             C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for j=1:1
                var1 = var1 + C_tT1(j)*C_tT1(j)*dAlpha_t(j)*dAlpha_t(j);
                var2 = var2 + C_tT2(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
                covar = covar + C_tT1(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function out = RebonatoFwdCorr2(lgm1F,tx,tnr1,tnr2)
           tau = 0.25; 
           rho_inf = 0.35;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function out = RebonatoFwdCorr(lgm1F,tx,tnr1,tnr2)
           rho_inf = 0.35;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function coVariance = CoVariance(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            totalVar = zeros(1,1);
            for j=1:1
                for k=1:1
                    totalVar(1,1) = totalVar(1,1) + c_nN_1(j)*c_nN_2(k)*variances(j,k);
%                     totalVar(1,2) = totalVar(1,2) + c_nN_1(j)*c_nN_2(k)*variances(j,k);
%                     totalVar(2,1) = totalVar(2,1) + c_nN_2(j)*c_nN_1(k)*variances(j,k);
%                     totalVar(2,2) = totalVar(2,2) + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                end
            end
            coVariance =  totalVar;
        end
        
        function out = TargetFunctionATMSwaptionGlobal(lgm1F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            lgm1FSwaptionPrice = zeros(1,length(tenor));
            
            for j=1:length(lgm1F.dH1.quote)
               lgm1F.dH1.quote(j) = tvar(j);
            end

            for k=1:length(lgm1F.Alpha1.quote)
               lgm1F.Alpha1.quote(k) = tvar(length(lgm1F.dH1.quote)+k);
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                    out = out + (lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end
        
        
        function out = TargetFunctionATMSwaptionGlobalLevMar(lgm1F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            lgm1FSwaptionPrice = zeros(1,length(tenor));
            % tracking calibration
            
            
%             for j=1:2*length(lgm1F.dH1.quote)+2*length(lgm1F.Alpha1.quote)
%                 lgm1F.debugInfo{1,1}.currentParameterVec(idx,j) = tvar(j);
%             end

            for j=1:length(lgm1F.dH1.quote)
               lgm1F.dH1.quote(j) = tvar(j);
            end

            for k=1:length(lgm1F.Alpha1.quote)
               lgm1F.Alpha1.quote(k) = tvar(length(lgm1F.dH1.quote)+k);
            end
            
             
            
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end
        
        
        function out = CalibrateToATMDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
%             blackSwaptionVega =  zeros(expirySize,tenorSize);
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
%                     blackSwaptionVega(k,j) = black.BlackATMSwaptionVega(expiry(k),tenor(j));
                end
            end
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
%             params.expiry = expiry;
%             params.tenor = tenor;
%             params.blackSwaptionPrice = blackSwaptionPrice;
%             params.blackSwaptionVega  = blackSwaptionVega;
%             params.expirySize = expirySize;
%             params.tenorSize =  tenorSize;
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = lgm1F.targetSize;
            params.target_size = target_size;
            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            params.swaption_expiry_use_yn = zeros(target_size,1);
            params.dH1.tenor = 1:1:target_size;
            params.dH1.tenor = params.dH1.tenor';
            params.dH1.quote = ones(target_size,1);
            
            params.Alpha1.tenor = 1:1:target_size;
            params.Alpha1.tenor = params.Alpha1.tenor';
            params.Alpha1.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            if lgm1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.Alpha1.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.dH1.tenor(i)     = temp_expiry(j);
                            params.Alpha1.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.dH1.tenor(i)     = params.dH1.tenor(i-1)     + 1;
                        params.Alpha1.tenor(i) = params.Alpha1.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,0.25);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(target_size,1);
            out.priceRelDiff = zeros(target_size,1);
            out.priceRmseTotal = 0.0;
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            out.re = zeros(target_size,1);
            out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            targetSwaptionSize = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    for j=1:length(lgm1F.Alpha1.quote)
                        if params.swaption_expiry_use_yn(j) == 1
                            targetSwaptionSize = targetSwaptionSize +1;
                        end
                    end
                    
                    optParams.dH_compare = params.dH1.quote(1);
                    
                    if lgm1F.matu <= 2.0
                        optParams.dHSize = targetSwaptionSize;
                    else
                        optParams.dHSize = targetSwaptionSize + 1;
                    end
                    
                    optParams.AlphaSize = targetSwaptionSize;
                    
                    optParams.modelParamsSize = optParams.dHSize + optParams.AlphaSize;
                    
                    if lgm1F.matu <= 2.0
                        optParams.numOfEquation = targetSwaptionSize + ... % # of target swaption instruments
                                                        0 + ...           % # of Alpha term structure gamma constraints
                                                  optParams.dHSize;    % # of dH absolute error relative to 1.0 
                    else
                        optParams.numOfEquation = targetSwaptionSize + ...         % # of target swaption instruments
                                                  optParams.AlphaSize -2 + ...     % # of Alpha term structure gamma constraints
                                                  optParams.dHSize;                % # of dH absolute error relative to 1.0 
                    end
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.dHSize
                        tvar(j) = params.dH1.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                    
                    for j=1:optParams.AlphaSize
                        tvar(j + optParams.dHSize) = params.Alpha1.quote(j);
                        lb(j + optParams.dHSize)= 0.0001;
                        ub(j + optParams.dHSize)= 10;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    k=1;
                    for j=1:optParams.dHSize
                       lgm1F.dH1.quote(j) = max(popt(j),0.0001);
                       k=k+1;
                    end

                    for j=k:length(lgm1F.dH1.quote)
                       lgm1F.dH1.quote(j) = lgm1F.dH1.quote(optParams.dHSize);
                    end

                    k = 1;
                    for j=1:optParams.AlphaSize
                       lgm1F.Alpha1.quote(j) = max(popt(optParams.dHSize+j),0.0001);
                       k=k+1;
                    end

                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    for j=k:length(lgm1F.Alpha1.quote)
                       lgm1F.Alpha1.quote(j) = lgm1F.Alpha1.quote(optParams.AlphaSize);
                    end
                    
                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = lgm1F.ATMSwaption(Ti,Tj);
                            out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                            N=N+1; 
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    
                case 'globalLevMar'
                    for j=1:length(lgm1F.dH1.quote)
                        tvar(j) = lgm1F.dH1.quote(j);
                        lb(j)= -10;
                        ub(j)= 10;
                    end

                    for k=1:length(lgm1F.Alpha1.quote)
                        tvar(length(lgm1F.dH1.quote)+k) = lgm1F.Alpha1.quote(k);
                        lb(length(lgm1F.dH1.quote)+k)= -10;
                        ub(length(lgm1F.dH1.quote)+k)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.init_lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.init_relDiff(j,k) = out.init_lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.init_rmseTotal = out.init_rmseTotal + out.init_relDiff(j,k)*out.init_relDiff(j,k);
                            N=N+1;
                        end
                    end
                    out.init_rmseTotal = sqrt(out.init_rmseTotal/N);
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMar(lgm1F,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt','MaxFunEvals',700);
                    [x, fval] = lsqnonlin(targetfun,tvar,[],[],options);
%                     [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
%                     options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for j=1:length(lgm1F.dH1.quote)
                        out.dH1.quote(j) = x(j);
                        lgm1F.dH1.quote(j) = x(j);
                    end
            
                    for k=1:length(lgm1F.Alpha1.quote)
                        out.Alpha1.quote(k) = x(length(lgm1F.dH1.quote)+k);
                        lgm1F.Alpha1.quote(k) = x(length(lgm1F.dH1.quote)+k);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
            
                    
                case 'dummy'
                    
                    for j=1:length(lgm1F.dH1.quote)
                        out.dH1.quote(j) = lgm1F.dH1.quote(j);
                        out.dH2.quote(j) = lgm1F.dH2.quote(j);
                    end
            
                    for k=1:length(lgm1F.Alpha1.quote)
                        out.Alpha1.quote(k) =  lgm1F.Alpha1.quote(k);
                        out.dAlpha2.quote(k) = lgm1F.dAlpha2.quote(k);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMCapDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
            capVol = black.capVol;
            capVolData = black.mktData('capVol');
            capVolrawData =  capVolData.params('rawData');
            
            capVolTermStructure.expiry = capVolrawData(:,1);
            capVolTermStructure.capVol = capVol; % capVol With interpolator
            
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = lgm1F.targetSize;
            params.target_size = target_size;
            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            
            params.cap_expiry_use_yn = zeros(target_size,1);
            params.swaption_expiry_use_yn = zeros(target_size,1);
            
            params.dH1.tenor = 1:1:target_size;
            params.dH1.tenor = params.dH1.tenor';
            params.dH1.quote = 0.1*ones(target_size,1);
            
            params.Alpha1.tenor = 1:1:target_size;
            params.Alpha1.tenor = params.Alpha1.tenor';
            params.Alpha1.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            params.cap.expiry = zeros(target_size,1);
            params.cap.quote = zeros(target_size,1);
            params.cap.strike = zeros(target_size,1);
            params.cap.blackPrice = zeros(target_size,1);
            
            % setting target diagonal swaption 
            if lgm1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.Alpha1.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.Alpha1.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.Alpha1.tenor(i) = params.Alpha1.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % setting target cap
            params.cap.expiry(1) = 1;
            params.cap.quote(1) = capVolTermStructure.capVol(params.cap.expiry(1));
            params.cap_expiry_use_yn(1) = 1;
            
            for i=2:target_size
                params.cap.expiry(i) = i;
                params.cap.quote(i) = capVolTermStructure.capVol(params.cap.expiry(i));
                if params.cap.expiry(i-1) < params.matu + 0.25
                    params.cap_expiry_use_yn(i) = 1;
                end
                if i == 10  % exclude 12Y, 15Y cap vol from target     
                    break;
                end
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,0.25);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                end
            end
            
           %% target cap price and forward ATM strike
            for i=1:target_size
                if params.cap_expiry_use_yn(i)
                    Ti = params.cap.expiry(i);
                    params.cap.strike(i) = black.Fsr(0.25,Ti,0.25);
                    params.cap.blackPrice(i) = black.BlackCapWithVol(Ti,params.cap.strike(i),0.25,params.cap.quote(i));
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            params.cap_size = target_size;
            params.swaption_size = target_size;
            
            out.cap_size = target_size;
            out.swaption_size = target_size;
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(target_size,1);
            out.swaptionPriceRelDiff = zeros(target_size,1);
            out.swaptionPriceRmseTotal = 0.0;
            
            out.blackCapPrice = params.cap.blackPrice;
            out.lgm1FCapPrice = zeros(target_size,1);
            out.capPriceRelDiff = zeros(target_size,1);
            out.capPriceRmseTotal = 0.0;
            
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            
            out.market_cap = zeros(target_size,1);
            out.model_cap = zeros(target_size,1);
            
%             out.re = zeros(target_size,1);
%             out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
%             targetSwaptionSize = 0;
%             targetCapSize = 0;
            
%             dH_size = 0;
%             Alpha_size = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    optParams.dH_size =0;
                    optParams.Alpha_size =0;
                    
                    for j=1:out.cap_size
                        if params.cap_expiry_use_yn(j) == 1
                            optParams.dH_size = optParams.dH_size +1;
                        end
                    end
                    
                    for j=1:out.swaption_size
                        if params.swaption_expiry_use_yn(j) == 1
                            optParams.Alpha_size = optParams.Alpha_size +1;
                        end
                    end
                    
                    if lgm1F.matu <= 10.0
                        optParams.dH_size = optParams.dH_size - 1;
                        optParams.Alpha_size = optParams.Alpha_size + 1;
                    end
                    
                    optParams.dH_1Y_fixed_value = params.dH1.quote(1);
                    
                    optParams.modelParamsSize = optParams.dH_size + optParams.Alpha_size;
                    
                    optParams.numOfEquation = optParams.dH_size + optParams.Alpha_size; 
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=2:optParams.dH_size+1
                        tvar(j-1) = params.dH1.quote(j);
                        lb(j-1)= -10.0;
                        ub(j-1)= 10.0;
                    end
                    
                    for j=1:optParams.Alpha_size
                        tvar(j + optParams.dH_size) = params.Alpha1.quote(j);
                        lb(j + optParams.dH_size)= -10.0;
                        ub(j + optParams.dH_size)= 10.0;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMCapSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    k=2;
                    for j=2:optParams.dH_size+1
                       lgm1F.dH1.quote(j) = max(abs(popt(j-1)),0.0001);
                       k=k+1;
                    end

                    for j=k:length(lgm1F.dH1.quote)
                       lgm1F.dH1.quote(j) = lgm1F.dH1.quote(optParams.dH_size+1);
                    end

                    k = 1;
                    for j=1:optParams.Alpha_size
                       lgm1F.Alpha1.quote(j) = max(abs(popt(optParams.dH_size+j)),0.0001);
                       k=k+1;
                    end

                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    for j=k:length(lgm1F.Alpha1.quote)
                       lgm1F.Alpha1.quote(j) = lgm1F.Alpha1.quote(optParams.Alpha_size);
                    end
                    
                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.swaption_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = lgm1F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i)*out.swaptionPriceRelDiff(i);
                            N=N+1; 
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
                    N=0;
                    for i=1:params.cap_size
                        if params.cap_expiry_use_yn(i) == 1
                            Ti = params.cap.expiry(i);
                            out.market_cap(i) = params.cap.blackPrice(i);
                            out.model_cap(i)  = lgm1F.Cap(params.cap.strike(i),0.25,Ti);
                            out.capPriceRelDiff(i) = out.model_cap(i)/out.market_cap(i) - 1.0;
                            out.capPriceRmseTotal = out.capPriceRmseTotal + out.capPriceRelDiff(i)*out.capPriceRelDiff(i);
                            N=N+1; 
                        end
                    end
                    out.capPriceRmseTotal = sqrt(out.capPriceRmseTotal/N);
                    
                
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMSwaptionTenorDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            % dummy
%             target_size = lgm1F.targetSize;

            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            params.joint_swap_tenor = lgm1F.joint_swap_tenor;
            
            params.dh_initial = 1.0;
            params.alpha_initial = 0.01;
            params.smallNumber = 0.000001;
            
            % set the target swaption expiry & tenor
            remainingMatu = params.matu - floor(params.matu);
            max_swaption_expiry_size = ceil(params.matu);
            
            if remainingMatu == 0
                remainingMatu = 1.0;
            end
            
            if params.matu -1  < params.joint_swap_tenor
                max_swaption_tenor_size = floor(params.joint_swap_tenor);
            else
                max_swaption_tenor_size = floor(params.matu - params.smallNumber);
            end
            
            params.targetSwaption.expiry = zeros(max_swaption_expiry_size,1);
            params.targetSwaption.tenor  = zeros(max_swaption_tenor_size,1);
            params.targetSwaption.quote  = zeros(max_swaption_expiry_size,max_swaption_tenor_size);
            params.swaption_use_yn = zeros(max_swaption_expiry_size,max_swaption_tenor_size);
            
            for i=1:max_swaption_expiry_size
                params.targetSwaption.expiry(i) = remainingMatu + i-1;
            end
            
            for i=1:max_swaption_tenor_size
                params.targetSwaption.tenor(i) = i;
            end
            
            params.target_size = 0;
            for i=1:max_swaption_expiry_size
                swaptionExpiry = params.targetSwaption.expiry(i);
                for j=1:max_swaption_tenor_size
                    swaptionTenor = params.targetSwaption.tenor(j);
                    diag_yn = 0;
                    tenor_yn = 0;
                    
                    if abs(swaptionExpiry + swaptionTenor - params.matu) < params.smallNumber 
                        diag_yn = 1;
                    end
                    
                    if (swaptionTenor == params.joint_swap_tenor) && ((swaptionExpiry <= params.matu) || abs(swaptionExpiry - params.matu) < params.smallNumber)
                        tenor_yn = 1;
                    end
                    
                    if (diag_yn) || (tenor_yn)
                        params.swaption_use_yn(i,j) = 1;
                        params.target_size = params.target_size +1;
                    end
                end
            end
            
            params.target_expiry_idx = zeros(params.target_size,1);
            params.target_tenor_idx = zeros(params.target_size,1);
            params.targetSwaption.strike = zeros(params.target_size,1);
            params.targetSwaption.blackPrice = zeros(params.target_size,1);
            
            idx = 1;
            for i=1:max_swaption_expiry_size
                for j=1:max_swaption_tenor_size
                    if params.swaption_use_yn(i,j) == 1
                        params.target_expiry_idx(idx) = i;
                        params.target_tenor_idx(idx) = j;
                        idx = idx +1;
                    end
                end
            end
            
            dh_size = floor( params.matu) + 1 + params.joint_swap_tenor + 1;
            params.dH1.tenor = zeros(dh_size,1);
            params.dH1.quote = zeros(dh_size,1);
            
            for i=1:dh_size
                params.dH1.tenor(i) = remainingMatu + i-1;
                params.dH1.quote(i) = params.dh_initial;
            end
            
            alpha_size = floor(params.matu - params.smallNumber) + 2;
            params.Alpha1.tenor = zeros(alpha_size,1);
            params.Alpha1.quote = zeros(alpha_size,1);
            
            for i=1:alpha_size
                params.Alpha1.tenor(i) = remainingMatu + i-1;
                params.Alpha1.quote(i) = params.alpha_initial;
            end
            
            upper_expiry = swaptionVolSurface.expiry(end);
            upper_tenor  = swaptionVolSurface.tenor(end);
            lower_expiry = swaptionVolSurface.expiry(1);
            
            for i=1:params.target_size
                expiry_idx = params.target_expiry_idx(i);
                tenor_idx = params.target_tenor_idx(i);
                swaptionExpiry = params.targetSwaption.expiry(expiry_idx);
                swaptionTenor = params.targetSwaption.tenor(tenor_idx);
                
                if(swaptionExpiry > upper_expiry && swaptionTenor <= upper_tenor)
                    params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(upper_expiry,swaptionTenor);
                elseif(swaptionExpiry <= upper_expiry && swaptionTenor > upper_tenor)
                    params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(swaptionExpiry,upper_tenor);    
                else
                    if swaptionExpiry < lower_expiry
                        params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(lower_expiry,swaptionTenor);
                    else
                        params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(swaptionExpiry,swaptionTenor);
                    end
                end
                
                params.targetSwaption.strike(i) = black.Fsr(swaptionExpiry,swaptionExpiry + swaptionTenor,0.25);
%                 annuity = black.PV01(swaptionExpiry,swaptionExpiry + swaptionTenor,0.25);
                blackVolQuote = params.targetSwaption.quote(expiry_idx,tenor_idx); 
                params.targetSwaption.blackPrice(i) = black.BlackSwaptionWithVol(swaptionExpiry,swaptionTenor,params.targetSwaption.strike(i),blackVolQuote);
            end
            
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            
          
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            params.swaption_size = params.target_size;
            out.swaption_size = params.target_size;
            
            out.blackSwaptionPrice = params.targetSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(params.target_size,1);
            out.swaptionPriceRelDiff = zeros(params.target_size,1);
            out.swaptionPriceRmseTotal = 0.0;
            
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(params.target_size,1);
            out.model_swaption = zeros(params.target_size,1);
            
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;

            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    optParams.dH_size = length(params.dH1.quote);  
                    optParams.Alpha_size = length(params.Alpha1.quote);
                    
                    % dH1(end-1) = dH1(end)
                    optParams.modelParamsSize = optParams.dH_size-1 + optParams.Alpha_size;
                    
                    optParams.numOfEquation = params.target_size + ...  % target swaption
                                              optParams.Alpha_size -2 + ... % tikhonov regularization gamma constraint
                                              optParams.dH_size;     % constraint on dH  
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.dH_size-1
                        tvar(j) = params.dH1.quote(j);
                        lb(j)= -10.0;
                        ub(j)= 10.0;
                    end
                    
                    for j=1:optParams.Alpha_size
                        tvar(j + optParams.dH_size-1) = params.Alpha1.quote(j);
                        lb(j + optParams.dH_size-1)= -10.0;
                        ub(j + optParams.dH_size-1)= 10.0;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMSwaptionTenorSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    for j=1:optParams.dH_size-1
                       lgm1F.dH1.quote(j) = max(abs(popt(j)),0.0001);
                    end
                    lgm1F.dH1.quote(optParams.dH_size) = lgm1F.dH1.quote(optParams.dH_size-1);
                    
                    for j=1:optParams.Alpha_size
                       lgm1F.Alpha1.quote(j) = max(abs(popt(optParams.dH_size-1 + j)),0.0001);
                    end

                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        expiry_idx = params.target_expiry_idx(i);
                        tenor_idx = params.target_tenor_idx(i);
                        swaptionExpiry = params.targetSwaption.expiry(expiry_idx);
                        swaptionTenor = params.targetSwaption.tenor(tenor_idx);
                        out.market_swaption(i) = params.targetSwaption.blackPrice(i);
                        out.model_swaption(i) = lgm1F.ATMSwaption(swaptionExpiry,swaptionTenor);
                        out.swaptionPriceRelDiff(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                        out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i)*out.swaptionPriceRelDiff(i);
                        N=N+1;
                    end
                    
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
                
                otherwise
                    disp('unImplemented')
            end
        end 
        

    end
    
end



In [ ]:
classdef QuantoLGM1F < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        domestic_dH;
        foreign_dH;
        domestic_Alpha;
        foreign_Alpha;
        
        corr_df;
        corr_dx;
        corr_fx;
        
        fxBlackVol;
        
        numOfFactor;
        modelType;
        freq;
    end
    
    methods
        function quantolgm1f = QuantoLGM1F(params)
            if nargin > 0

                quantolgm1f.zeroCurve = params.foreignModel.zeroCurve;
                quantolgm1f.freq = params.foreignModel.freq;
                
                quantolgm1f.numOfFactor = 1;
                quantolgm1f.modelType = 'QuantoLGM1F';
                
                
                if params.fxVolCalibrated == true
                    % we do not calibrate fxBlackVol
                    quantolgm1f.corr_df = params.corr_df;
                    quantolgm1f.corr_dx = params.corr_dx;
                    quantolgm1f.corr_fx = params.corr_fx;
                    
                    out = quantolgm1f.synchronizeModelParams(params);
                    
                    quantolgm1f.domestic_Alpha.tenor = out.sigmaTimes;
                    quantolgm1f.domestic_Alpha.quote = out.domestic_AlphaValues;

                    quantolgm1f.foreign_Alpha.tenor = out.sigmaTimes;
                    quantolgm1f.foreign_Alpha.quote = out.foreign_AlphaValues;

                    quantolgm1f.domestic_dH.tenor = out.sigmaTimes;
                    quantolgm1f.domestic_dH.quote = out.domestic_dHValues;

                    quantolgm1f.foreign_dH.tenor = out.sigmaTimes;
                    quantolgm1f.foreign_dH.quote = out.foreign_dHValues;

                    quantolgm1f.fxBlackVol.tenor = out.sigmaTimes;
                    quantolgm1f.fxBlackVol.quote = out.fxBlackVolValues;
                    
                else
                    
                    % black vol calibration start
                    paramsIn.fxImpVol = params.fxBlackVol;
                    paramsIn.fxBlackVol = paramsIn.fxImpVol;
                    paramsIn.domestic_dH = params.domesticModel.dH1;
                    paramsIn.domestic_Alpha = params.domesticModel.Alpha1;
                    paramsIn.foreign_dH = params.foreignModel.dH1;
                    paramsIn.foreign_Alpha = params.foreignModel.Alpha1;
                    
                    
                    paramsIn.corr_df = params.corr_df;
                    paramsIn.corr_dx = params.corr_dx;
                    paramsIn.corr_fx = params.corr_fx;
                    
                     % BSLGMLGM Calibration Start
                    out = quantolgm1f.CalibrateBSLGMLGMVol(paramsIn);
                    re =  out.re;
                    rmseTotal = out.rmseTotal;
                    
                    % BSLGMLGM Calibration End
                    
                    params.fxBlackVol = out.fxBlackVol;
                    params.fxVolCalibrated =  true;
                    quantolgm1f = QuantoLGM1F(params); 
                    
                end
                %end
                
                
            end
        end
        
        function out =  synchronizeModelParams(quantolgm1f,params)
            fxBlackVolTenor = params.fxBlackVol.tenor;
            fxBlackVolQuote = params.fxBlackVol.quote;

            % synchronize model parameters time-steps start
            domestic_dHTenor = params.domesticModel.dH1.tenor;
            domestic_dHQuote = params.domesticModel.dH1.quote;

            domestic_AlphaTenor = params.domesticModel.Alpha1.tenor;
            domestic_AlphaQuote = params.domesticModel.Alpha1.quote;

            foreign_dHTenor = params.foreignModel.dH1.tenor;
            foreign_dHQuote = params.foreignModel.dH1.quote;

            foreign_AlphaTenor = params.foreignModel.Alpha1.tenor;
            foreign_AlphaQuote = params.foreignModel.Alpha1.quote;

            sigmaTimes = unique(union([domestic_AlphaTenor(:);foreign_AlphaTenor(:); ...
                         domestic_dHTenor(:);foreign_dHTenor(:)],fxBlackVolTenor));


            sigmaTimes = sort(sigmaTimes,'ascend');

            domestic_AlphaValues = zeros(length(sigmaTimes),1);
            foreign_AlphaValues = zeros(length(sigmaTimes),1);
            domestic_dHValues = zeros(length(sigmaTimes),1);
            foreign_dHValues = zeros(length(sigmaTimes),1);
            fxBlackVolValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                domestic_AlphaValues(k) = H_interpolation(domestic_AlphaTenor, domestic_AlphaQuote,sigmaTimes(k),0);
                foreign_AlphaValues(k) = H_interpolation(foreign_AlphaTenor, foreign_AlphaQuote,sigmaTimes(k),0);
                domestic_dHValues(k) = H_interpolation(domestic_dHTenor, domestic_dHQuote,sigmaTimes(k),0);
                foreign_dHValues(k) = H_interpolation(foreign_dHTenor, foreign_dHQuote,sigmaTimes(k),0);
                fxBlackVolValues(k) = H_interpolation(fxBlackVolTenor, fxBlackVolQuote,sigmaTimes(k),0);
            end
            
            out.sigmaTimes = sigmaTimes;
            out.domestic_AlphaValues = domestic_AlphaValues;
            out.foreign_AlphaValues = foreign_AlphaValues;
            out.domestic_dHValues = domestic_dHValues;
            out.foreign_dHValues = foreign_dHValues;
            out.fxBlackVolValues = fxBlackVolValues;
            
        end
        
        function out =  Libor_SimpleDate(quantolgm1f,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           
           % foreign Libor in domestic Terminal measure
           
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = quantolgm1f.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = quantolgm1f.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
         function out =  CMS_PSA_SimpleDate(quantolgm1f,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMT
%            tau = 0.25;
           tau = 1.0/(quantolgm1f.freq);
            % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           
           if abs(tenor- 0.25) < 1e-8
                tau = tenor;
           end
           
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = quantolgm1f.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = quantolgm1f.DF(to);
           df_TN = quantolgm1f.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = quantolgm1f.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = quantolgm1f.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
%            h_nTerm = quantolgm1f.H_nTermDate(valueDate,toDate,numMatDate);
           
           %h_nTerm is replace by i_Term_tilda & i_Tn in the following
           
           h_nTerm(1) = 0.0;
           
           varCovar = quantolgm1f.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:1
               for j=1:1
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           % additional drift correction term from domestic Term KRW to foreign Term measure KTB
           i_Term_tilda = I_Term_tilda(quantolgm1f,0,to,numMatTime/365.0);
           i_Tn = I_Tn(quantolgm1f,0,to,to);
           j_x_tilda = J_x_tilda(quantolgm1f,0,to,to);
           
%            additionalDrift = 0;
           additionalDrift = c_nN(1) * (i_Term_tilda + i_Tn + j_x_tilda);
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + additionalDrift + c_nN(1)*modelStatesT(1,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
         end
         
         function out = Foreign_HtT_r(quantolgm1f,t,T)
            out =  quantolgm1f.H(T,quantolgm1f.foreign_dH) - quantolgm1f.H(t,quantolgm1f.foreign_dH); 
         end
         
         function out = Domestic_HtT_r(quantolgm1f,t,T)
            out =  quantolgm1f.H(T,quantolgm1f.domestic_dH) - quantolgm1f.H(t,quantolgm1f.domestic_dH); 
         end
         
         function out = H(quantolgm1f,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
         end

        
        function out = fwdZcVarianceForeign(quantolgm1f, a, b, T1, T2)
            
            localVariance = zeros(3,1);
            measureChange = zeros(2,1);
            quantoChange = zeros(1,1);
            sigmaTimes = quantolgm1f.domestic_Alpha.tenor;
            foreign_AlphaValues = quantolgm1f.foreign_Alpha.quote;
            domestic_AlphaValues = quantolgm1f.domestic_Alpha.quote;
            foreign_dHValues = quantolgm1f.foreign_dH.quote;
            domestic_dHValues = quantolgm1f.domestic_dH.quote;
            
            fxValues = quantolgm1f.fxBlackVol.quote;
            
            localVariance(1) =  quantolgm1f.Foreign_HtT_r(a,T1) * ...
                                quantolgm1f.Foreign_HtT_r(a,T1) * ...
                                quantolgm1f.LocalVariance_Alpha2(a,b,sigmaTimes,foreign_AlphaValues,foreign_AlphaValues);
            
            localVariance(2) =  quantolgm1f.Foreign_HtT_r(a,b) * ...
                                quantolgm1f.Foreign_HtT_r(a,b) * ...
                                quantolgm1f.LocalVariance_Alpha2(a,b,sigmaTimes,foreign_AlphaValues,foreign_AlphaValues);

            localVariance(3) =  (quantolgm1f.Foreign_HtT_r(a,T1) - ...
                                quantolgm1f.Foreign_HtT_r(a,b)) * ...
                                quantolgm1f.LocalVariance_H_Alpha2(a,b,sigmaTimes,foreign_dHValues,foreign_AlphaValues,foreign_AlphaValues);
            
            
            measureChange(1) = (quantolgm1f.Foreign_HtT_r(a,T1) - ...
                                quantolgm1f.Foreign_HtT_r(a,b)) * ...
                                quantolgm1f.Domestic_HtT_r(a,T2) * ...
                                quantolgm1f.LocalVariance_Alpha2(a,b,sigmaTimes,foreign_AlphaValues,domestic_AlphaValues);
                            
            measureChange(2) = (quantolgm1f.Foreign_HtT_r(a,T1) - ...
                                quantolgm1f.Foreign_HtT_r(a,b)) * ...
                                quantolgm1f.LocalVariance_H_Alpha2(a,b,sigmaTimes,domestic_dHValues,foreign_AlphaValues,domestic_AlphaValues);
                                
            
            quantoChange(1) = (quantolgm1f.Foreign_HtT_r(a,T1) - ...
                                quantolgm1f.Foreign_HtT_r(a,b)) * ...
                                quantolgm1f.LocalVariance_Alpha2(a,b,sigmaTimes,fxValues,foreign_AlphaValues);
            
            value = localVariance(1) - localVariance(2) - 2.0*localVariance(3);
            value = value -2.0 * quantolgm1f.corr_df * (measureChange(1) - measureChange(2));
            value = value -2.0 * quantolgm1f.corr_fx * quantoChange(1);
            
            out = value;
            
        end
        
        function beta = Beta(quantolgm1f,t,T)
            beta =[];
            beta(1) = quantolgm1f.H(T,quantolgm1f.foreign_dH)-quantolgm1f.H(t,quantolgm1f.foreign_dH);
        end
        
        function beta = BetaDate(quantolgm1f,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            beta =[];
            
            beta(1) = quantolgm1f.H(T,quantolgm1f.foreign_dH)-quantolgm1f.H(t,quantolgm1f.foreign_dH);
        end
        
        function c_nN = C_nN(quantolgm1f,tn,tN,tau,pnN,ynN)
            p = @quantolgm1f.DF;
            c_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = quantolgm1f.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
        function c_nN = C_nNDate(quantolgm1f,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @quantolgm1f.DF;
            c_nN=[0.0];
            beta = [0.0];
            df = 0.0;
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = quantolgm1f.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(1) = c_nN(1) +deltaT * beta(1) * df;
                prevTime = tp;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
        end
        
        function g_nN = G_nN(quantolgm1f,tn,tN,tau,pnN)
            p = @quantolgm1f.DF;
            g_nN=[0.0];
            
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = quantolgm1f.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function g_nN = G_nNDate(quantolgm1f,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @quantolgm1f.DF;
            g_nN=[0.0];
            
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = quantolgm1f.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(1) = g_nN(1) + deltaT * beta(1) * df;
                prevTime = tp;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function h_nTerm = H_nTerm(quantolgm1f,tn,tTerm)
           h_nTerm =[];
           beta = quantolgm1f.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function h_nTerm = H_nTermDate(quantolgm1f,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = quantolgm1f.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function i_Term_tilda = I_Term_tilda(quantolgm1f,from,to,tTerm)

            sigmaTimes = quantolgm1f.domestic_Alpha.tenor;
            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1 = quantolgm1f.domestic_dH;
           sigmaValues1 = quantolgm1f.domestic_Alpha.quote;
           sigmaValues2 = quantolgm1f.foreign_Alpha.quote;
           corr12 = quantolgm1f.corr_df;

           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH = dH1.quote(i);else dH = dH1.quote(sigmaSize);end;
                
                H_Term  = quantolgm1f.H(tTerm,dH1);
                H_lastU = quantolgm1f.H(lastU,dH1);
                
                variances = variances + corr12 * vol1 * vol2 * ((H_Term - H_lastU)*(U - lastU) - ...
                                        0.5 * dH * (U - lastU) * (U - lastU));
                lastU = U;
           end
           
           % we add minus 1 so that we don't need additional minus in
           % drift adjust term
           i_Term_tilda = -1.0 * variances;
           
        end
        
        function i_Tn = I_Tn(quantolgm1f,from,to,tn)

            sigmaTimes = quantolgm1f.domestic_Alpha.tenor;
            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1 = quantolgm1f.foreign_dH;
           sigmaValues1 = quantolgm1f.foreign_Alpha.quote;
           sigmaValues2 = quantolgm1f.foreign_Alpha.quote;
           corr12 = 1.0;

           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH = dH1.quote(i);else dH = dH1.quote(sigmaSize);end;
                
                H_Tn  = quantolgm1f.H(tn,dH1);
                H_lastU = quantolgm1f.H(lastU,dH1);
                
                variances = variances + corr12 * vol1 * vol2 * ((H_Tn - H_lastU)*(U - lastU) - ...
                                        0.5 * dH * (U - lastU) * (U - lastU));
                lastU = U;
           end
           
           % we don't need minus 1
           i_Tn = 1.0 * variances;
           
        end
        
        function j_x_tilda = J_x_tilda(quantolgm1f,from,to,tn)

            sigmaTimes = quantolgm1f.domestic_Alpha.tenor;
            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
%            dH1 = quantolgm1f.foreign_dH;
           sigmaValues1 = quantolgm1f.fxBlackVol.quote;
           sigmaValues2 = quantolgm1f.foreign_Alpha.quote;
%            corr12 = quantolgm1f.corr_df;
           corr12 =  quantolgm1f.corr_fx;
%            corr12 = 0.15; 
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
%                 if (i< sigmaSize) dH = dH1.quote(i);else dH = dH1.quote(sigmaSize);end;
                
%                 H_Tn  = quantolgm1f.H(tn,dH1);
%                 H_lastU = quantolgm1f.H(lastU,dH1);
                
                variances = variances + corr12 * vol1 * vol2 * (U - lastU);
                lastU = U;
           end
           
           % we need minus 1
           j_x_tilda = -1.0 * variances;
           
        end
        
        function out = discountFactor(quantolgm1f, startTime, endTime,numMatTime,modelStatesT)
           
            zcT = quantolgm1f.DF(endTime/365.0);
           
            %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = quantolgm1f.DF(startTime/365.0);
           beta = quantolgm1f.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = quantolgm1f.fwdZcVarianceForeign(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           % quantolgm1F is one factor model
           for i=1:stateSize
%                discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
           end
           
           out = discountFactorOut;
           
        end
        
%         function out = discountPayoff(quantolgm1f,eventTime,numMatTime ...
%                 ,modelStatesT,cashflow)
%             
%             dfNumMat = quantolgm1f.GetChildModel('d').DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             domesticModelStatesT = modelStatesT(1,:);
%             discountT = quantolgm1f.GetChildModel('d').discountFactor(eventTime, numMatTime,numMatTime,domesticModelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        function variances = LocalVariance(quantolgm1f,from,to)

            sig(1) = quantolgm1f.foreign_Alpha;
            sig(2) = quantolgm1f.foreign_Alpha;
            corr = 1.0;
            
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = zeros(2,2);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                
                correl =ones(2,2);
                correl(1,2) = corr;
                correl(2,1) = corr;
                for j=1:2
                    for k=1:2
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function variances = LocalCovariance(quantolgm1f,from,to)

            sig(1) = quantolgm1f.domestic_Alpha;
            sig(2) = quantolgm1f.foreign_Alpha;
            corr = quantolgm1f.corr_df;
            
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = zeros(2,2);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                
                correl =ones(2,2);
                correl(1,2) = corr;
                correl(2,1) = corr;
                scale = U - lastU;
%                 variances(1,1) = variances(1,1) + vol(1)*vol(2)*corr...
%                                        * scale;
                for j=1:2
                    for k=1:2
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = LocalVariance_H2_Alpha2(quantolgm1f,from,to,sigmaTimes,dH1Values, dH2Values, sigmaValues1,sigmaValues2)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1.quote = dH1Values;
           dH1.tenor = sigmaTimes;
           
           dH2.quote = dH2Values;
           dH2.tenor = sigmaTimes;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH1Value = dH1.quote(i);else dH1Value = dH1.quote(sigmaSize);end;
                if (i< sigmaSize) dH2Value = dH2.quote(i);else dH2Value = dH2.quote(sigmaSize);end;
                
                variances = variances + quantolgm1f.H(lastU,dH1)*quantolgm1f.H(lastU,dH2)*vol1*vol2*(U - lastU);
                variances = variances + dH1Value*quantolgm1f.H(lastU,dH2)*vol1*vol2*0.5*(U - lastU)*(U - lastU);
                variances = variances + quantolgm1f.H(lastU,dH1)*dH2Value*vol1*vol2*0.5*(U - lastU)*(U - lastU);
                variances = variances + dH1Value*dH2Value*vol1*vol2*1.0/3.0*(U - lastU)*(U - lastU)*(U - lastU);
                
                
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = LocalVariance_H_Alpha2(quantolgm1f,from,to,sigmaTimes,dH1Values,sigmaValues1,sigmaValues2)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1.quote = dH1Values;
           dH1.tenor = sigmaTimes;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH = dH1.quote(i);else dH = dH1.quote(sigmaSize);end;
                
                variances = variances + quantolgm1f.H(lastU,dH1) * vol1 * vol2 * (U - lastU);
                variances = variances + 0.5 * dH * vol1 * vol2 * (U - lastU) * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = LocalVariance_Alpha2(quantolgm1f,from,to,sigmaTimes,sigmaValues1,sigmaValues2)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                
                variances = variances + vol1 * vol2 * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = GeneralVariance_LGMLGM(quantolgm1f,a,b,T1,T2,sigmaTimes,dH1Values,dH2Values,Alpha1,Alpha2)
            dH1.quote = dH1Values;
            dH1.tenor = sigmaTimes;
            
            dH2.quote = dH2Values;
            dH2.tenor = sigmaTimes;
            
            variances = zeros(4,1);
            variances(1) = quantolgm1f.LocalVariance_Alpha2(a,b,sigmaTimes, Alpha1,Alpha2);
            variances(1) = variances(1) * quantolgm1f.H(T1,dH1) * quantolgm1f.H(T2,dH2);
            
            variances(2) = quantolgm1f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH1Values,Alpha1,Alpha2);
            variances(2) = variances(2) * -1.0* quantolgm1f.H(T2,dH2);
            
            variances(3) = quantolgm1f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH2Values,Alpha1,Alpha2);
            variances(3) = variances(3) * -1.0* quantolgm1f.H(T1,dH1);
            
            variances(4) = quantolgm1f.LocalVariance_H2_Alpha2(a,b,sigmaTimes,dH1Values,dH2Values,Alpha1,Alpha2);
            
            value = variances(1) + variances(2) + variances(3) + variances(4);
            out = value;
        end
        
        function out = GeneralVariance_BSLGM(quantolgm1f,a,b,T1,sigmaTimes,dH1Values,Alpha1,Alpha2)
            dH1.quote = dH1Values;
            dH1.tenor = sigmaTimes;
            
            variances = zeros(2,1);
            variances(1) = quantolgm1f.LocalVariance_Alpha2(a,b,sigmaTimes, Alpha1,Alpha2);
            variances(1) = variances(1) * quantolgm1f.H(T1,dH1);
            
            variances(2) = quantolgm1f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH1Values,Alpha1,Alpha2);
            variances(2) = variances(2) * -1.0;
            
            value = variances(1) + variances(2);
            out = value;
        end
        
        function out = CMSDigitalArgDate(quantolgm1f,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           to = toDate.DateDiff(valueDate)/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           tauIn = tau;
           if abs(tenor1- 0.25) < 1e-8
                tauIn = tenor1;
           end
           
           p_nN1 = quantolgm1f.PV01Date(valueDate,toDate,tN1Date,tauIn);
           
           df_Tn1 = quantolgm1f.DF(to);
           df_TN1 = quantolgm1f.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = quantolgm1f.C_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1,y_nN1);
           g_nN1 = quantolgm1f.G_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1);
%            h_nTerm1 = quantolgm1f.H_nTermDate(valueDate,toDate,tpDate);
           % h_nTerm1 is replaced with i_Term_tilda & i_Tn as in the
           % following discussion
           
           h_nTerm1(1) = 0.0;
           
           varCovar = quantolgm1f.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           tpTime = tpDate.DateDiff(valueDate);
           % additional drift correction term from domestic Term KRW to foreign Term measure KTB
           i_Term_tilda = I_Term_tilda(quantolgm1f,0,to,tpTime/365.0);
           i_Tn = I_Tn(quantolgm1f,0,to,to);
           j_x_tilda = J_x_tilda(quantolgm1f,0,to,to);
           
%            additionalDrift = 0;
           additionalDrift = c_nN1(1) *(i_Term_tilda + i_Tn + j_x_tilda);
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.fwdCMS = y_nN1;
           out.mu = y_nN1 + driftTerm + additionalDrift;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
        end
        
        function out =computeBSLGMLGMVariance(quantolgm1f,a,b,T,corr_df,corr_dx,corr_fx, ...
                                  fxBlackVol, domestic_dH, domestic_Alpha, foreign_dH, foreign_Alpha)
                 
          % synchronize model parameters time-steps start

            fxBlackVolTenor = fxBlackVol.tenor;
            fxBlackVolQuote = fxBlackVol.quote;

            domestic_dHTenor = domestic_dH.tenor;
            domestic_dHQuote = domestic_dH.quote;

            foreign_dHTenor = foreign_dH.tenor;
            foreign_dHQuote = foreign_dH.quote;

            domestic_AlphaTenor = domestic_Alpha.tenor;
            domestic_AlphaQuote = domestic_Alpha.quote;

            foreign_AlphaTenor = foreign_Alpha.tenor;
            foreign_AlphaQuote = foreign_Alpha.quote;



            sigmaTimes = unique(union([domestic_AlphaTenor(:);foreign_AlphaTenor(:); ...
                         domestic_dHTenor(:);foreign_dHTenor(:)],fxBlackVolTenor));


            sigmaTimes = sort(sigmaTimes,'ascend');



            domestic_AlphaValues = zeros(length(sigmaTimes),1);
            foreign_AlphaValues = zeros(length(sigmaTimes),1);
            domestic_dHValues = zeros(length(sigmaTimes),1);
            foreign_dHValues = zeros(length(sigmaTimes),1);
            fxBlackVolValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                domestic_AlphaValues(k) = H_interpolation(domestic_AlphaTenor, domestic_AlphaQuote,sigmaTimes(k),0);
                foreign_AlphaValues(k) = H_interpolation(foreign_AlphaTenor, foreign_AlphaQuote,sigmaTimes(k),0);
                domestic_dHValues(k) = H_interpolation(domestic_dHTenor, domestic_dHQuote,sigmaTimes(k),0);
                foreign_dHValues(k) = H_interpolation(foreign_dHTenor, foreign_dHQuote,sigmaTimes(k),0);
                fxBlackVolValues(k) = H_interpolation(fxBlackVolTenor, fxBlackVolQuote,sigmaTimes(k),0);
            end
            
            % synchronize model parameters time-steps end
            
            var_x_x = quantolgm1f.LocalVariance_Alpha2(a,b,sigmaTimes,fxBlackVolValues,fxBlackVolValues);
            
            var_f_f = quantolgm1f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,foreign_dHValues,foreign_dHValues, ...
                                                         foreign_AlphaValues,foreign_AlphaValues);
                                                     
            var_d_d = quantolgm1f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dHValues,domestic_dHValues, ...
                                                         domestic_AlphaValues,domestic_AlphaValues);
                                                     
            covar_f_x =  1.0*corr_fx*-1.0*quantolgm1f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
                                            foreign_dHValues,foreign_AlphaValues,fxBlackVolValues);
                                        
            covar_d_x = -1.0*corr_dx*-1.0*quantolgm1f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
                                            domestic_dHValues,domestic_AlphaValues,fxBlackVolValues);
                                        
            covar_d_f =  -1.0*corr_df* 1.0*quantolgm1f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,...
                                            domestic_dHValues,foreign_dHValues,domestic_AlphaValues,foreign_AlphaValues);
                                                         
            value = var_x_x + var_f_f + var_d_d + 2.0*(covar_f_x + covar_d_x + covar_d_f);                                         
            
            out = value;                                         
                                                     
            
        end
        
        function out = FindFXBlackVol(quantolgm1f,index,params)
            
            expiry = params.fxImpVol.tenor(index);
            impVol = params.fxImpVol.quote(index);
            blackVariance = impVol*impVol*expiry;
            optParams.blackVariance = blackVariance;
            
            optParams.params = params;
            optParams.index = index;
            modelParamSize = 1;
            tvar = zeros(modelParamSize,1);
            lb = zeros(modelParamSize,1);
            ub = zeros(modelParamSize,1);
            
            tvar(1) = params.fxImpVol.quote(index);
            lb(1)= 0.0001;
            ub(1)= 10;
            
            %options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
            options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
            numOfEquation =1;
            x=  zeros(numOfEquation,1);
            [ret, popt, info, covar]=levmar('TargetFunctionBSLGMLGMVariance',tvar,x, 200, options,'unc',quantolgm1f,optParams);
            
            out.fxBlackVol_index = popt(1);
            out.re = TargetFunctionBSLGMLGMVariance(popt,quantolgm1f,optParams);
%             params.fxBlackVol.quote(index) = popt(1);

        end
        
        function out = CalibrateBSLGMLGMVol(quantolgm1f, paramsIn)
 
                expirySize = length(paramsIn.fxImpVol.tenor);
                out.fxBlackVol.quote = zeros(expirySize,1);
                out.fxBlackVol.tenor = paramsIn.fxImpVol.tenor;
                out.re =  zeros(expirySize,1);
                out.rmseTotal = 0;
                for i=1:expirySize
                    index = i;
                    BootstrapOut = quantolgm1f.FindFXBlackVol(index,paramsIn);
                    out.fxBlackVol.quote(i) = BootstrapOut.fxBlackVol_index;
                    params.fxBlackVol.quote(i) = BootstrapOut.fxBlackVol_index;
                    out.re(i) = BootstrapOut.re;
                    out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                end
                out.rmseTotal = out.rmseTotal/expirySize;
                
                out.rmseTotal = sqrt(out.rmseTotal);
                 
                
        end
        
        
    end
    
end



In [ ]:
classdef QuantoCK1F_LGM2F < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        domestic_dH_r;
        domestic_dH_s;
        
        foreign_meanR_r;
        
        domestic_Alpha_r;
        domestic_Alpha_s;
        
        foreign_vol_r;
        
        %market input
        corr_df;
        corr_dx;
        corr_fx;
        
        %modelParams
        
        corr_drds;
        
        corr_drfr;
        corr_dsfr;
        
        corr_drx;
        corr_dsx;
        
        corr_frx;
        
        
        fxBlackVol;
        
        numOfFactor;
        modelType;
        freq;
    end
    
    methods
        function quantock1f_lgm2f = QuantoCK1F_LGM2F(params)
            if nargin > 0

                quantock1f_lgm2f.zeroCurve = params.foreignModel.zeroCurve;
                quantock1f_lgm2f.freq = params.foreignModel.freq;
                
                quantock1f_lgm2f.numOfFactor = 1;
                quantock1f_lgm2f.modelType = 'QuantoCK1F_LGM2F';
                
                quantock1f_lgm2f.foreign_meanR_r = params.foreignModel.meanR_r;
                
                
                if params.fxVolCalibrated == true
                    % we do not calibrate fxBlackVol
                    quantock1f_lgm2f.corr_df = params.corr_df;
                    quantock1f_lgm2f.corr_dx = params.corr_dx;
                    quantock1f_lgm2f.corr_fx = params.corr_fx;
                    
                    
                    
                    out = quantock1f_lgm2f.synchronizeModelParams(params);
                    
                    quantock1f_lgm2f.corr_drds.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.corr_drds.quote = out.corr_drds_Values;
                    
                    quantock1f_lgm2f.corr_drfr.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.corr_drfr.quote = out.corr_drfr_Values;
                    
                    quantock1f_lgm2f.corr_dsfr.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.corr_dsfr.quote = out.corr_dsfr_Values;
                    
                    quantock1f_lgm2f.corr_drx.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.corr_drx.quote = out.corr_drx_Values;
                    
                    quantock1f_lgm2f.corr_dsx.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.corr_dsx.quote = out.corr_dsx_Values;
                    
                    quantock1f_lgm2f.corr_frx.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.corr_frx.quote = out.corr_frx_Values;
                    
                    quantock1f_lgm2f.domestic_Alpha_r.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.domestic_Alpha_r.quote = out.domestic_Alpha_r_Values;
                    
                    quantock1f_lgm2f.domestic_Alpha_s.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.domestic_Alpha_s.quote = out.domestic_Alpha_s_Values;

                    quantock1f_lgm2f.foreign_vol_r.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.foreign_vol_r.quote = out.foreign_vol_r_Values;

                    quantock1f_lgm2f.domestic_dH_r.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.domestic_dH_r.quote = out.domestic_dH_r_Values;

                    quantock1f_lgm2f.domestic_dH_s.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.domestic_dH_s.quote = out.domestic_dH_s_Values;
                    
                    quantock1f_lgm2f.fxBlackVol.tenor = out.sigmaTimes;
                    quantock1f_lgm2f.fxBlackVol.quote = out.fxBlackVolValues;
                    
                else
                    
                    % black vol calibration start
                    paramsIn.fxImpVol = params.fxBlackVol;
                    paramsIn.fxBlackVol = paramsIn.fxImpVol;
                    paramsIn.domestic_dH_r = params.domesticModel.dH1;
                    paramsIn.domestic_dH_s = params.domesticModel.dH2;
                    
                    paramsIn.domestic_Alpha_r = params.domesticModel.Alpha1;
                    paramsIn.domestic_Alpha_s = params.domesticModel.Alpha2;
                    
                    
                    paramsIn.foreign_meanR_r = params.foreignModel.meanR_r;
                    paramsIn.foreign_vol_r = params.foreignModel.vol_r;
                    
                    
                    paramsIn.corr_df = params.corr_df;
                    paramsIn.corr_dx = params.corr_dx;
                    paramsIn.corr_fx = params.corr_fx;
                    
                    paramsIn.corr_drds = params.corr_drds;
                    
                    paramsIn.corr_drfr = params.corr_drfr;
                    paramsIn.corr_dsfr = params.corr_dsfr;

                    paramsIn.corr_drx = params.corr_drx;
                    paramsIn.corr_dsx = params.corr_dsx;
                    
                    paramsIn.corr_frx = params.corr_frx;
                    
                    
                     % BSCK1FLGM2F Calibration Start
                    out = quantock1f_lgm2f.CalibrateBSCK1FLGM2FVol(paramsIn);
                    re =  out.re;
                    marketVar = out.marketVar;
                    modelVar = out.modelVar;
                    rmseTotal = out.rmseTotal;
                    
                    % BSCK1FLGM2F Calibration End
                    
                    params.fxBlackVol = out.fxBlackVol;
                    params.fxVolCalibrated =  true;
                    quantock1f_lgm2f = QuantoCK1F_LGM2F(params); 
                    
                end
                %end
                
                
            end
        end
        
        function out =  synchronizeModelParams(quantock1f_lgm2f,params)
            fxBlackVolTenor = params.fxBlackVol.tenor;
            fxBlackVolQuote = params.fxBlackVol.quote;

            % synchronize model parameters time-steps start
            corr_drds_Tenor = params.corr_drds.tenor;
            corr_drds_Quote = params.corr_drds.quote;
            
            corr_drfr_Tenor = params.corr_drfr.tenor;
            corr_drfr_Quote = params.corr_drfr.quote;
            
            corr_dsfr_Tenor = params.corr_dsfr.tenor;
            corr_dsfr_Quote = params.corr_dsfr.quote;
            
            corr_drx_Tenor = params.corr_drx.tenor;
            corr_drx_Quote = params.corr_drx.quote;
            
            corr_dsx_Tenor = params.corr_dsx.tenor;
            corr_dsx_Quote = params.corr_dsx.quote;
            
            corr_frx_Tenor = params.corr_frx.tenor;
            corr_frx_Quote = params.corr_frx.quote;
            
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;

            foreign_vol_r_Tenor = params.foreignModel.vol_r.tenor;
            foreign_vol_r_Quote = params.foreignModel.vol_r.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);corr_drfr_Tenor(:);corr_dsfr_Tenor(:);...
                         corr_drx_Tenor(:);corr_dsx_Tenor(:);corr_frx_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:);foreign_vol_r_Tenor(:); ...
                         domestic_dH_r_Tenor(:);domestic_dH_s_Tenor(:)],fxBlackVolTenor));


            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            corr_drfr_Values = zeros(length(sigmaTimes),1);
            corr_dsfr_Values = zeros(length(sigmaTimes),1);
            corr_drx_Values = zeros(length(sigmaTimes),1);
            corr_dsx_Values = zeros(length(sigmaTimes),1);
            corr_frx_Values = zeros(length(sigmaTimes),1);

            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            foreign_vol_r_Values = zeros(length(sigmaTimes),1);
            
            fxBlackVolValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                corr_drfr_Values(k) = H_interpolation(corr_drfr_Tenor, corr_drfr_Quote,sigmaTimes(k),0);
                corr_dsfr_Values(k) = H_interpolation(corr_dsfr_Tenor, corr_dsfr_Quote,sigmaTimes(k),0);
                corr_drx_Values(k) = H_interpolation(corr_drx_Tenor, corr_drx_Quote,sigmaTimes(k),0);
                corr_dsx_Values(k) = H_interpolation(corr_dsx_Tenor, corr_dsx_Quote,sigmaTimes(k),0);
                corr_frx_Values(k) = H_interpolation(corr_frx_Tenor, corr_frx_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_vol_r_Values(k) = H_interpolation(foreign_vol_r_Tenor, foreign_vol_r_Quote,sigmaTimes(k),0);
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                fxBlackVolValues(k) = H_interpolation(fxBlackVolTenor, fxBlackVolQuote,sigmaTimes(k),0);
            end
            
            out.sigmaTimes = sigmaTimes;
            
            out.corr_drds_Values = corr_drds_Values;
            out.corr_drfr_Values = corr_drfr_Values;
            out.corr_dsfr_Values = corr_dsfr_Values;
            out.corr_drx_Values = corr_drx_Values;
            out.corr_dsx_Values = corr_dsx_Values;
            out.corr_frx_Values = corr_frx_Values;
            
            out.domestic_Alpha_r_Values = domestic_Alpha_r_Values;
            out.domestic_Alpha_s_Values = domestic_Alpha_s_Values;
            
            out.foreign_vol_r_Values = foreign_vol_r_Values;
            
            out.domestic_dH_r_Values = domestic_dH_r_Values;
            out.domestic_dH_s_Values = domestic_dH_s_Values;
            
            out.fxBlackVolValues = fxBlackVolValues;
            
        end
        
        function out =  Libor_SimpleDate(quantock1f_lgm2f,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           
           % foreign Libor in domestic Terminal measure
           
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = quantock1f_lgm2f.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = quantock1f_lgm2f.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
        function out =  CMS_SimpleDate(quantock1f_lgm2f,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMT
%            tau = 0.25;
           tau = 1.0/(quantock1f_lgm2f.freq);
            % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           
           if abs(tenor- 0.25) < 1e-8
                tau = tenor;
           end
           
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tNTime = tNDate.DateDiff(valueDate);
           tN     =  tNTime/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           addMonthUnit = round(12*tau);
           cnt = round((tN-to)/tau);
           
           stateSize = size(modelStatesT,2);
           
           annuity = zeros(1,stateSize);
           df_Tp = zeros(1,stateSize);
           
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           for i=1:cnt
                tpDate = toDate.AddDate(i*addMonthUnit,'month');
                tpTime = tpDate.DateDiff(valueDate);
                df_Tp = quantock1f_lgm2f.discountFactor(observeTime,tpTime,numMatTime,modelStatesT);
                annuity = annuity + tau * df_Tp;
            end
           
           df_Tn = quantock1f_lgm2f.discountFactor(observeTime,observeTime,numMatTime,modelStatesT);
           df_TN = quantock1f_lgm2f.discountFactor(observeTime,tNTime,numMatTime,modelStatesT);
           
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           for i=1:stateSize
                cms_PCA_SimpleOut(i) = (df_Tn(i) - df_TN(i))/annuity(i);
           end
           
           out = cms_PCA_SimpleOut;

            
         end
        
         function out =  CMS_PSA_SimpleDate(quantock1f_lgm2f,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMT
%            tau = 0.25;
           tau = 1.0/(quantock1f_lgm2f.freq);
            % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           
           if abs(tenor- 0.25) < 1e-8
                tau = tenor;
           end
           
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = quantock1f_lgm2f.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = quantock1f_lgm2f.DF(to);
           df_TN = quantock1f_lgm2f.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = quantock1f_lgm2f.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = quantock1f_lgm2f.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
%            h_nTerm = quantock1f_lgm2f.H_nTermDate(valueDate,toDate,numMatDate);
           
           %h_nTerm is replace by i_Term & i_Tn_tilda in the following
           
           h_nTerm(1) = 0.0;
           
           varCovar = quantock1f_lgm2f.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:1
               for j=1:1
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           % additional drift correction term from foreign Term USD to
           % domestic Term measure KRW
           i_Term = I_Term(quantock1f_lgm2f,0,to,numMatTime/365.0);
           i_Tn_tilda = I_Tn_tilda(quantock1f_lgm2f,0,to,to);
           j_x_tilda = J_x_tilda(quantock1f_lgm2f,0,to,to);
           
%            additionalDrift = 0;
           additionalDrift = c_nN(1) * (i_Term + i_Tn_tilda + j_x_tilda);
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + additionalDrift + c_nN(1)*modelStatesT(1,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
         end
         
         function out = Foreign_HtT_r(quantock1f_lgm2f,t,T)
            out =  quantock1f_lgm2f.H(T,quantock1f_lgm2f.foreign_dH) - quantock1f_lgm2f.H(t,quantock1f_lgm2f.foreign_dH); 
         end
         
         function out = Domestic_HtT_r(quantock1f_lgm2f,t,T)
            out =  quantock1f_lgm2f.H(T,quantock1f_lgm2f.domestic_dH) - quantock1f_lgm2f.H(t,quantock1f_lgm2f.domestic_dH); 
         end
         
         function out = H(quantock1f_lgm2f,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
         end

        
        function out = fwdZcVarianceForeign(quantock1f_lgm2f, from, to, T1, T2)
            
            localVariance = zeros(2,1);
            measureChange = zeros(4,1);
            quantoChange = zeros(2,1);
            sigmaTimes = quantock1f_lgm2f.domestic_Alpha_r.tenor;
            
            foreign_vol_r_Values = quantock1f_lgm2f.foreign_vol_r.quote;
            
            domestic_Alpha_r_Values = quantock1f_lgm2f.domestic_Alpha_r.quote;
            domestic_Alpha_s_Values = quantock1f_lgm2f.domestic_Alpha_s.quote;
            
            domestic_dH_r_Values = quantock1f_lgm2f.domestic_dH_r.quote;
            domestic_dH_s_Values = quantock1f_lgm2f.domestic_dH_s.quote;
            
            fxValues = quantock1f_lgm2f.fxBlackVol.quote;
            
            MRS_r = quantock1f_lgm2f.foreign_meanR_r;
            
            correl_1 = ones(length(sigmaTimes),1);
            
            corr_frx_Values = quantock1f_lgm2f.corr_frx.quote;
            
            corr_drfr_Values = quantock1f_lgm2f.corr_drfr.quote;
            corr_dsfr_Values = quantock1f_lgm2f.corr_dsfr.quote;
            
            
            localVariance(1) = quantock1f_lgm2f.GeneralVariance_HWHW(from,to,T1,T1,MRS_r,MRS_r,sigmaTimes,foreign_vol_r_Values,foreign_vol_r_Values,correl_1);
            localVariance(2) = quantock1f_lgm2f.GeneralVariance_HWHW(from,to,to,to,MRS_r,MRS_r,sigmaTimes,foreign_vol_r_Values,foreign_vol_r_Values,correl_1);
            
            quantoChange(1) = -1.0*quantock1f_lgm2f.GeneralVariance_BSHW(from,to,T1,MRS_r,sigmaTimes,foreign_vol_r_Values,fxValues,corr_frx_Values);
            
            quantoChange(2) = -1.0*quantock1f_lgm2f.GeneralVariance_BSHW(from,to,to,MRS_r,sigmaTimes,foreign_vol_r_Values,fxValues,corr_frx_Values);
            
            
            measureChange(1) = quantock1f_lgm2f.GeneralVariance_HWLGM(from,to,T1,T2,MRS_r,sigmaTimes,domestic_dH_r_Values,domestic_Alpha_r_Values, ...
                                                                                                    foreign_vol_r_Values,corr_drfr_Values);
            
            measureChange(2) = quantock1f_lgm2f.GeneralVariance_HWLGM(from,to,to,T2,MRS_r,sigmaTimes,domestic_dH_r_Values,domestic_Alpha_r_Values, ...
                                                                                                    foreign_vol_r_Values,corr_drfr_Values);
                                                                                                
            measureChange(3) = quantock1f_lgm2f.GeneralVariance_HWLGM(from,to,T1,T2,MRS_r,sigmaTimes,domestic_dH_s_Values,domestic_Alpha_s_Values, ...
                                                                                                    foreign_vol_r_Values,corr_dsfr_Values);
            
            measureChange(4) = quantock1f_lgm2f.GeneralVariance_HWLGM(from,to,to,T2,MRS_r,sigmaTimes,domestic_dH_s_Values,domestic_Alpha_s_Values, ...
                                                                                                    foreign_vol_r_Values,corr_dsfr_Values);
            
                                                                                                
            
            value = localVariance(1) - localVariance(2);
            value = value -2.0 * -1.0 * (quantoChange(1) - quantoChange(2));
            
            value = value -2.0 * 1.0 * (measureChange(1) - measureChange(2) + measureChange(3) - measureChange(4));
            
            
            out = value;
            
        end
        
        function beta = Beta(quantock1f_lgm2f,t,T)
            beta =[];
            a_r = quantock1f_lgm2f.foreign_meanR_r;
            beta =[];
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0-exp(-a_r*(T-t)))/a_r;
            else
                beta(1) = (T-t);
            end
            
        end
        
        function beta = BetaDate(quantock1f_lgm2f,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            a_r = quantock1f_lgm2f.foreign_meanR_r;
            beta =[];
            
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0-exp(-a_r*(T-t)))/a_r;
            else
                beta(1) = (T-t);
            end
            
        end
        
        function c_nN = C_nN(quantock1f_lgm2f,tn,tN,tau,pnN,ynN)
            p = @quantock1f_lgm2f.DF;
            c_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = quantock1f_lgm2f.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
        function c_nN = C_nNDate(quantock1f_lgm2f,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @quantock1f_lgm2f.DF;
            c_nN=[0.0];
            beta = [0.0];
            df = 0.0;
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = quantock1f_lgm2f.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(1) = c_nN(1) +deltaT * beta(1) * df;
                prevTime = tp;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
        end
        
        function g_nN = G_nN(quantock1f_lgm2f,tn,tN,tau,pnN)
            p = @quantock1f_lgm2f.DF;
            g_nN=[0.0];
            
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = quantock1f_lgm2f.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function g_nN = G_nNDate(quantock1f_lgm2f,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @quantock1f_lgm2f.DF;
            g_nN=[0.0];
            
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = quantock1f_lgm2f.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(1) = g_nN(1) + deltaT * beta(1) * df;
                prevTime = tp;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function h_nTerm = H_nTerm(quantock1f_lgm2f,tn,tTerm)
           h_nTerm =[];
           beta = quantock1f_lgm2f.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function h_nTerm = H_nTermDate(quantock1f_lgm2f,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = quantock1f_lgm2f.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function i_Term = I_Term(quantock1f_lgm2f,from,to,tTerm)

            MRS1 = quantock1f_lgm2f.foreign_meanR_r;
            sigmaTimes = quantock1f_lgm2f.foreign_vol_r.tenor;
            foreign_vol_r_Values = quantock1f_lgm2f.foreign_vol_r.quote;
            
            domestic_dH_r_Values = quantock1f_lgm2f.domestic_dH_r.quote;
            domestic_Alpha_r_Values = quantock1f_lgm2f.domestic_Alpha_r.quote;
            corr_drfr_Values = quantock1f_lgm2f.corr_drfr.quote;
            
            domestic_dH_s_Values = quantock1f_lgm2f.domestic_dH_s.quote;
            domestic_Alpha_s_Values = quantock1f_lgm2f.domestic_Alpha_s.quote;
            corr_dsfr_Values = quantock1f_lgm2f.corr_dsfr.quote;
            
            variance_drfr = quantock1f_lgm2f.GeneralVariance_HWLGM_type2(from,to,to,tTerm,MRS1,sigmaTimes,domestic_dH_r_Values,domestic_Alpha_r_Values,...
                                                         foreign_vol_r_Values,corr_drfr_Values);
            
            variance_dsfr = quantock1f_lgm2f.GeneralVariance_HWLGM_type2(from,to,to,tTerm,MRS1,sigmaTimes,domestic_dH_s_Values,domestic_Alpha_s_Values,...
                 foreign_vol_r_Values,corr_dsfr_Values);
            
            % we add minus 1 so that we don't need additional minus in
            % drift adjust term
            i_Term = -1.0 * (variance_drfr + variance_dsfr);
           
        end
        
        function i_Tn_tilda = I_Tn_tilda(quantock1f_lgm2f,from,to,tn)

            MRS1 = quantock1f_lgm2f.foreign_meanR_r;
            sigmaTimes = quantock1f_lgm2f.foreign_vol_r.tenor;
            foreign_vol_r_Values = quantock1f_lgm2f.foreign_vol_r.quote;
            corr_frfr_Values = ones(length(sigmaTimes),1);
            
            variance_frfr = quantock1f_lgm2f.GeneralVariance_HWHW_type2(from,to,tn,tn,MRS1,MRS1,sigmaTimes,...
                                                                        foreign_vol_r_Values,foreign_vol_r_Values,corr_frfr_Values);
           % we don't need minus 1
           i_Tn_tilda = 1.0 * variance_frfr;
           
        end
        
        function i_Tn_tilda = I_Tn_tilda_old(quantock1f_lgm2f,from,to,tn)

            sigmaTimes = quantock1f_lgm2f.foreign_vol_r.tenor;
            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
           
           MRS2 = quantock1f_lgm2f.foreign_meanR_r;
           
           newExpScale = zeros(2,1);
           lastExpScale = zeros(2,1);
           variances = zeros(2,1);
            
           lastU  = from;
           
           lastExpScale(1) = exp(MRS2*lastU);
           lastExpScale(2) = exp((MRS2 + MRS2)*lastU);
           
           sigmaValues1 = quantock1f_lgm2f.foreign_vol_r.quote;
           sigmaValues2 = quantock1f_lgm2f.foreign_vol_r.quote;
           corr12 = 1.0;

           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                
                newExpScale(1) = exp(MRS2*U);
                variances(1) = variances(1) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(1)-lastExpScale(1));
                lastExpScale(1) = newExpScale(1);
                
                newExpScale(2) = exp((MRS2+MRS2)*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/(MRS2+MRS2)) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                lastU = U;
           end
           
           value = exp(-1.0*MRS2*tn)*(1.0/ MRS2)*variances(1);
           value = value + -1.0*exp(-1.0*(MRS2 + MRS2)*tn)*(1.0/ MRS2)*variances(2);
           % we don't need minus 1
           i_Tn_tilda = 1.0 * value;
           
        end
        
        
        
        function j_x_tilda = J_x_tilda(quantock1f_lgm2f,from,to,tn)

            MRS1 = quantock1f_lgm2f.foreign_meanR_r;
            sigmaTimes = quantock1f_lgm2f.foreign_vol_r.tenor;
            foreign_vol_r_Values = quantock1f_lgm2f.foreign_vol_r.quote;
            fxBlackVol_Values = quantock1f_lgm2f.fxBlackVol.quote;
            corr_frx_Values = quantock1f_lgm2f.corr_frx.quote;
            
            variance_frx = quantock1f_lgm2f.GeneralVariance_BSHW_type2(from,to,tn,MRS1,sigmaTimes,...
                                                                        foreign_vol_r_Values,fxBlackVol_Values,corr_frx_Values);
           
           % we need minus 1
           j_x_tilda = -1.0 * variance_frx;
           
        end
        
        function j_x_tilda = J_x_tilda_old(quantock1f_lgm2f,from,to,tn)

            sigmaTimes = quantock1f_lgm2f.foreign_vol_r.tenor;
            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
           
            
           newExpScale = zeros(1,1);
           lastExpScale = zeros(1,1);
           
           lastU  = from;
           MRS2 = quantock1f_lgm2f.foreign_meanR_r;
           lastExpScale(1) = exp(MRS2*lastU);
           variances = 0.0;
           
           sigmaValues1 = quantock1f_lgm2f.fxBlackVol.quote;
           sigmaValues2 = quantock1f_lgm2f.foreign_vol_r.quote;
           corrValues12 = quantock1f_lgm2f.corr_frx.quote;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) corr12 = corrValues12(i);else corr12 = corrValues12(sigmaSize);end;
                
                newExpScale(1) = exp(MRS2*U);
                variances = variances + corr12 * vol1 * vol2 * (1.0/MRS2) * (newExpScale(1)-lastExpScale(1));
                lastExpScale(1) = newExpScale(1);
                
                lastU = U;
           end
           
           % we need minus 1
           j_x_tilda = -1.0 * exp(-1.0*MRS2*tn) * (1.0/MRS2) * variances;
           
        end
        
        function out = discountFactor(quantock1f_lgm2f, startTime, endTime,numMatTime,modelStatesT)
           
            zcT = quantock1f_lgm2f.DF(endTime/365.0);
           
            %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = quantock1f_lgm2f.DF(startTime/365.0);
           beta = quantock1f_lgm2f.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = quantock1f_lgm2f.fwdZcVarianceForeign(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           % quantock1f_lgm2f is one factor model
           for i=1:stateSize
%                discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
           end
           
           out = discountFactorOut;
           
        end
        
%         function out = discountPayoff(quantock1f_lgm2f,eventTime,numMatTime ...
%                 ,modelStatesT,cashflow)
%             
%             dfNumMat = quantock1f_lgm2f.GetChildModel('d').DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             domesticModelStatesT = modelStatesT(1,:);
%             discountT = quantock1f_lgm2f.GetChildModel('d').discountFactor(eventTime, numMatTime,numMatTime,domesticModelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        function variances = LocalVariance(quantock1f_lgm2f,from,to)

            sig(1) = quantock1f_lgm2f.foreign_vol_r;
            meanR(1) = quantock1f_lgm2f.foreign_meanR_r;
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
            tiny = zeros(1,1);
            lastExpScale = zeros(1,1);
            newExpScale  = zeros(1,1);
            
            for i=1:1
                for j=1:1
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
            end
  
           lastU  = from;
           
           variances = zeros(1,1);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                
                correl =ones(1,1);
                for j=1:1
                    for k=1:1
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale =newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
           for j=1:1
                for k=1:1
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
           end
           
        end
        
        function out = LocalDrift(quantock1f_lgm2f,from,to)
            drift = eye(3,3);
            drift(3,3) =  exp(-1.0*quantock1f_lgm2f.foreign_meanR_r*(to-from));
            out = drift;
        end
        
        function variances = LocalCovariance(quantock1f_lgm2f,from,to)

            sig(1) = quantock1f_lgm2f.domestic_Alpha_r;
            sig(2) = quantock1f_lgm2f.domestic_Alpha_s;
            
            sig(3) = quantock1f_lgm2f.foreign_vol_r;
            
            corr_drds = quantock1f_lgm2f.corr_drds;
            corr_drfr = quantock1f_lgm2f.corr_drfr;
            corr_dsfr = quantock1f_lgm2f.corr_dsfr;
            
            correl = zeros(3,3);
            
            meanR = zeros(3,1);
            
            meanR(1) = 0.0;
            meanR(2) = 0.0;
            meanR(3) = quantock1f_lgm2f.foreign_meanR_r;
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
            tiny = zeros(3,3);
            lastExpScale = zeros(3,3);
            newExpScale  = zeros(3,3);
            
            for i=1:3
                for j=1:3
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
            end
            
  
           lastU  = from;
           
           variances = zeros(3,3);
           correl =ones(3,3);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end
                if (i< alphaSize) vol(3) = sig(3).quote(i);else vol(3) = sig(3).quote(alphaSize);end
                
                if (i< alphaSize) correl(1,2) = corr_drds.quote(i);else correl(1,2) = corr_drds.quote(alphaSize);end
                if (i< alphaSize) correl(1,3) = corr_drfr.quote(i);else correl(1,3) = corr_drfr.quote(alphaSize);end
                if (i< alphaSize) correl(2,3) = corr_dsfr.quote(i);else correl(2,3) = corr_dsfr.quote(alphaSize);end
                
                correl(2,1) = correl(1,2);
                correl(3,1) = correl(1,3);
                correl(3,2) = correl(2,3);
                
                for j=1:3
                    for k=1:3
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale =newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
           for j=1:3
                for k=1:3
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
            end
           
        end
        
        function out = LocalVariance_H2_Alpha2(quantock1f_lgm2f,from,to,sigmaTimes,dH1Values, dH2Values, sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1.quote = dH1Values;
           dH1.tenor = sigmaTimes;
           
           dH2.quote = dH2Values;
           dH2.tenor = sigmaTimes;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH1Value = dH1.quote(i);else dH1Value = dH1.quote(sigmaSize);end;
                if (i< sigmaSize) dH2Value = dH2.quote(i);else dH2Value = dH2.quote(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + quantock1f_lgm2f.H(lastU,dH1)*quantock1f_lgm2f.H(lastU,dH2)*correl*vol1*vol2*(U - lastU);
                variances = variances + dH1Value*quantock1f_lgm2f.H(lastU,dH2)*correl*vol1*vol2*0.5*(U - lastU)*(U - lastU);
                variances = variances + quantock1f_lgm2f.H(lastU,dH1)*dH2Value*correl*vol1*vol2*0.5*(U - lastU)*(U - lastU);
                variances = variances + dH1Value*dH2Value*correl*vol1*vol2*1.0/3.0*(U - lastU)*(U - lastU)*(U - lastU);
                
                
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = LocalVariance_H_Alpha2(quantock1f_lgm2f,from,to,sigmaTimes,dH1Values,sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1.quote = dH1Values;
           dH1.tenor = sigmaTimes;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH = dH1.quote(i);else dH = dH1.quote(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + quantock1f_lgm2f.H(lastU,dH1) * correl * vol1 * vol2 * (U - lastU);
                variances = variances + 0.5 * dH * correl * vol1 * vol2 * (U - lastU) * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = LocalVariance_Alpha2(quantock1f_lgm2f,from,to,sigmaTimes,sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + correl* vol1 * vol2 * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = GeneralVariance_HWHW(quantock1f_lgm2f,from,to,T1,T2,MRS1,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            % Variance in [from,to] of 1/a*(1-e^{-a(T1-s)})*1/b*(1-e^{-b(T2-s)})sigma1(s)sigma2(s)
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(4,1);
            lastExpScale = zeros(4,1);
            variances = zeros(4,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS1*lastU);
            lastExpScale(3) = exp(MRS2*lastU);
            lastExpScale(4) = exp((MRS1 + MRS2)*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end;
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end;
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end;
                
                variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS1*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                newExpScale(3) = exp(MRS2*U);
                variances(3) = variances(3) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(3)-lastExpScale(3));
                lastExpScale(3) = newExpScale(3);
                
                newExpScale(4) = exp((MRS1+MRS2)*U);
                variances(4) = variances(4) + corr12 * sig1 * sig2 * (1.0/(MRS1+MRS2)) * (newExpScale(4)-lastExpScale(4));
                lastExpScale(4) = newExpScale(4);
                
                lastU = U;
                
            end
            
            value = (1.0/ MRS1)*(1.0/MRS2)*variances(1);
            value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/ MRS1)*(1.0/MRS2)*variances(2);
            value = value + -1.0*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(3);
            value = value +  1.0*exp(-1.0*MRS1*T1)*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(4);

            out  = value;
        end
        
        function out = GeneralVariance_HWHW_type2(quantock1f_lgm2f,from,to,T1,T2,MRS1,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            % Variance in [a,b] of 1/a*(1-e^{-a(T1-s)})*e^{-b(T2-s)}*sigma1(s)sigma2(s)
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(4,1);
            lastExpScale = zeros(4,1);
            variances = zeros(4,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS1*lastU);
            lastExpScale(3) = exp(MRS2*lastU);
            lastExpScale(4) = exp((MRS1 + MRS2)*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end;
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end;
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end;
                
%                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
%                 newExpScale(2) = exp(MRS1*U);
%                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
%                 lastExpScale(2) = newExpScale(2);
                
                newExpScale(3) = exp(MRS2*U);
                variances(3) = variances(3) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(3)-lastExpScale(3));
                lastExpScale(3) = newExpScale(3);
                
                newExpScale(4) = exp((MRS1+MRS2)*U);
                variances(4) = variances(4) + corr12 * sig1 * sig2 * (1.0/(MRS1+MRS2)) * (newExpScale(4)-lastExpScale(4));
                lastExpScale(4) = newExpScale(4);
                
                lastU = U;
                
            end
            
%             value = (1.0/ MRS1)*(1.0/MRS2)*variances(1);
%             value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/ MRS1)*(1.0/MRS2)*variances(2);
            value =  +1.0*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*variances(3);
            value = value +  -1.0*exp(-1.0*MRS1*T1)*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*variances(4);

            out  = value;
        end
        
        function out = GeneralVariance_HWLGM(quantock1f_lgm2f,from,to,T1,T2,MRS1,sigmaTimes,dH1Values,sigmaValue1,sigmaValue2,corrValue)
            % Variance in [from,to] of 1/a*(1-e^{-a(T1-s)})*(H(T2)-H(s))*sigma1(s)sigma2(s)
            
            dH1.quote = dH1Values;
            dH1.tenor = sigmaTimes;
            
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(4,1);
            lastExpScale = zeros(4,1);
            variances = zeros(4,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS1*lastU);
            lastExpScale(3) = exp(MRS1*lastU);
            lastExpScale(4) = exp(MRS1*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
                if (i< sigmaSize) dH = dH1Values(i);else dH = dH1Values(sigmaSize);end
                
                variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS1*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                newExpScale(3) = exp(MRS1*U);
                variances(3) = variances(3) + quantock1f_lgm2f.H(lastU,dH1) * corr12 * sig1 * sig2 * (U - lastU);
                variances(3) = variances(3) + dH * corr12 * sig1 * sig2 * 0.5 * (U - lastU) * (U - lastU);
                lastExpScale(3) = newExpScale(3);
                
                newExpScale(4) = exp(MRS1*U);
                variances(4) = variances(4) + quantock1f_lgm2f.H(lastU,dH1) * corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
                variances(4) = variances(4) + dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (U*newExpScale(4)-lastU*lastExpScale(4));
                variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
                variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * lastU * (newExpScale(4)-lastExpScale(4));
                lastExpScale(4) = newExpScale(4);
                
                lastU = U;
                
            end
            
            value = quantock1f_lgm2f.H(T2,dH1)*(1.0/MRS1)*variances(1);
            value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/MRS1)*quantock1f_lgm2f.H(T2,dH1)*variances(2);
            value = value + -1.0*(1.0/MRS1)*variances(3);
            value = value +  1.0*exp(-1.0*MRS1*T1)*(1.0/MRS1)*variances(4);

            out  = value;
        end
        
        function out = GeneralVariance_HWLGM_type2(quantock1f_lgm2f,from,to,T1,T2,MRS1,sigmaTimes,dH1Values,sigmaValue1,sigmaValue2,corrValue)
        % Variance in [a,b] of e^{-a(T1-s)}*(H(T2)-H(s))*sigma1(s)sigma2(s)
            
            dH1.quote = dH1Values;
            dH1.tenor = sigmaTimes;
            
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(4,1);
            lastExpScale = zeros(4,1);
            variances = zeros(4,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS1*lastU);
            lastExpScale(3) = exp(MRS1*lastU);
            lastExpScale(4) = exp(MRS1*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
                if (i< sigmaSize) dH = dH1Values(i);else dH = dH1Values(sigmaSize);end
                
%                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS1*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
%                 newExpScale(3) = exp(MRS1*U);
%                 variances(3) = variances(3) + quantock1f_lgm2f.H(lastU,dH1) * corr12 * sig1 * sig2 * (U - lastU);
%                 variances(3) = variances(3) + dH * corr12 * sig1 * sig2 * 0.5 * (U - lastU) * (U - lastU);
%                 lastExpScale(3) = newExpScale(3);
                
                newExpScale(4) = exp(MRS1*U);
                variances(4) = variances(4) + quantock1f_lgm2f.H(lastU,dH1) * corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
                variances(4) = variances(4) + dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (U*newExpScale(4)-lastU*lastExpScale(4));
                variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
                variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * lastU * (newExpScale(4)-lastExpScale(4));
                lastExpScale(4) = newExpScale(4);
                
                lastU = U;
                
            end
            
%             value = quantock1f_lgm2f.H(T2,dH1)*(1.0/MRS1)*variances(1);
%             value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/MRS1)*quantock1f_lgm2f.H(T2,dH1)*variances(2);
            value = +1.0*exp(-1.0*MRS1*T1)*quantock1f_lgm2f.H(T2,dH1)*variances(2);
            
%             value = value + -1.0*(1.0/MRS1)*variances(3);
            value = value +  -1.0*exp(-1.0*MRS1*T1)*variances(4);

            out  = value;
        end
        
        function out = GeneralVariance_BSHW(quantock1f_lgm2f,from,to,T2,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            % Variance in [from,to] of 1/a*(1-e^{-a(T1-s)})sigma1(s)sigma2(s)
            
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(2,1);
            lastExpScale = zeros(2,1);
            variances = zeros(2,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS2*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
                
                variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS2*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                lastU = U;
                
            end
            
            value = (1.0/MRS2)*variances(1);
            value = value + -1.0*exp(-1.0*MRS2*T2)*(1.0/MRS2)*variances(2);
            
            out  = value;
        end
        
        function out = GeneralVariance_BSHW_type2(quantock1f_lgm2f,from,to,T2,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            % Variance in [a,b] of (e^{-a(T1-s)})sigma1(s)sigma2(s)
            
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(2,1);
            lastExpScale = zeros(2,1);
            variances = zeros(2,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS2*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
                
%                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS2*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                lastU = U;
                
            end
            
%             value = (1.0/MRS2)*variances(1);
            value = +1.0*exp(-1.0*MRS2*T2)*variances(2);
            
            out  = value;
        end
        
        function out = GeneralVariance_LGMLGM(quantock1f_lgm2f,a,b,T1,T2,sigmaTimes,dH1Values,dH2Values,Alpha1,Alpha2,corrValues)
            % Variance in [a,b] of (H(T1)-H(s))(H(T2)-H(s))alpha1(s)alpha2(s)
            dH1.quote = dH1Values;
            dH1.tenor = sigmaTimes;
            
            dH2.quote = dH2Values;
            dH2.tenor = sigmaTimes;
            
            variances = zeros(4,1);
            variances(1) = quantock1f_lgm2f.LocalVariance_Alpha2(a,b,sigmaTimes, Alpha1,Alpha2,corrValues);
            variances(1) = variances(1) * quantock1f_lgm2f.H(T1,dH1) * quantock1f_lgm2f.H(T2,dH2);
            
            variances(2) = quantock1f_lgm2f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH1Values,Alpha1,Alpha2,corrValues);
            variances(2) = variances(2) * -1.0* quantock1f_lgm2f.H(T2,dH2);
            
            variances(3) = quantock1f_lgm2f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH2Values,Alpha1,Alpha2,corrValues);
            variances(3) = variances(3) * -1.0* quantock1f_lgm2f.H(T1,dH1);
            
            variances(4) = quantock1f_lgm2f.LocalVariance_H2_Alpha2(a,b,sigmaTimes,dH1Values,dH2Values,Alpha1,Alpha2,corrValues);
            
            value = variances(1) + variances(2) + variances(3) + variances(4);
            out = value;
        end
        
        function out = GeneralVariance_BSLGM(quantock1f_lgm2f,a,b,T1,sigmaTimes,dH1Values,Alpha1,Alpha2,corrValues)
            % Variance in [a,b] of (H(T1)-H(s))alpha1(s)alpha2(s)
            dH1.quote = dH1Values;
            dH1.tenor = sigmaTimes;
            
            variances = zeros(2,1);
            variances(1) = quantock1f_lgm2f.LocalVariance_Alpha2(a,b,sigmaTimes, Alpha1,Alpha2,corrValues);
            variances(1) = variances(1) * quantock1f_lgm2f.H(T1,dH1);
            
            variances(2) = quantock1f_lgm2f.LocalVariance_H_Alpha2(a,b,sigmaTimes,dH1Values,Alpha1,Alpha2,corrValues);
            variances(2) = variances(2) * -1.0;
            
            value = variances(1) + variances(2);
            out = value;
        end
        
        function out = CMSDigitalArgDate(quantock1f_lgm2f,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           to = toDate.DateDiff(valueDate)/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           tauIn = tau;
           if abs(tenor1- 0.25) < 1e-8
                tauIn = tenor1;
           end
           
           p_nN1 = quantock1f_lgm2f.PV01Date(valueDate,toDate,tN1Date,tauIn);
           
           df_Tn1 = quantock1f_lgm2f.DF(to);
           df_TN1 = quantock1f_lgm2f.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = quantock1f_lgm2f.C_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1,y_nN1);
           g_nN1 = quantock1f_lgm2f.G_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1);
%            h_nTerm1 = quantock1f_lgm2f.H_nTermDate(valueDate,toDate,tpDate);
           % h_nTerm1 is replaced with i_Term & i_Tn_tilda as in the
           % following discussion
           
           h_nTerm1(1) = 0.0;
           
           varCovar = quantock1f_lgm2f.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           tpTime = tpDate.DateDiff(valueDate);
           % additional drift correction term from foreign Term measure USD to KRW domestic Term 
           i_Term = I_Term(quantock1f_lgm2f,0,to,tpTime/365.0);
           i_Tn_tilda = I_Tn_tilda(quantock1f_lgm2f,0,to,to);
           j_x_tilda = J_x_tilda(quantock1f_lgm2f,0,to,to);
           
%            additionalDrift = 0;
           additionalDrift = c_nN1(1) *(i_Term + i_Tn_tilda + j_x_tilda);
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.fwdCMS = y_nN1;
           out.mu = y_nN1 + driftTerm + additionalDrift;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
        end
        
        function out =computeBSCK1FLGM2FVariance(quantock1f_lgm2f,a,b,T,corr_drds,corr_drfr,corr_dsfr,corr_drx,corr_dsx,corr_frx, ...
                                  fxBlackVol, domestic_dH_r,domestic_dH_s, domestic_Alpha_r,domestic_Alpha_s,...
                                  foreign_meanR_r, foreign_vol_r)
                 
          % synchronize model parameters time-steps start
            corr_drds_Tenor = corr_drds.tenor;
            corr_drds_Quote = corr_drds.quote;
            
            corr_drfr_Tenor = corr_drfr.tenor;
            corr_drfr_Quote = corr_drfr.quote;
            
            corr_dsfr_Tenor = corr_dsfr.tenor;
            corr_dsfr_Quote = corr_dsfr.quote;
            
            corr_drx_Tenor = corr_drx.tenor;
            corr_drx_Quote = corr_drx.quote;
            
            corr_dsx_Tenor = corr_dsx.tenor;
            corr_dsx_Quote = corr_dsx.quote;
            
            corr_frx_Tenor = corr_frx.tenor;
            corr_frx_Quote = corr_frx.quote;
            
            
            fxBlackVolTenor = fxBlackVol.tenor;
            fxBlackVolQuote = fxBlackVol.quote;

            domestic_dH_r_Tenor = domestic_dH_r.tenor;
            domestic_dH_r_Quote = domestic_dH_r.quote;

            domestic_dH_s_Tenor = domestic_dH_s.tenor;
            domestic_dH_s_Quote = domestic_dH_s.quote;
            
            domestic_Alpha_r_Tenor = domestic_Alpha_r.tenor;
            domestic_Alpha_r_Quote = domestic_Alpha_r.quote;

            domestic_Alpha_s_Tenor = domestic_Alpha_s.tenor;
            domestic_Alpha_s_Quote = domestic_Alpha_s.quote;

            foreign_vol_r_Tenor = foreign_vol_r.tenor;
            foreign_vol_r_Quote = foreign_vol_r.quote;

            


            sigmaTimes = unique(union([corr_drds_Tenor(:);corr_drfr_Tenor(:);corr_dsfr_Tenor(:);...
                         corr_drx_Tenor(:);corr_dsx_Tenor(:);corr_frx_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:);foreign_vol_r_Tenor(:); ...
                         domestic_dH_r_Tenor(:);domestic_dH_s_Tenor(:)],fxBlackVolTenor));


            sigmaTimes = sort(sigmaTimes,'ascend');

            corr_drds_Values = zeros(length(sigmaTimes),1);
            corr_drfr_Values = zeros(length(sigmaTimes),1);
            corr_dsfr_Values = zeros(length(sigmaTimes),1);
            corr_drx_Values = zeros(length(sigmaTimes),1);
            corr_dsx_Values = zeros(length(sigmaTimes),1);
            corr_frx_Values = zeros(length(sigmaTimes),1);
            

            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            foreign_vol_r_Values = zeros(length(sigmaTimes),1);
            
            fxBlackVolValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                corr_drfr_Values(k) = H_interpolation(corr_drfr_Tenor, corr_drfr_Quote,sigmaTimes(k),0);
                corr_dsfr_Values(k) = H_interpolation(corr_dsfr_Tenor, corr_dsfr_Quote,sigmaTimes(k),0);
                corr_drx_Values(k) = H_interpolation(corr_drx_Tenor, corr_drx_Quote,sigmaTimes(k),0);
                corr_dsx_Values(k) = H_interpolation(corr_dsx_Tenor, corr_dsx_Quote,sigmaTimes(k),0);
                corr_frx_Values(k) = H_interpolation(corr_frx_Tenor, corr_frx_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_vol_r_Values(k) = H_interpolation(foreign_vol_r_Tenor, foreign_vol_r_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                fxBlackVolValues(k) = H_interpolation(fxBlackVolTenor, fxBlackVolQuote,sigmaTimes(k),0);
            end
            
            % synchronize model parameters time-steps end
            % dummy correlation
            correl_1 = ones(length(sigmaTimes),1);
             
            var_x_x = quantock1f_lgm2f.LocalVariance_Alpha2(a,b,sigmaTimes,fxBlackVolValues,fxBlackVolValues,correl_1);
            
            var_fr_fr = quantock1f_lgm2f.GeneralVariance_HWHW(a,b,T,T,foreign_meanR_r,foreign_meanR_r,sigmaTimes, ...
                                        foreign_vol_r_Values,foreign_vol_r_Values,correl_1);
                                                     
            var_dr_dr = quantock1f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_r_Values,domestic_dH_r_Values, ...
                                                         domestic_Alpha_r_Values,domestic_Alpha_r_Values,correl_1);
                                                     
            var_ds_ds = quantock1f_lgm2f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dH_s_Values,domestic_dH_s_Values, ...
                                                         domestic_Alpha_s_Values,domestic_Alpha_s_Values,correl_1);
           % 1st= sign in techNote , 2nd = HW bond vol? -1,1                               
            covar_fr_x =  1.0*-1.0*quantock1f_lgm2f.GeneralVariance_BSHW(a,b,T,foreign_meanR_r,sigmaTimes, ...
                                            foreign_vol_r_Values,fxBlackVolValues,corr_frx_Values);
                                        
            covar_dr_x = -1.0*-1.0*quantock1f_lgm2f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
                                            domestic_dH_r_Values,domestic_Alpha_r_Values,fxBlackVolValues,corr_drx_Values);
                                        
            covar_ds_x = -1.0*-1.0*quantock1f_lgm2f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
                                            domestic_dH_s_Values,domestic_Alpha_s_Values,fxBlackVolValues,corr_dsx_Values);
                                        
            covar_dr_fr =  -1.0*1.0*quantock1f_lgm2f.GeneralVariance_HWLGM(a,b,T,T,foreign_meanR_r,sigmaTimes,...
                                            domestic_dH_r_Values,domestic_Alpha_r_Values,foreign_vol_r_Values,corr_drfr_Values);
                                        
            covar_ds_fr =  -1.0*1.0*quantock1f_lgm2f.GeneralVariance_HWLGM(a,b,T,T,foreign_meanR_r,sigmaTimes,...
                                            domestic_dH_s_Values,domestic_Alpha_s_Values,foreign_vol_r_Values,corr_dsfr_Values);
                                        
            covar_dr_ds =  1.0*1.0*0.0;  % corr_drds is zero
                                                         
            value = var_x_x + var_fr_fr + var_dr_dr + var_ds_ds + 2.0*(covar_fr_x + covar_dr_x + covar_ds_x + covar_dr_fr + covar_ds_fr + covar_dr_ds);                                         
            
            out = value;                                         
                                                     
            
        end
        
        function out = FindFXBlackVol(quantock1f_lgm2f,index,params)
            
            expiry = params.fxImpVol.tenor(index);
            impVol = params.fxImpVol.quote(index);
            blackVariance = impVol*impVol*expiry;
            optParams.blackVariance = blackVariance;
            
            optParams.params = params;
            optParams.index = index;
            modelParamSize = 1;
            tvar = zeros(modelParamSize,1);
            lb = zeros(modelParamSize,1);
            ub = zeros(modelParamSize,1);
            
            tvar(1) = params.fxImpVol.quote(index);
            lb(1)= 0.0001;
            ub(1)= 10;
            
            %options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
            options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
            numOfEquation =1;
            x=  zeros(numOfEquation,1);
            [ret, popt, info, covar]=levmar('TargetFunctionBSCK1FLGM2FVariance',tvar,x, 500, options,'unc',quantock1f_lgm2f,optParams);
            
            out.fxBlackVol_index = popt(1);
            out.re = TargetFunctionBSCK1FLGM2FVariance(popt,quantock1f_lgm2f,optParams);
            out.marketVar = blackVariance;
            out.modelVar = (1.0+out.re)*blackVariance;
%             params.fxBlackVol.quote(index) = popt(1);

        end
        
        function out = CalibrateBSCK1FLGM2FVol(quantock1f_lgm2f, paramsIn)
 
                expirySize = length(paramsIn.fxImpVol.tenor);
                out.fxBlackVol.quote = zeros(expirySize,1);
                out.fxBlackVol.tenor = paramsIn.fxImpVol.tenor;
                out.re =  zeros(expirySize,1);
                out.modelVar = zeros(expirySize,1);
                out.marketVar = zeros(expirySize,1);
                out.rmseTotal = 0;
                for i=1:expirySize
                    index = i;
                    BootstrapOut = quantock1f_lgm2f.FindFXBlackVol(index,paramsIn);
                    out.fxBlackVol.quote(i) = BootstrapOut.fxBlackVol_index;
                    paramsIn.fxBlackVol.quote(i) = BootstrapOut.fxBlackVol_index;
                    out.re(i) = BootstrapOut.re;
                    out.marketVar(i) = BootstrapOut.marketVar;
                    out.modelVar(i) = BootstrapOut.modelVar;
                    out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                end
                out.rmseTotal = out.rmseTotal/expirySize;
                
                out.rmseTotal = sqrt(out.rmseTotal);
                 
                
        end
        
        
    end
    
end



In [ ]:
classdef QuantoCK1F_CK1F < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        domestic_meanR_r;
        foreign_meanR_r;
        
        domestic_vol_r;
        foreign_vol_r;
        
        %market input
        corr_df;
        corr_dx;
        corr_fx;
        
        %modelParams
        corr_drfr;
        corr_drx;        
        corr_frx;
        
        fxBlackVol;
        
        numOfFactor;
        modelType;
        freq;
    end
    
    methods
        function quantock1f_ck1f = QuantoCK1F_CK1F(params)
            if nargin > 0
                quantock1f_ck1f.zeroCurve = params.foreignModel.zeroCurve;
                quantock1f_ck1f.freq = params.foreignModel.freq;
                
                quantock1f_ck1f.numOfFactor = 1;
                quantock1f_ck1f.modelType = 'QuantoCK1F_CK1F';
                
                quantock1f_ck1f.domestic_meanR_r = params.domesticModel.meanR_r;
                quantock1f_ck1f.foreign_meanR_r = params.foreignModel.meanR_r;
                
                if params.fxVolCalibrated == true
                    % we do not calibrate fxBlackVol
                    quantock1f_ck1f.corr_df = params.corr_df;
                    quantock1f_ck1f.corr_dx = params.corr_dx;
                    quantock1f_ck1f.corr_fx = params.corr_fx;
                    
                    out = quantock1f_ck1f.synchronizeModelParams(params);
                                        
                    quantock1f_ck1f.corr_drfr.tenor = out.sigmaTimes;
                    quantock1f_ck1f.corr_drfr.quote = out.corr_drfr_Values;
                    
                    quantock1f_ck1f.corr_drx.tenor = out.sigmaTimes;
                    quantock1f_ck1f.corr_drx.quote = out.corr_drx_Values;

                    quantock1f_ck1f.corr_frx.tenor = out.sigmaTimes;
                    quantock1f_ck1f.corr_frx.quote = out.corr_frx_Values;
                    
                    quantock1f_ck1f.domestic_vol_r.tenor = out.sigmaTimes;
                    quantock1f_ck1f.domestic_vol_r.quote = out.domestic_vol_r_Values;
                    
                    quantock1f_ck1f.foreign_vol_r.tenor = out.sigmaTimes;
                    quantock1f_ck1f.foreign_vol_r.quote = out.foreign_vol_r_Values;
                    
                    quantock1f_ck1f.fxBlackVol.tenor = out.sigmaTimes;
                    quantock1f_ck1f.fxBlackVol.quote = out.fxBlackVolValues;
                    
                else                    
                    % black vol calibration start
                    paramsIn.fxImpVol = params.fxBlackVol;
                    paramsIn.fxBlackVol = paramsIn.fxImpVol;
                    
                    paramsIn.domestic_meanR_r = params.domesticModel.meanR_r;
                    paramsIn.domestic_vol_r = params.domesticModel.vol_r;
                    
                    paramsIn.foreign_meanR_r = params.foreignModel.meanR_r;
                    paramsIn.foreign_vol_r = params.foreignModel.vol_r;
                    
                    paramsIn.corr_df = params.corr_df;
                    paramsIn.corr_dx = params.corr_dx;
                    paramsIn.corr_fx = params.corr_fx;
                    
                    paramsIn.corr_drfr = params.corr_drfr;
                    paramsIn.corr_drx = params.corr_drx;
                    paramsIn.corr_frx = params.corr_frx;
                    
                     % BSCK1FLGM2F Calibration Start
                    out = quantock1f_ck1f.CalibrateBSCK1FLGM2FVol(paramsIn);
                    re =  out.re;
                    
                    marketVar = out.marketVar;
                    modelVar = out.modelVar;
                    rmseTotal = out.rmseTotal;
                    
                    % BSCK1FLGM2F Calibration End
                    
                    params.fxBlackVol = out.fxBlackVol;
                    params.fxVolCalibrated =  true;
                    quantock1f_ck1f = QuantoCK1F_CK1F(params); 
                    
                end
            end
        end
        
        function out =  synchronizeModelParams(quantock1f_ck1f, params)
            fxBlackVolTenor = params.fxBlackVol.tenor;
            fxBlackVolQuote = params.fxBlackVol.quote;

            % synchronize model parameters time-steps start
            corr_drfr_Tenor = params.corr_drfr.tenor;
            corr_drfr_Quote = params.corr_drfr.quote;
                        
            corr_drx_Tenor = params.corr_drx.tenor;
            corr_drx_Quote = params.corr_drx.quote;
            
            corr_frx_Tenor = params.corr_frx.tenor;
            corr_frx_Quote = params.corr_frx.quote;
            
            domestic_vol_r_Tenor = params.domesticModel.vol_r.tenor;
            domestic_vol_r_Quote = params.domesticModel.vol_r.quote;

            foreign_vol_r_Tenor = params.foreignModel.vol_r.tenor;
            foreign_vol_r_Quote = params.foreignModel.vol_r.quote;

            sigmaTimes = unique(union([corr_drfr_Tenor(:);...
                         corr_drx_Tenor(:); corr_frx_Tenor(:);...
                         domestic_vol_r_Tenor(:); foreign_vol_r_Tenor(:)], fxBlackVolTenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drfr_Values = zeros(length(sigmaTimes),1);
            corr_drx_Values = zeros(length(sigmaTimes),1);
            corr_frx_Values = zeros(length(sigmaTimes),1);
            
            domestic_vol_r_Values = zeros(length(sigmaTimes), 1);
            foreign_vol_r_Values = zeros(length(sigmaTimes), 1);
            
            fxBlackVolValues = zeros(length(sigmaTimes),1);
            
            for k=1:length(sigmaTimes)
                corr_drfr_Values(k) = H_interpolation(corr_drfr_Tenor, corr_drfr_Quote,sigmaTimes(k), 0);
                corr_drx_Values(k) = H_interpolation(corr_drx_Tenor, corr_drx_Quote,sigmaTimes(k), 0);
                corr_frx_Values(k) = H_interpolation(corr_frx_Tenor, corr_frx_Quote,sigmaTimes(k), 0);
                
                domestic_vol_r_Values(k) = H_interpolation(domestic_vol_r_Tenor, domestic_vol_r_Quote, sigmaTimes(k), 0);
                foreign_vol_r_Values(k) = H_interpolation(foreign_vol_r_Tenor, foreign_vol_r_Quote,sigmaTimes(k),0);
                
                fxBlackVolValues(k) = H_interpolation(fxBlackVolTenor, fxBlackVolQuote,sigmaTimes(k),0);
            end
            
            out.sigmaTimes = sigmaTimes;
            
            out.corr_drfr_Values = corr_drfr_Values;
            out.corr_drx_Values = corr_drx_Values;
            out.corr_frx_Values = corr_frx_Values;
            
            out.domestic_vol_r_Values = domestic_vol_r_Values;
            out.foreign_vol_r_Values = foreign_vol_r_Values;
            
            out.fxBlackVolValues = fxBlackVolValues;
            
        end
        
        function out =  Libor_SimpleDate(quantock1f_ck1f, valueDate, evalTime, fwdStartTime, numMatTime, tenor, modelStatesT)
           % foreign Libor in domestic Terminal measure
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = quantock1f_ck1f.discountFactor(evalTime, fwdStartTime, numMatTime, modelStatesT);
           df_TN = quantock1f_ck1f.discountFactor(evalTime, fwdEndTime, numMatTime, modelStatesT);
           
           liborOut = zeros(1, stateSize);
           
           for i = 1:stateSize
                liborOut(i) = (df_Tn(i) / df_TN(i) - 1.0) / deltaT;
           end
           
           out = liborOut;
            
        end
        
        function out = CMS_SimpleDate(quantock1f_ck1f, valueDate, observeTime, numMatTime, tenor, modelStatesT)
           % CMT
%            tau = 0.25;
           tau = 1.0 / (quantock1f_ck1f.freq);
           % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           
           if abs(tenor - 0.25) < 1e-8
                tau = tenor;
           end
           
           to     = observeTime / 365.0;
           toDate = valueDate.AddDate(observeTime, 'day');
           tNDate = toDate.AddDate(tenor, 'year');
           tNTime = tNDate.DateDiff(valueDate);
           tN     =  tNTime / 365.0;
           
           numMatDate = valueDate.AddDate(numMatTime, 'day');
           
           addMonthUnit = round(12 * tau);
           cnt = round((tN - to) / tau);
           
           stateSize = size(modelStatesT, 2);
           
           annuity = zeros(1, stateSize);
           df_Tp = zeros(1, stateSize);           
           df_Tn = zeros(1, stateSize);
           df_TN = zeros(1, stateSize);
           
           for i = 1:cnt
               tpDate = toDate.AddDate(i * addMonthUnit,'month');
               tpTime = tpDate.DateDiff(valueDate);
               df_Tp = quantock1f_ck1f.discountFactor(observeTime, tpTime, numMatTime, modelStatesT);
               annuity = annuity + tau * df_Tp;
           end
           
           df_Tn = quantock1f_ck1f.discountFactor(observeTime, observeTime, numMatTime, modelStatesT);
           df_TN = quantock1f_ck1f.discountFactor(observeTime, tNTime, numMatTime, modelStatesT);
           
           cms_PCA_SimpleOut = zeros(1, stateSize);
           
           for i = 1:stateSize
                cms_PCA_SimpleOut(i) = (df_Tn(i) - df_TN(i)) / annuity(i);
           end
           
           out = cms_PCA_SimpleOut;
            
        end
        
        function out =  CMS_PSA_SimpleDateGeneric(quantock1f_ck1f,valueDate,observeTime,numMatTime, tenor,modelStatesT)
            if observeTime < 0
                allFixings = quantock1f_ck1f.modelParams('allFixings');
                fixingDate = AddDate(valueDate,observeTime,'day');
                % if fixing is less than a year then we assume that CMS_PSA as Libor
                if tenor < 1
                    tenorMonth = round(tenor*12);
                    fixingTicker = ['LIBOR' num2str(tenorMonth,'%d') 'M'];
                else
                    fixingTicker = ['CMS' num2str(tenor,'%d') 'Y'];
                end
                irIndexFixings = allFixings(fixingTicker);
                try
                    fixing = irIndexFixings(StrDate(fixingDate,'str'));
                    stateSize = size(modelStatesT,2);
                    out = ones(1,stateSize)*fixing;
                    
                catch exception
                    throw(exception)
                end
                
            else
                out = CMS_PSA_SimpleDate(quantock1f_ck1f,valueDate,observeTime,numMatTime, tenor,modelStatesT);
            end
        
        end
        
         % Should check variance functions for HWHW later. 2020-02-18 by LIB
         function out =  CMS_PSA_SimpleDate(quantock1f_ck1f,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMT
%            tau = 0.25;
           tau = 1.0/(quantock1f_ck1f.freq);
            % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           
           if abs(tenor- 0.25) < 1e-8
                tau = tenor;
           end
           
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = quantock1f_ck1f.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = quantock1f_ck1f.DF(to);
           df_TN = quantock1f_ck1f.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = quantock1f_ck1f.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = quantock1f_ck1f.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
%            h_nTerm = quantock1f_ck1f.H_nTermDate(valueDate,toDate,numMatDate);
           
           %h_nTerm is replace by i_Term & i_Tn_tilda in the following
           
           h_nTerm(1) = 0.0;
           
           varCovar = quantock1f_ck1f.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:1
               for j=1:1
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           % additional drift correction term from foreign Term USD to
           % domestic Term measure KRW
           i_Term = I_Term(quantock1f_ck1f,0,to,numMatTime/365.0);
           i_Tn_tilda = I_Tn_tilda(quantock1f_ck1f,0,to,to);
           j_x_tilda = J_x_tilda(quantock1f_ck1f,0,to,to);
           
%            additionalDrift = 0;
           additionalDrift = c_nN(1) * (i_Term + i_Tn_tilda + j_x_tilda);
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + additionalDrift + c_nN(1)*modelStatesT(1,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
         end
         
%          function out = Foreign_HtT_r(quantock1f_ck1f,t,T)
%             out =  quantock1f_ck1f.H(T,quantock1f_ck1f.foreign_dH) - quantock1f_ck1f.H(t,quantock1f_ck1f.foreign_dH); 
%          end
%          
%          function out = Domestic_HtT_r(quantock1f_ck1f,t,T)
%             out =  quantock1f_ck1f.H(T,quantock1f_ck1f.domestic_dH) - quantock1f_ck1f.H(t,quantock1f_ck1f.domestic_dH); 
%          end
         
%          function out = H(quantock1f_ck1f,t,dH)
%             for i=1:length(dH.tenor)
%                 if t<= dH.tenor(i)
%                     break;
%                 end
%             end
%             tidx = i;
%             sum = 0.0;
%             prev_t = 0.0;
%             curr_t = 0.0;
%             for i=1:tidx
%                 if i< tidx
%                     curr_t = dH.tenor(i);
%                 else
%                     curr_t = t;
%                 end
%                 sum = sum + dH.quote(i)*(curr_t-prev_t);
% 
%                 prev_t = dH.tenor(i);
%             end
%             out = sum;
%          end
        
        function out = fwdZcVarianceForeign(quantock1f_ck1f, from, to, T1, T2)
            localVariance = zeros(2,1);
            measureChange = zeros(2,1);
            quantoChange = zeros(2,1);
            
            sigmaTimes = quantock1f_ck1f.domestic_vol_r.tenor;
            
            domestic_vol_r_Values = quantock1f_ck1f.domestic_vol_r.quote;
            foreign_vol_r_Values = quantock1f_ck1f.foreign_vol_r.quote;
            
            fxValues = quantock1f_ck1f.fxBlackVol.quote;
            
            MRS_dr = quantock1f_ck1f.domestic_meanR_r;
            MRS_fr = quantock1f_ck1f.foreign_meanR_r;
            
            correl_1 = ones(length(sigmaTimes),1);
            corr_frx_Values = quantock1f_ck1f.corr_frx.quote;
            corr_drfr_Values = quantock1f_ck1f.corr_drfr.quote;
            
            localVariance(1) = quantock1f_ck1f.GeneralVariance_HWHW(from,to,T1,T1,MRS_fr,MRS_fr,sigmaTimes,foreign_vol_r_Values,foreign_vol_r_Values,correl_1);
            localVariance(2) = quantock1f_ck1f.GeneralVariance_HWHW(from,to,to,to,MRS_fr,MRS_fr,sigmaTimes,foreign_vol_r_Values,foreign_vol_r_Values,correl_1);
            
            quantoChange(1) = -1.0*quantock1f_ck1f.GeneralVariance_BSHW(from,to,T1,MRS_fr,sigmaTimes,foreign_vol_r_Values,fxValues,corr_frx_Values);
            quantoChange(2) = -1.0*quantock1f_ck1f.GeneralVariance_BSHW(from,to,to,MRS_fr,sigmaTimes,foreign_vol_r_Values,fxValues,corr_frx_Values);
            
            measureChange(1) = quantock1f_ck1f.GeneralVariance_HWHW(from,to,T1,T2,MRS_fr,MRS_dr,sigmaTimes,foreign_vol_r_Values,domestic_vol_r_Values,corr_drfr_Values);
            measureChange(2) = quantock1f_ck1f.GeneralVariance_HWHW(from,to,to,T2,MRS_fr,MRS_dr,sigmaTimes,foreign_vol_r_Values,domestic_vol_r_Values,corr_drfr_Values);
            
            value = localVariance(1) - localVariance(2);
            value = value - 2.0 * -1.0 * (quantoChange(1) - quantoChange(2));
            value = value - 2.0 * 1.0 * (measureChange(1) - measureChange(2));    
            
            out = value;
            
        end
        
        function beta = Beta(quantock1f_ck1f, t, T)
            a_r = quantock1f_ck1f.foreign_meanR_r;
            beta =[];
            
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0 - exp(-a_r * (T - t))) / a_r;
            else
                beta(1) = (T - t);
            end
            
        end
        
        function beta = BetaDate(quantock1f_ck1f, valueDate, tDate, TDate)
            t = tDate.DateDiff(valueDate) / 365.0;
            T = TDate.DateDiff(valueDate) / 365.0;
            
            a_r = quantock1f_ck1f.foreign_meanR_r;
            beta =[];
            
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0 - exp(-a_r * (T - t))) / a_r;
            else
                beta(1) = (T-t);
            end
            
        end
        
        function c_nN = C_nN(quantock1f_ck1f, tn, tN, tau, pnN, ynN)
            p = @quantock1f_ck1f.DF;
            c_nN = [0.0];
            
            for i = 1:round((tN - tn) / tau)
                tp = tn + i * tau;
                beta = quantock1f_ck1f.Beta(tn, tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1) * ynN;
            c_nN(1) = c_nN(1) + beta(1) * df;
            c_nN(1) = c_nN(1) / pnN;
            
        end
        
        function c_nN = C_nNDate(quantock1f_ck1f,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate) / 365.0;
            tN = tNDate.DateDiff(valueDate) / 365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @quantock1f_ck1f.DF;
            c_nN=[0.0];
            beta = [0.0];
            df = 0.0;
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = quantock1f_ck1f.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(1) = c_nN(1) +deltaT * beta(1) * df;
                prevTime = tp;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
        end
        
        function g_nN = G_nN(quantock1f_ck1f,tn,tN,tau,pnN)
            p = @quantock1f_ck1f.DF;
            g_nN=[0.0];
            
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = quantock1f_ck1f.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function g_nN = G_nNDate(quantock1f_ck1f,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @quantock1f_ck1f.DF;
            g_nN=[0.0];
            
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = quantock1f_ck1f.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(1) = g_nN(1) + deltaT * beta(1) * df;
                prevTime = tp;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function h_nTerm = H_nTerm(quantock1f_ck1f,tn,tTerm)
           h_nTerm =[];
           beta = quantock1f_ck1f.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function h_nTerm = H_nTermDate(quantock1f_ck1f,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = quantock1f_ck1f.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function i_Term = I_Term(quantock1f_ck1f,from,to,tTerm)

            fMRS = quantock1f_ck1f.foreign_meanR_r;
            sigmaTimes = quantock1f_ck1f.foreign_vol_r.tenor;
            foreign_vol_r_Values = quantock1f_ck1f.foreign_vol_r.quote;
            
            dMRS = quantock1f_ck1f.domestic_meanR_r;
            domestic_vol_r_Values = quantock1f_ck1f.domestic_vol_r.quote;
            
            corr_drfr_Values = quantock1f_ck1f.corr_drfr.quote;
                        
            % QuantoCK1F_LGM2F case
%             variance_drfr = quantock1f_ck1f.GeneralVariance_HWLGM_type2(from,to,to,tTerm,MRS1,sigmaTimes,domestic_dH_r_Values,domestic_Alpha_r_Values,...
%                                                          foreign_vol_r_Values,corr_drfr_Values);
            
            % QuantoCK1F_CK1F case
            variance_drfr = quantock1f_ck1f.GeneralVariance_HWHW_type2(from, to, to, tTerm, dMRS, fMRS, sigmaTimes,...
                                                            domestic_vol_r_Values, foreign_vol_r_Values, corr_drfr_Values);
            
            % we add minus 1 so that we don't need additional minus in
            % drift adjust term
            i_Term = -1.0 * (variance_drfr);
           
        end
        
        function i_Tn_tilda = I_Tn_tilda(quantock1f_ck1f,from,to,tn)

            MRS1 = quantock1f_ck1f.foreign_meanR_r;
            sigmaTimes = quantock1f_ck1f.foreign_vol_r.tenor;
            foreign_vol_r_Values = quantock1f_ck1f.foreign_vol_r.quote;
            corr_frfr_Values = ones(length(sigmaTimes),1);
            
            variance_frfr = quantock1f_ck1f.GeneralVariance_HWHW_type2(from,to,tn,tn,MRS1,MRS1,sigmaTimes,...
                                                                        foreign_vol_r_Values,foreign_vol_r_Values,corr_frfr_Values);
           % we don't need minus 1
           i_Tn_tilda = 1.0 * variance_frfr;
           
        end
        
%         function i_Tn_tilda = I_Tn_tilda_old(quantock1f_ck1f,from,to,tn)
% 
%             sigmaTimes = quantock1f_ck1f.foreign_vol_r.tenor;
%             sigmaSize =  length(sigmaTimes);
%             
%             for i=1:sigmaSize
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:sigmaSize
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%            
%            MRS2 = quantock1f_ck1f.foreign_meanR_r;
%            
%            newExpScale = zeros(2,1);
%            lastExpScale = zeros(2,1);
%            variances = zeros(2,1);
%             
%            lastU  = from;
%            
%            lastExpScale(1) = exp(MRS2*lastU);
%            lastExpScale(2) = exp((MRS2 + MRS2)*lastU);
%            
%            sigmaValues1 = quantock1f_ck1f.foreign_vol_r.quote;
%            sigmaValues2 = quantock1f_ck1f.foreign_vol_r.quote;
%            corr12 = 1.0;
% 
%            
%            for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end;
%                 if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
%                 if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
%                 
%                 newExpScale(1) = exp(MRS2*U);
%                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(1)-lastExpScale(1));
%                 lastExpScale(1) = newExpScale(1);
%                 
%                 newExpScale(2) = exp((MRS2+MRS2)*U);
%                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/(MRS2+MRS2)) * (newExpScale(2)-lastExpScale(2));
%                 lastExpScale(2) = newExpScale(2);
%                 
%                 lastU = U;
%            end
%            
%            value = exp(-1.0*MRS2*tn)*(1.0/ MRS2)*variances(1);
%            value = value + -1.0*exp(-1.0*(MRS2 + MRS2)*tn)*(1.0/ MRS2)*variances(2);
%            % we don't need minus 1
%            i_Tn_tilda = 1.0 * value;
%            
%         end
        
        
        
        function j_x_tilda = J_x_tilda(quantock1f_ck1f,from,to,tn)

            MRS1 = quantock1f_ck1f.foreign_meanR_r;
            sigmaTimes = quantock1f_ck1f.foreign_vol_r.tenor;
            foreign_vol_r_Values = quantock1f_ck1f.foreign_vol_r.quote;
            fxBlackVol_Values = quantock1f_ck1f.fxBlackVol.quote;
            corr_frx_Values = quantock1f_ck1f.corr_frx.quote;
            
            variance_frx = quantock1f_ck1f.GeneralVariance_BSHW_type2(from,to,tn,MRS1,sigmaTimes,...
                                                                        foreign_vol_r_Values,fxBlackVol_Values,corr_frx_Values);
           
           % we need minus 1
           j_x_tilda = -1.0 * variance_frx;
           
        end
        
%         function j_x_tilda = J_x_tilda_old(quantock1f_ck1f,from,to,tn)
% 
%             sigmaTimes = quantock1f_ck1f.foreign_vol_r.tenor;
%             sigmaSize =  length(sigmaTimes);
%             
%             for i=1:sigmaSize
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:sigmaSize
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%            
%             
%            newExpScale = zeros(1,1);
%            lastExpScale = zeros(1,1);
%            
%            lastU  = from;
%            MRS2 = quantock1f_ck1f.foreign_meanR_r;
%            lastExpScale(1) = exp(MRS2*lastU);
%            variances = 0.0;
%            
%            sigmaValues1 = quantock1f_ck1f.fxBlackVol.quote;
%            sigmaValues2 = quantock1f_ck1f.foreign_vol_r.quote;
%            corrValues12 = quantock1f_ck1f.corr_frx.quote;
%            
%            for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end;
%                 if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
%                 if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
%                 if (i< sigmaSize) corr12 = corrValues12(i);else corr12 = corrValues12(sigmaSize);end;
%                 
%                 newExpScale(1) = exp(MRS2*U);
%                 variances = variances + corr12 * vol1 * vol2 * (1.0/MRS2) * (newExpScale(1)-lastExpScale(1));
%                 lastExpScale(1) = newExpScale(1);
%                 
%                 lastU = U;
%            end
%            
%            % we need minus 1
%            j_x_tilda = -1.0 * exp(-1.0*MRS2*tn) * (1.0/MRS2) * variances;
%            
%         end
        
        function out = discountFactor(quantock1f_ck1f, startTime, endTime, numMatTime, modelStatesT) 
           zcT = quantock1f_ck1f.DF(endTime/365.0);
           
           %numOfPath
           stateSize = size(modelStatesT, 2);
           discountFactorOut = zeros(1, stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
               for i = 1:stateSize
                   discountFactorOut(i) = zcT;
               end
                
               out = discountFactorOut;
               return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
               out = ones(1,stateSize);
               return;
           end
           
           zct = quantock1f_ck1f.DF(startTime / 365.0);
           beta = quantock1f_ck1f.Beta(startTime/365.0, endTime/365.0);
           zcDriftTerm = quantock1f_ck1f.fwdZcVarianceForeign(0, startTime/365.0, endTime/365.0, numMatTime/365.0);
           
           % quantock1f_ck1f is one factor model
           for i = 1:stateSize
%                discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
               discountFactorOut(i) = zcT / zct * exp(-0.5 * zcDriftTerm - beta(1) * modelStatesT(1, i));
           end
           
           out = discountFactorOut;
           
        end
        
%         function out = discountPayoff(quantock1f_ck1f,eventTime,numMatTime ...
%                 ,modelStatesT,cashflow)
%             
%             dfNumMat = quantock1f_ck1f.GetChildModel('d').DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             domesticModelStatesT = modelStatesT(1,:);
%             discountT = quantock1f_ck1f.GetChildModel('d').discountFactor(eventTime, numMatTime,numMatTime,domesticModelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        function variances = LocalVariance(quantock1f_ck1f,from,to)

            sig(1) = quantock1f_ck1f.foreign_vol_r;
            meanR(1) = quantock1f_ck1f.foreign_meanR_r;
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
            tiny = zeros(1,1);
            lastExpScale = zeros(1,1);
            newExpScale  = zeros(1,1);
            
            for i=1:1
                for j=1:1
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
            end
  
           lastU  = from;
           
           variances = zeros(1,1);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                
                correl =ones(1,1);
                for j=1:1
                    for k=1:1
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale =newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
           for j=1:1
                for k=1:1
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
           end
           
        end
        
        function out = LocalDrift(quantock1f_ck1f,from,to)
            drift = eye(2,2);
            drift(1,1) = exp(-1.0*quantock1f_ck1f.domestic_meanR_r*(to-from));
            drift(2,2) = exp(-1.0*quantock1f_ck1f.foreign_meanR_r*(to-from));
            out = drift;
        end
        
        function variances = LocalCovariance(quantock1f_ck1f,from,to)

            sig(1) = quantock1f_ck1f.domestic_vol_r;
            sig(2) = quantock1f_ck1f.foreign_vol_r;
            
            corr_drfr = quantock1f_ck1f.corr_drfr;
            
            meanR = zeros(2, 1);
            
            meanR(1) = quantock1f_ck1f.domestic_meanR_r;
            meanR(2) = quantock1f_ck1f.foreign_meanR_r;
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
            tiny = zeros(2, 2);
            lastExpScale = zeros(2, 2);
            newExpScale  = zeros(2, 2);
            
            for i = 1:2
                for j = 1:2
                    if(abs(meanR(i) + meanR(j)) < 1.0E-12) tiny(i, j) = 1; else tiny(i, j) = 0; end
                    lastExpScale(i, j) =  exp((meanR(i) + meanR(j)) * from);
                end
            end
            
  
           lastU  = from;
           
           variances = zeros(2, 2);
           correl = ones(2, 2);
           
           for i = startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end
                if (i< alphaSize); vol(1) = sig(1).quote(i); else; vol(1) = sig(1).quote(alphaSize); end
                if (i< alphaSize); vol(2) = sig(2).quote(i); else; vol(2) = sig(2).quote(alphaSize); end
                if (i< alphaSize); correl(1, 2) = corr_drfr.quote(i); else; correl(1, 2) = corr_drfr.quote(alphaSize); end
                
                correl(2, 1) = correl(1, 2);
                
                for j = 1:2
                    for k = 1:2
                        newExpScale(j, k) = exp((meanR(j) + meanR(k)) * U);
                        if(tiny(j, k) ~= 1)
                            scale = newExpScale(j, k) - lastExpScale(j, k);
                        else
                            scale = U - lastU;
                        end
                        
                        variances(j, k) = variances(j, k) + vol(j) * vol(k) * correl(j, k) * scale;
                        lastExpScale(j, k) = newExpScale(j, k);
                    end
                end
                lastU = U;
           end
           
           for j = 1:2
                for k = 1:2
                    if(tiny(j, k) ~= 1)
                        variances(j, k) = variances(j, k) / (meanR(j) + meanR(k)) * exp(-1.0 * (meanR(j) + meanR(k)) * to);
                    end
                end
            end
           
        end
        
        function out = LocalVariance_H2_Alpha2(quantock1f_ck1f,from,to,sigmaTimes,dH1Values, dH2Values, sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1.quote = dH1Values;
           dH1.tenor = sigmaTimes;
           
           dH2.quote = dH2Values;
           dH2.tenor = sigmaTimes;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH1Value = dH1.quote(i);else dH1Value = dH1.quote(sigmaSize);end;
                if (i< sigmaSize) dH2Value = dH2.quote(i);else dH2Value = dH2.quote(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + quantock1f_ck1f.H(lastU,dH1)*quantock1f_ck1f.H(lastU,dH2)*correl*vol1*vol2*(U - lastU);
                variances = variances + dH1Value*quantock1f_ck1f.H(lastU,dH2)*correl*vol1*vol2*0.5*(U - lastU)*(U - lastU);
                variances = variances + quantock1f_ck1f.H(lastU,dH1)*dH2Value*correl*vol1*vol2*0.5*(U - lastU)*(U - lastU);
                variances = variances + dH1Value*dH2Value*correl*vol1*vol2*1.0/3.0*(U - lastU)*(U - lastU)*(U - lastU);
                
                
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = LocalVariance_H_Alpha2(quantock1f_ck1f,from,to,sigmaTimes,dH1Values,sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           dH1.quote = dH1Values;
           dH1.tenor = sigmaTimes;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) dH = dH1.quote(i);else dH = dH1.quote(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + quantock1f_ck1f.H(lastU,dH1) * correl * vol1 * vol2 * (U - lastU);
                variances = variances + 0.5 * dH * correl * vol1 * vol2 * (U - lastU) * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = LocalVariance_Alpha2(quantock1f_ck1f,from,to,sigmaTimes,sigmaValues1,sigmaValues2,corrValues)

            sigmaSize =  length(sigmaTimes);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) vol1 = sigmaValues1(i);else vol1 = sigmaValues1(sigmaSize);end;
                if (i< sigmaSize) vol2 = sigmaValues2(i);else vol2 = sigmaValues2(sigmaSize);end;
                if (i< sigmaSize) correl = corrValues(i);else correl = corrValues(sigmaSize);end;
                
                variances = variances + correl* vol1 * vol2 * (U - lastU);
                lastU = U;
           end
           
           out = variances;
           
        end
        
        function out = GeneralVariance_HWHW(quantock1f_ck1f,from,to,T1,T2,MRS1,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            % Variance in [from,to] of 1/a*(1-e^{-a(T1-s)})*1/b*(1-e^{-b(T2-s)})sigma1(s)sigma2(s)
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(4,1);
            lastExpScale = zeros(4,1);
            variances = zeros(4,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS1*lastU);
            lastExpScale(3) = exp(MRS2*lastU);
            lastExpScale(4) = exp((MRS1 + MRS2)*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end;
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end;
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end;
                
                variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS1*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                newExpScale(3) = exp(MRS2*U);
                variances(3) = variances(3) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(3)-lastExpScale(3));
                lastExpScale(3) = newExpScale(3);
                
                newExpScale(4) = exp((MRS1+MRS2)*U);
                variances(4) = variances(4) + corr12 * sig1 * sig2 * (1.0/(MRS1+MRS2)) * (newExpScale(4)-lastExpScale(4));
                lastExpScale(4) = newExpScale(4);
                
                lastU = U;
                
            end
            
            value = (1.0/ MRS1)*(1.0/MRS2)*variances(1);
            value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/ MRS1)*(1.0/MRS2)*variances(2);
            value = value + -1.0*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(3);
            value = value +  1.0*exp(-1.0*MRS1*T1)*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(4);

            out  = value;
        end
        
        function out = GeneralVariance_HWHW_type2(quantock1f_ck1f,from,to,T1,T2,MRS1,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            % Variance in [a,b] of 1/a*(1-e^{-a(T1-s)})*e^{-b(T2-s)}*sigma1(s)sigma2(s)
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(4,1);
            lastExpScale = zeros(4,1);
            variances = zeros(4,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS1*lastU);
            lastExpScale(3) = exp(MRS2*lastU);
            lastExpScale(4) = exp((MRS1 + MRS2)*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end;
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end;
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end;
                
%                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
%                 newExpScale(2) = exp(MRS1*U);
%                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
%                 lastExpScale(2) = newExpScale(2);
                
                newExpScale(3) = exp(MRS2*U);
                variances(3) = variances(3) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(3)-lastExpScale(3));
                lastExpScale(3) = newExpScale(3);
                
                newExpScale(4) = exp((MRS1+MRS2)*U);
                variances(4) = variances(4) + corr12 * sig1 * sig2 * (1.0/(MRS1+MRS2)) * (newExpScale(4)-lastExpScale(4));
                lastExpScale(4) = newExpScale(4);
                
                lastU = U;
                
            end
            
%             value = (1.0/ MRS1)*(1.0/MRS2)*variances(1);
%             value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/ MRS1)*(1.0/MRS2)*variances(2);
            value =  +1.0*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*variances(3);
            value = value +  -1.0*exp(-1.0*MRS1*T1)*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*variances(4);

            out  = value;
        end
        
%         function out = GeneralVariance_HWLGM(quantock1f_ck1f,from,to,T1,T2,MRS1,sigmaTimes,dH1Values,sigmaValue1,sigmaValue2,corrValue)
%             % Variance in [from,to] of 1/a*(1-e^{-a(T1-s)})*(H(T2)-H(s))*sigma1(s)sigma2(s)
%             
%             dH1.quote = dH1Values;
%             dH1.tenor = sigmaTimes;
%             
%             sigmaSize = length(sigmaValue1);
%             
%             for i=1:length(sigmaTimes)
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:length(sigmaTimes)
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%             
%             
%             newExpScale = zeros(4,1);
%             lastExpScale = zeros(4,1);
%             variances = zeros(4,1);
%             
%             lastU  = from;
%             lastExpScale(2) = exp(MRS1*lastU);
%             lastExpScale(3) = exp(MRS1*lastU);
%             lastExpScale(4) = exp(MRS1*lastU);
%             
%             for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end
%                 if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
%                 if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
%                 if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
%                 if (i< sigmaSize) dH = dH1Values(i);else dH = dH1Values(sigmaSize);end
%                 
%                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
%                 
%                 newExpScale(2) = exp(MRS1*U);
%                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
%                 lastExpScale(2) = newExpScale(2);
%                 
%                 newExpScale(3) = exp(MRS1*U);
%                 variances(3) = variances(3) + quantock1f_ck1f.H(lastU,dH1) * corr12 * sig1 * sig2 * (U - lastU);
%                 variances(3) = variances(3) + dH * corr12 * sig1 * sig2 * 0.5 * (U - lastU) * (U - lastU);
%                 lastExpScale(3) = newExpScale(3);
%                 
%                 newExpScale(4) = exp(MRS1*U);
%                 variances(4) = variances(4) + quantock1f_ck1f.H(lastU,dH1) * corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
%                 variances(4) = variances(4) + dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (U*newExpScale(4)-lastU*lastExpScale(4));
%                 variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
%                 variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * lastU * (newExpScale(4)-lastExpScale(4));
%                 lastExpScale(4) = newExpScale(4);
%                 
%                 lastU = U;
%                 
%             end
%             
%             value = quantock1f_ck1f.H(T2,dH1)*(1.0/MRS1)*variances(1);
%             value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/MRS1)*quantock1f_ck1f.H(T2,dH1)*variances(2);
%             value = value + -1.0*(1.0/MRS1)*variances(3);
%             value = value +  1.0*exp(-1.0*MRS1*T1)*(1.0/MRS1)*variances(4);
% 
%             out  = value;
%         end
        
%         function out = GeneralVariance_HWLGM_type2(quantock1f_ck1f,from,to,T1,T2,MRS1,sigmaTimes,dH1Values,sigmaValue1,sigmaValue2,corrValue)
%         % Variance in [a,b] of e^{-a(T1-s)}*(H(T2)-H(s))*sigma1(s)sigma2(s)
%             
%             dH1.quote = dH1Values;
%             dH1.tenor = sigmaTimes;
%             
%             sigmaSize = length(sigmaValue1);
%             
%             for i=1:length(sigmaTimes)
%                 if from<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             startIdx = i;
%             
%             for i=1:length(sigmaTimes)
%                 if to<= sigmaTimes(i)
%                     break;
%                 end
%             end
%             endIdx = i;
%             
%             
%             newExpScale = zeros(4,1);
%             lastExpScale = zeros(4,1);
%             variances = zeros(4,1);
%             
%             lastU  = from;
%             lastExpScale(2) = exp(MRS1*lastU);
%             lastExpScale(3) = exp(MRS1*lastU);
%             lastExpScale(4) = exp(MRS1*lastU);
%             
%             for i=startIdx:endIdx
%                 if (i< endIdx) U = sigmaTimes(i);else U = to;end
%                 if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
%                 if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
%                 if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
%                 if (i< sigmaSize) dH = dH1Values(i);else dH = dH1Values(sigmaSize);end
%                 
% %                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
%                 
%                 newExpScale(2) = exp(MRS1*U);
%                 variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
%                 lastExpScale(2) = newExpScale(2);
%                 
% %                 newExpScale(3) = exp(MRS1*U);
% %                 variances(3) = variances(3) + quantock1f_ck1f.H(lastU,dH1) * corr12 * sig1 * sig2 * (U - lastU);
% %                 variances(3) = variances(3) + dH * corr12 * sig1 * sig2 * 0.5 * (U - lastU) * (U - lastU);
% %                 lastExpScale(3) = newExpScale(3);
%                 
%                 newExpScale(4) = exp(MRS1*U);
%                 variances(4) = variances(4) + quantock1f_ck1f.H(lastU,dH1) * corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
%                 variances(4) = variances(4) + dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (U*newExpScale(4)-lastU*lastExpScale(4));
%                 variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * (1.0/MRS1) * (newExpScale(4)-lastExpScale(4));
%                 variances(4) = variances(4) -1.0*dH * corr12 * sig1 * sig2 * (1.0/MRS1) * lastU * (newExpScale(4)-lastExpScale(4));
%                 lastExpScale(4) = newExpScale(4);
%                 
%                 lastU = U;
%                 
%             end
%             
% %             value = quantock1f_ck1f.H(T2,dH1)*(1.0/MRS1)*variances(1);
% %             value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/MRS1)*quantock1f_ck1f.H(T2,dH1)*variances(2);
%             value = +1.0*exp(-1.0*MRS1*T1)*quantock1f_ck1f.H(T2,dH1)*variances(2);
%             
% %             value = value + -1.0*(1.0/MRS1)*variances(3);
%             value = value +  -1.0*exp(-1.0*MRS1*T1)*variances(4);
% 
%             out  = value;
%         end
        
        function out = GeneralVariance_BSHW(quantock1f_ck1f,from,to,T2,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            % Variance in [from,to] of 1/a*(1-e^{-a(T1-s)})sigma1(s)sigma2(s)
            
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(2,1);
            lastExpScale = zeros(2,1);
            variances = zeros(2,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS2*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
                
                variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS2*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                lastU = U;
                
            end
            
            value = (1.0/MRS2)*variances(1);
            value = value + -1.0*exp(-1.0*MRS2*T2)*(1.0/MRS2)*variances(2);
            
            out  = value;
        end
        
        function out = GeneralVariance_BSHW_type2(quantock1f_ck1f,from,to,T2,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            % Variance in [a,b] of (e^{-a(T1-s)})sigma1(s)sigma2(s)
            
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(2,1);
            lastExpScale = zeros(2,1);
            variances = zeros(2,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS2*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end
                
%                 variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS2*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                lastU = U;
                
            end
            
%             value = (1.0/MRS2)*variances(1);
            value = +1.0*exp(-1.0*MRS2*T2)*variances(2);
            
            out  = value;
        end
        
        function out = CMSDigitalArgDate(quantock1f_ck1f,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           to = toDate.DateDiff(valueDate)/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           tauIn = tau;
           if abs(tenor1- 0.25) < 1e-8
                tauIn = tenor1;
           end
           
           p_nN1 = quantock1f_ck1f.PV01Date(valueDate,toDate,tN1Date,tauIn);
           
           df_Tn1 = quantock1f_ck1f.DF(to);
           df_TN1 = quantock1f_ck1f.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = quantock1f_ck1f.C_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1,y_nN1);
           g_nN1 = quantock1f_ck1f.G_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1);
%            h_nTerm1 = quantock1f_ck1f.H_nTermDate(valueDate,toDate,tpDate);
           % h_nTerm1 is replaced with i_Term & i_Tn_tilda as in the
           % following discussion
           
           h_nTerm1(1) = 0.0;
           
           varCovar = quantock1f_ck1f.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           tpTime = tpDate.DateDiff(valueDate);
           % additional drift correction term from foreign Term measure USD to KRW domestic Term 
           i_Term = I_Term(quantock1f_ck1f,0,to,tpTime/365.0);
           i_Tn_tilda = I_Tn_tilda(quantock1f_ck1f,0,to,to);
           j_x_tilda = J_x_tilda(quantock1f_ck1f,0,to,to);
           
%            additionalDrift = 0;
           additionalDrift = c_nN1(1) *(i_Term + i_Tn_tilda + j_x_tilda);
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.fwdCMS = y_nN1;
           out.mu = y_nN1 + driftTerm + additionalDrift;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
        end
        
        function out = computeBSCK1FCK1FVariance(quantock1f_ck1f,a,b,T,corr_drfr,corr_drx,corr_frx,...
                                  fxBlackVol, domestic_meanR_r, domestic_vol_r,...
                                  foreign_meanR_r, foreign_vol_r)
          % synchronize model parameters time-steps start 
            corr_drfr_Tenor = corr_drfr.tenor;
            corr_drfr_Quote = corr_drfr.quote;
            
            corr_drx_Tenor = corr_drx.tenor;
            corr_drx_Quote = corr_drx.quote;
            
            corr_frx_Tenor = corr_frx.tenor;
            corr_frx_Quote = corr_frx.quote;
            
            fxBlackVolTenor = fxBlackVol.tenor;
            fxBlackVolQuote = fxBlackVol.quote;

            domestic_vol_r_Tenor = domestic_vol_r.tenor;
            domestic_vol_r_Quote = domestic_vol_r.quote;
            
            foreign_vol_r_Tenor = foreign_vol_r.tenor;
            foreign_vol_r_Quote = foreign_vol_r.quote;

            sigmaTimes = unique(union([corr_drfr_Tenor(:);...
                         corr_drx_Tenor(:);corr_frx_Tenor(:);...
                         domestic_vol_r_Tenor(:);...
                         foreign_vol_r_Tenor(:)],fxBlackVolTenor));

            sigmaTimes = sort(sigmaTimes,'ascend');

            corr_drfr_Values = zeros(length(sigmaTimes),1);
            corr_drx_Values = zeros(length(sigmaTimes),1);
            corr_frx_Values = zeros(length(sigmaTimes),1);

            domestic_vol_r_Values = zeros(length(sigmaTimes), 1);
            foreign_vol_r_Values = zeros(length(sigmaTimes), 1);
            
            fxBlackVolValues = zeros(length(sigmaTimes),1);

            for k=1:length(sigmaTimes)
                corr_drfr_Values(k) = H_interpolation(corr_drfr_Tenor, corr_drfr_Quote,sigmaTimes(k),0);
                corr_drx_Values(k) = H_interpolation(corr_drx_Tenor, corr_drx_Quote,sigmaTimes(k),0);
                corr_frx_Values(k) = H_interpolation(corr_frx_Tenor, corr_frx_Quote,sigmaTimes(k),0);
                
                domestic_vol_r_Values(k) = H_interpolation(domestic_vol_r_Tenor, domestic_vol_r_Quote, sigmaTimes(k), 0);
                foreign_vol_r_Values(k) = H_interpolation(foreign_vol_r_Tenor, foreign_vol_r_Quote,sigmaTimes(k), 0);
                                
                fxBlackVolValues(k) = H_interpolation(fxBlackVolTenor, fxBlackVolQuote,sigmaTimes(k), 0);
            end
            
            % synchronize model parameters time-steps end
            % dummy correlation
            correl_1 = ones(length(sigmaTimes),1);
             
            var_x_x = quantock1f_ck1f.LocalVariance_Alpha2(a,b,sigmaTimes,fxBlackVolValues,fxBlackVolValues,correl_1);
            
            var_fr_fr = quantock1f_ck1f.GeneralVariance_HWHW(a,b,T,T,foreign_meanR_r,foreign_meanR_r,sigmaTimes, ...
                                        foreign_vol_r_Values,foreign_vol_r_Values,correl_1);
                                                     
            var_dr_dr = quantock1f_ck1f.GeneralVariance_HWHW(a,b,T,T,domestic_meanR_r, domestic_meanR_r, sigmaTimes,...
                                                         domestic_vol_r_Values, domestic_vol_r_Values, correl_1);
                                                     
           % 1st= sign in techNote , 2nd = HW bond vol? -1,1                               
            covar_fr_x =  1.0*-1.0*quantock1f_ck1f.GeneralVariance_BSHW(a,b,T,foreign_meanR_r,sigmaTimes, ...
                                            foreign_vol_r_Values,fxBlackVolValues,corr_frx_Values);
                                        
            covar_dr_x = -1.0*-1.0*quantock1f_ck1f.GeneralVariance_BSHW(a,b,T,domestic_meanR_r,sigmaTimes, ...
                                            domestic_vol_r_Values,domestic_Alpha_r_Values,fxBlackVolValues,corr_drx_Values);
                                        
            covar_dr_fr =  -1.0*1.0*quantock1f_ck1f.GeneralVariance_HWHW(a,b,T,T,domestic_meanR_r, foreign_meanR_r,sigmaTimes,...
                                            domestic_meanR_r,foreign_vol_r_Values, corr_drfr_Values);
                                                         
            value = var_x_x + var_fr_fr + var_dr_dr + 2.0 * (covar_fr_x + covar_dr_x + covar_dr_fr);                                         
            
            out = value;                                         
                                                     
            
        end
        
        function out = FindFXBlackVol(quantock1f_ck1f,index,params)
            
            expiry = params.fxImpVol.tenor(index);
            impVol = params.fxImpVol.quote(index);
            blackVariance = impVol * impVol * expiry;
            optParams.blackVariance = blackVariance;
            
            optParams.params = params;
            optParams.index = index;
            modelParamSize = 1;
            tvar = zeros(modelParamSize, 1);
            lb = zeros(modelParamSize, 1);
            ub = zeros(modelParamSize, 1);
            
            tvar(1) = params.fxImpVol.quote(index);
            lb(1)= 0.0001;
            ub(1)= 10;
            
            %options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
            options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
            numOfEquation =1;
            x =  zeros(numOfEquation,1);
            [ret, popt, info, covar] = levmar('TargetFunctionBSCK1FCK1FFVariance',tvar,x, 500, options,'unc',quantock1f_ck1f,optParams);
            
            out.fxBlackVol_index = popt(1);
            out.re = TargetFunctionBSCK1FCK1FVariance(popt,quantock1f_ck1f,optParams);
            out.marketVar = blackVariance;
            out.modelVar = (1.0+out.re)*blackVariance;
%             params.fxBlackVol.quote(index) = popt(1);

        end
        
        function out = CalibrateBSCK1FCK1FVol(quantock1f_ck1f, paramsIn)
            expirySize = length(paramsIn.fxImpVol.tenor);
            out.fxBlackVol.quote = zeros(expirySize,1);
            out.fxBlackVol.tenor = paramsIn.fxImpVol.tenor;
            out.re =  zeros(expirySize,1);
            out.modelVar = zeros(expirySize,1);
            out.marketVar = zeros(expirySize,1);
            out.rmseTotal = 0;

            for i = 1:expirySize
                index = i;
                BootstrapOut = quantock1f_ck1f.FindFXBlackVol(index,paramsIn);
                out.fxBlackVol.quote(i) = BootstrapOut.fxBlackVol_index;
                paramsIn.fxBlackVol.quote(i) = BootstrapOut.fxBlackVol_index;
                out.re(i) = BootstrapOut.re;
                out.marketVar(i) = BootstrapOut.marketVar;
                out.modelVar(i) = BootstrapOut.modelVar;
                out.rmseTotal = out.rmseTotal + out.re(i) * out.re(i);
            end
            
            out.rmseTotal = out.rmseTotal / expirySize;
            out.rmseTotal = sqrt(out.rmseTotal); 
                
        end
        
        
    end
    
end



In [ ]:
classdef QuantoCheyette < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        numOfFactor;
        numOfFactor_d;
        numOfFactor_f;
        
        inv_H_tau_x;
        
        corr_fwd = [];
        rho = [];
        rho_d = [];
        rho_f = [];
        rho_x = [];
        
        fxBlckVol;
        domesticModel;
        foreignModel;
        
    end
    
    methods
        function ch = QuantoCheyette(params)
            if nargin > 0
                
%                 domestic Cheyette part
                ch.domesticModel = params('domesticModel');
                ch.foreignModel  = params('foreignModel');
                
                ch.zeroCurve = ch.domesticModel.zeroCurve;
                ch.numOfFactor_d = ch.domesticModel.modelParams('numOfFactor');
                ch.numOfFactor_f = ch.foreignModel.modelParams('numOfFactor');
                ch.numOfFactor = ch.numOfFactor_d + ch.numOfFactor_f + 1;
                sizeN = ch.numOfFactor;
                
                ch.inv_H_tau_x = [1.0];
                
                ch.corr_fwd = zeros(sizeN,sizeN);
                ch.corr_fwd =  params('corr_fwd');
                
                ch.rho = zeros(sizeN,sizeN); 
                [PC,V]=eig(ch.corr_fwd);
                PCW= PC*sqrt(V);
                for i=1:sizeN
                    for j=1:sizeN
                        ch.rho(i,j)=PCW(i,j);
                    end
                end
                
                for i=1:ch.numOfFactor_d
                    for j=1:sizeN
                        ch.rho_d(i,j)=ch.rho(i,j);
                    end
                end
                
                for i=1:ch.numOfFactor_f
                    for j=1:sizeN
                        ch.rho_f(i,j)=ch.rho(ch.numOfFactor_d+i,j);
                    end
                end
                
                i=1;
                for j=1:sizeN
                    ch.rho_x(i,j)=ch.rho(sizeN,j);
                end
                
                fxBlackVol.tenor= ch.domesticModel.lambda{1,1}.tenor;
                fxBlackVol.quote = zeros(length(fxBlackVol.tenor),1);
                
            end
        end
        
        function out = GetChildModel(ch,type)
            if type == 'd'
                out = ch.domesticModel;
            elseif type == 'f'
                out = ch.foreignModel;
            end
        
        end
        
        function out= Fwd(ch,t,tnr,type)
            if type =='d' || type == 'f'
                out = ch.GetChildModel(type).Fwd(t,tnr);
            elseif type == 'x'
                out = 1.0;
            end
        end
        
        function out= GetEta(ch,tIndex,type)
            plambda = {};
             
            if type =='d' || type =='f'
                child =  ch.GetChildModel(type);
                sizeR = child.numOfFactor;
                
                plambda =cell(sizeR,1);
                for i=1:sizeR
                    plambda{i,1}= child.lambda{i,1};
                end
                
                tau = child.tau;
                if type == 'd'
                    rho = ch.rho_d;
                else
                    rho = ch.rho_f;
                end
                inv_H_tau = child.inv_H_tau;
            
            elseif type =='x'
                sizeR = 1;
                tau = [1.0]; % dummy
                rho = ch.rho_x;
                inv_H_tau = ch.inv_H_tau_x;
            end
            
            sizeC   = ch.numOfFactor;
            sigma_f = [];
            sigma_f = zeros(sizeR,sizeC);

            for i=1:sizeR
                for j=1:sizeC
                    sigma_f(i,j)=plambda{i,1}.quote(tIndex)*ch.Fwd(0,tau(i),type)*rho(i,j);
                end
            end
            
           eta =[];
           eta = zeros(sizeR,sizeC);
           eta = inv_H_tau*sigma_f;
           out = eta; 
        end
        
        function variances = LocalVariance(ch,from,to,type1,type2,t1,t2)
           model1  = ch.GetChildModel(type1);
           eta1 = ch.GetEta(1, type1);
           
           model2 = ch.GetChildModel(type2);
           eta2 = ch.GetEta(1,type2);
           
          %% To find the index 
           
           alphaSize = length(model1.lambda{1,1}.tenor);
            
           for i=1:alphaSize
               if from<= model1.lambda{1,1}.tenor(i);
                   break;
               end
           end
            
           startIdx = i;
            
%             endIdx = size(sig(1).tenor,1); 
           for i=1:alphaSize
               if to<= model1.lambda{1,1}.tenor(i);
                   break;
               end
           end
           endIdx = i;
            
           lastU  = from;
           
           sizeN1 = model1.numOfFactor;
           sizeN2 = model2.numOfFactor;
           
           tiny = zeros(sizeN1,sizeN2);
           lastExpScale = zeros(sizeN1,sizeN2);
           newExpScale  = zeros(sizeN1,sizeN2);
           scale = 0.0;
           
           meanR1 = zeros(sizeN1,1);
           meanR2 = zeros(sizeN2,1);
           
           for i=1:sizeN1
                meanR1(i) = model1.kappa(i);
           end
           
           for i=1:sizeN2
                meanR2(i) = model2.kappa(i);
           end
           
           for i=1:sizeN1
                for j=1:sizeN2
                    if(abs(meanR1(i)+meanR2(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end;
                    lastExpScale(i,j) =  exp((meanR1(i)+meanR1(j))*from);
                end
           end
            
           variances = zeros(sizeN1,sizeN2);
           tindex = 1;
           for i=startIdx:endIdx
                if (i< endIdx) U = model1.lambda{1,1}.tenor(i);else U = to;end;
                if (i< alphaSize) tindex = i;else tindex = alphaSize;end;
            %% converting forward rate volatlity into short rate volatlitity
                eta1 = ch.GetEta(tindex, type1);
                eta2 = ch.GetEta(tindex, type2);
                eta12 = eta1*eta2';
            %%
                for j=1:sizeN1
                    for k=1:sizeN2
                        newExpScale(j,k) =  exp((meanR1(j)+meanR2(k))*U);
                        if(tiny(j,k)~=1)
                            scale = newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + eta12(j,k) * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
           for j=1:sizeN1
                for k=1:sizeN2
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR1(j)+meanR2(k))*exp(-1.0*(meanR1(j)*t1 + meanR2(k)*t2));
                    end;
                end
            end
            
        end
        
         function variances = LocalVariance2(ch,from,to,type1,type2,t1,t2)
           model1  = ch.GetChildModel(type1);
           eta1 = ch.GetEta(1, type1);
           
           model2 = ch.GetChildModel(type2);
           eta2 = ch.GetEta(1,type2);
           
          %% To find the index 
           
           alphaSize = length(model1.lambda{1,1}.tenor);
            
           for i=1:alphaSize
               if from<= model1.lambda{1,1}.tenor(i);
                   break;
               end
           end
            
           startIdx = i;
            
%             endIdx = size(sig(1).tenor,1); 
           for i=1:alphaSize
               if to<= model1.lambda{1,1}.tenor(i);
                   break;
               end
           end
           endIdx = i;
            
           lastU  = from;
           
           sizeN1 = model1.numOfFactor;
           sizeN2 = model2.numOfFactor;
           
           tiny1 = zeros(sizeN1,sizeN2);
           tiny2 = zeros(sizeN1,sizeN2);
           
           lastExpScale1 = zeros(sizeN1,sizeN2);
           newExpScale1  = zeros(sizeN1,sizeN2);
           
           lastExpScale2 = zeros(sizeN1,sizeN2);
           newExpScale2  = zeros(sizeN1,sizeN2);
           
           scale1 = 0.0;
           scale2 = 0.0;
           
           meanR1 = zeros(sizeN1,1);
           meanR2 = zeros(sizeN2,1);
           
           for i=1:sizeN1
                meanR1(i) = model1.kappa(i);
           end
           
           for i=1:sizeN2
                meanR2(i) = model2.kappa(i);
           end
           
           for i=1:sizeN1
                for j=1:sizeN2
                    if(abs(meanR1(i)) < 1.0E-12) tiny1(i,j) = 1; else tiny1(i,j) = 0; end;
                    if(abs(meanR1(i)+meanR2(j)) < 1.0E-12) tiny2(i,j) = 1; else tiny2(i,j) = 0; end;
                    lastExpScale1(i,j) =  exp((meanR1(i))*from);
                    lastExpScale2(i,j) =  exp((meanR1(i)+meanR2(j))*from);
                end
           end
            
           variances1 = zeros(sizeN1,sizeN2);
           variances2 = zeros(sizeN1,sizeN2);
           
           tindex = 1;
           for i=startIdx:endIdx
                if (i< endIdx) U = model1.lambda{1,1}.tenor(i);else U = to;end;
                if (i< alphaSize) tindex = i;else tindex = alphaSize;end;
            %% converting forward rate volatlity into short rate volatlitity
                eta1 = ch.GetEta(tindex, type1);
                eta2 = ch.GetEta(tindex, type2);
                eta12 = eta1*eta2';
            %%
                for j=1:sizeN1
                    for k=1:sizeN2
                        newExpScale1(j,k) =  exp((meanR1(j))*U);
                        newExpScale2(j,k) =  exp((meanR1(j)+meanR2(k))*U);
                        if(tiny1(j,k)~=1)
                            scale1 = newExpScale1(j,k)-lastExpScale1(j,k);
                        else
                            scale1 = U - lastU;
                        end
                        
                        if(tiny2(j,k)~=1)
                            scale2 = newExpScale2(j,k)-lastExpScale2(j,k);
                        else
                            scale2 = U - lastU;
                        end
                        
                        variances1(j,k) = variances1(j,k) + eta12(j,k) * scale1;
                        variances2(j,k) = variances2(j,k) + eta12(j,k) * scale2;
                        lastExpScale1(j,k) = newExpScale1(j,k);
                        lastExpScale2(j,k) = newExpScale2(j,k);
                    end
                end
                lastU = U;
           end
           
           for j=1:sizeN1
                for k=1:sizeN2
                    if(tiny1(j,k)~= 1)
                        variances1(j,k) = variances1(j,k)/(meanR1(j))*exp(-1.0*(meanR1(j)*t1));
                    end;
                    
                    if(tiny2(j,k)~= 1)
                        variances2(j,k) = variances2(j,k)/(meanR1(j)+meanR2(k))*exp(-1.0*(meanR1(j)*t1 + meanR2(k)*t2));
                    end;
                    
                end
           end
           
           for j=1:sizeN1
                for k=1:sizeN2
                    variances(j,k) = (variances1(j,k) - variances2(j,k))/(meanR2(k));
                end
                
           end
           
         end

        function beta = Beta(ch,t,T,type)
             beta = ch.GetChildModel(type).Beta(t,T);
        end
        
        function c_nN = C_nN(ch,tn,tN,tau,pnN,ynN,type)
            c_nN = ch.GetChildModel(type).C_nN(tn,tN,tau,pnN,ynN);
        end
        
        function g_nN = G_nN(ch,tn,tN,tau,pnN,type)
            g_nN = ch.GetChildModel(type).G_nN(tn,tN,tau,pnN);
        end
        
        function h_nTerm = H_nTerm(ch,tn,tTerm)
           h_nTerm = ch.GetChildModel(type).H_nTerm(tn,tTerm);
        end
        
        function out = QuantoCMSSpreadDigitalArg(ch,to,tp,tenor1,tenor2,type1,type2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           % we assume that CMS1 is domestic and CMS2 is foreign
           
           %CMS 1  
           model1 = ch.GetChildModel(type1);
           p_nN1 = model1.PV01(to,to+tenor1,tau);
           df_Tn1 = model1.DF(to);
           df_TN1 = model1.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = model1.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = model1.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = model1.H_nTerm(to,tp);
           
           sizeN1 = model1.numOfFactor;
           
           varCovar_dd = ch.LocalVariance(0,to,'d','d',to,to);
           driftTerm1=0.0;
           var1 = 0.0;
           
           for i=1:sizeN1
               for j=1:sizeN1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar_dd(i,j);
                    var1 = var1 + c_nN1(i)* c_nN1(j) * varCovar_dd(i,j);
               end
           end
           
           %CMS2
           model2 = ch.GetChildModel(type2);
           p_nN2 = model2.PV01(to,to+tenor2,tau);
           df_Tn2 = model2.DF(to);
           df_TN2 = model2.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = model2.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = model2.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = model2.H_nTerm(to,tp);
           sizeN2 = model2.numOfFactor;
           
           varCovar_ff = ch.LocalVariance(0,to,'f','f',to,to);
           
           i_Term = ch.LocalVariance2(0,to,'f','d',to,tp);
           i_n    = ch.LocalVariance2(0,to,'f','f',to,to);
           j_x = 0.0;
           
           driftTerm2=0.0;
           for i=1:sizeN2
               for j=1:sizeN2
                    driftTerm2 = driftTerm2 + c_nN2(i) * (g_nN2(j) * varCovar_ff(i,j) + i_n(i,j));
               end
           end
           
           for i=1:sizeN2
               for j=1:sizeN1
                    driftTerm2 = driftTerm2 + c_nN2(i) * (- 1.0*i_Term(i,j));
               end
           end
           
           var2 = 0.0;
           for i=1:sizeN2
               for j=1:sizeN2
                    var2 = var2 + c_nN2(i) * c_nN2(j) * varCovar_ff(i,j);
               end
           end
           
           varCovar_df = ch.LocalVariance(0,to,'d','f',to,to);
           
           covar = 0.0;
           for i=1:sizeN1
               for j=1:sizeN2
                    covar = covar + c_nN1(i) * c_nN2(j) * varCovar_df(i,j);
               end
           end
           
           
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           spreadVariance =  var1 + var2 - 2.0*covar;
           out.sigma = sqrt(spreadVariance);
        end
        
        function out = QuantoCMSSpreadDigital(ch,to,tp,tenor1,tenor2,type1,type2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ch.QuantoCMSSpreadDigitalArg(to,tp,tenor1,tenor2,type1,type2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = QuantoCMSSpreadDigitalRange(ch,to,tp,tenor1,tenor2,type1,type2,tau,lower,upper)
            lower_value = ch.QuantoCMSSpreadDigital(to,tp,tenor1,tenor2,type1,type2,tau,lower);
            upper_value = ch.QuantoCMSSpreadDigital(to,tp,tenor1,tenor2,type1,type2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function atmSwaption = ATMSwaption(ch,tx,tnr,type)
            atmSwaption =  GetChildModel(type).ATMSwaption(tx,tnr);
        end
   
    end
    
end



In [ ]:
classdef PricingColumnInfoSimple < H_RootObject
    % pricingColumnInfo process, add or assing payoff for the given column
    % cashflow
    % payoff, cashflow, cashflow_npv
    % payoffStates 
    
    properties(SetAccess = public)
        pathSize
        cashflowSize
        npv
        payoff
        cashflow
        cashflow_npv
        
    end
    
    methods
        function pricerInfo = PricingColumnInfoSimple(pathSize,cashflowSize)
            if nargin > 0
                pricerInfo.pathSize = pathSize;
                pricerInfo.cashflowSize = cashflowSize;
                pricerInfo.npv = 0.0;
                pricerInfo.payoff = zeros(pathSize,1);
                pricerInfo.cashflow = zeros(pathSize,cashflowSize);
                pricerInfo.cashflow_npv = zeros(pathSize,cashflowSize);
            end
        end
        
        function out = ProcessAssignPayoff(pricerInfo,model,eventTime,payTime, ...
                                     aCashflow,currentNodeIdx)
            
            % if payTime is positive, then cashflow_npv will be
            % overwritten to payoff
            % EOD handling is done at cashflow level and not in the process
            % payoff level
            
            pricerInfo.cashflow(:,currentNodeIdx) = aCashflow(:);
            if eventTime <= 0 
                
                if payTime >= 0
                    df_p = model.discountCurve.DF(payTime/365.0);
                    aCashflow_npv = aCashflow*df_p;
                    pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                    pricerInfo.payoff = aCashflow_npv;
                else
                    aCashflow_npv = aCashflow;
                    pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                end
                
            else
                df_p = model.discountCurve.DF(payTime/365.0);
                aCashflow_npv = aCashflow * df_p;
                pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                pricerInfo.payoff = aCashflow_npv;
                
            end
            
            out = aCashflow_npv;
        end
        
        function out = ProcessAddPayoff(pricerInfo,model,eventTime,payTime, ...
                                     aCashflow,currentNodeIdx)
            
            % if payTime is positive or zero, then cashflow_npv will be
            % added to payoff
            % EOD handling is done at cashflow level and not in the process
            % payoff level
            
            pricerInfo.cashflow(:,currentNodeIdx) = aCashflow;
            
            if eventTime <= 0
                
                if payTime >= 0
                    df_p = model.discountCurve.DF(payTime/365.0);
                    aCashflow_npv = aCashflow*df_p;
                    pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                     pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
                else
                    aCashflow_npv = aCashflow;
                    pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                end
                
            else
                df_p = model.discountCurve.DF(payTime/365.0);
                aCashflow_npv = aCashflow * df_p;
                pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
                
            end
            
            out = aCashflow_npv;
        end
        
    end
    
end



In [ ]:
classdef PricingColumnInfoGenericCashflow < H_RootObject
    % pricingColumnInfo add or assign payoff for the given column's cashflow
    % for a given pricing column we can have multiple cashflows
    % we choose dictioinary structured for aggregatinng cashflows
    % key: cashflow names value : corrensponding cashflows
    %
    % attribute: pathSize,cashflowNameSize,cashflowNames
    % npv, payoff, cashflowMap, cashflow_npvMap
    %
    % method : ProcessAssignPayoff, ProcessAddPayoff
    
    properties(SetAccess = public)
        pathSize
        cashflowNameSize
        cashflowNames
        npv
        payoff
        cashflowMap
        cashflowMap_npv
        
%         cashflow
%         cashflow_npv
        
    end
    
    methods
        function pricerInfo = PricingColumnInfoGenericCashflow(cashflowInfo)
            if nargin > 0
                
                pricerInfo.pathSize          = cashflowInfo.pathSize;
                pricerInfo.cashflowNames     = cashflowInfo.cashflowNames;
                
                pricerInfo.cashflowNameSize = length(cashflowInfo.cashflowNames);
                
                pricerInfo.cashflowMap     = containers.Map;
                pricerInfo.cashflowMap_npv = containers.Map;
                
                for i=1: pricerInfo.cashflowNameSize
                    cashflowSize = cashflowInfo.cashflowSizeMap(pricerInfo.cashflowNames{i});
                    pricerInfo.cashflowMap(pricerInfo.cashflowNames{i})     = zeros(cashflowSize,pricerInfo.pathSize);
                    pricerInfo.cashflowMap_npv(pricerInfo.cashflowNames{i}) = zeros(cashflowSize,pricerInfo.pathSize);
                end
                
                pricerInfo.npv = 0.0;
                pricerInfo.payoff = zeros(1,pricerInfo.pathSize);
%                 pricerInfo.cashflow = zeros(cashflowSize,pathSize);
%                 pricerInfo.cashflow_npv = zeros(cashflowSize,pathSize);
            end
        end
        
       %% Assign Event happen at CallEventDate
       %% we omit cashflowName and currentNodeIdx
       %% simply overwrite aCashflow_npv into payoff
        function out = ProcessAssignPayoff(pricerInfo,model,eventTime,payTime,numeraire,modelStateAt, ...
                                     aCashflow)
            
            % if payTime is positive, then cashflow_npv will be
            % overwritten to payoff
            % EOD handling is done at cashflow level and not in the process
            % payoff level
            
            try
                
                if eventTime <= 0 
                
                    if payTime >= 0
                        df_p = model.DFRisky(payTime/365.0);
                        aCashflow_npv = aCashflow*df_p;
                        % assign
                        pricerInfo.payoff = aCashflow_npv;
                    else
                        aCashflow_npv = aCashflow;
                    end

                else
                    df_ep = model.discountFactorRisky(eventTime, payTime,numeraire.numMatTime,modelStateAt);
                    aCashflow1 = aCashflow .* df_ep;
                    aCashflow_npv = model.discountPayoffRisky(eventTime,numeraire.numMatTime ...
                       ,modelStateAt,aCashflow1);
                    pricerInfo.payoff = aCashflow_npv;

                end

                out = aCashflow_npv;
                
            catch exception
                throw(exception)
            end
            
        end
        
        function out = ProcessAddPayoff(pricerInfo,model,eventTime,payTime,numeraire,modelStateAt, ...
                                     aCashflow,cashflowName,currentNodeIdx)
            
            % if payTime is positive or zero, then cashflow_npv will be
            % added to payoff
            % EOD handling is done at cashflow level and not in the process
            % payoff level
            
%             pricerInfo.cashflow(currentNodeIdx,:) = aCashflow;
            
            try
                 %referencing specific cashflow
                cashflowSingle     = pricerInfo.cashflowMap(cashflowName);
                cashflowSingle_npv = pricerInfo.cashflowMap_npv(cashflowName);
            
                % Add
                cashflowSingle(currentNodeIdx,:) = cashflowSingle(currentNodeIdx,:) + aCashflow(1,:);
                pricerInfo.cashflowMap(cashflowName) = cashflowSingle;
            
                if eventTime <= 0

                    if payTime >= 0
%                         df_p = model.DFRisky(payTime/365.0);
                        df_p = model.zeroCurve.DF(payTime/365.0);
                        
                        
                        aCashflow_npv = aCashflow*df_p;
                        cashflowSingle_npv(currentNodeIdx,:) = cashflowSingle_npv(currentNodeIdx,:) + aCashflow_npv(1,:);
                        pricerInfo.cashflowMap_npv(cashflowName) = cashflowSingle_npv;
                        pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
                    else
                        aCashflow_npv = aCashflow;
                        cashflowSingle_npv(currentNodeIdx,:) = cashflowSingle_npv(currentNodeIdx,:) + aCashflow_npv(1,:);
                        pricerInfo.cashflowMap_npv(cashflowName) = cashflowSingle_npv;
                        
                    end

                else
   
                    df_ep = model.discountFactorRisky(eventTime, payTime,numeraire.numMatTime,modelStateAt);
                    aCashflow1 = aCashflow .* df_ep;
                    aCashflow_npv = model.discountPayoffRisky(eventTime,numeraire.numMatTime ...
                       ,modelStateAt,aCashflow1);
                    
                    cashflowSingle_npv(currentNodeIdx,:) = cashflowSingle_npv(currentNodeIdx,:) + aCashflow_npv(1,:);
                    pricerInfo.cashflowMap_npv(cashflowName) = cashflowSingle_npv;
                    pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;

                end

                out = aCashflow_npv;

            catch exception
                throw(exception)
            end
            
        end
        
%         function out = ProcessAddPayoff(pricerInfo,model,eventTime,payTime,numeraire,modelStateAt, ...
%                                      aCashflow,currentNodeIdx)
%             
%             % if payTime is positive or zero, then cashflow_npv will be
%             % added to payoff
%             % EOD handling is done at cashflow level and not in the process
%             % payoff level
%             
%             pricerInfo.cashflow(currentNodeIdx,:) = aCashflow;
%             
%             if eventTime <= 0
%                 
%                 if payTime >= 0
%                     df_p = model.DFRisky(payTime/365.0);
%                     aCashflow_npv = aCashflow*df_p;
%                     pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv(:);
%                      pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
%                 else
%                     aCashflow_npv = aCashflow;
%                     pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv(:);
%                 end
%                 
%             else
% %                 df_p = model.discountCurve.DF(payTime/365.0);
% %                 aCashflow_npv = aCashflow * df_p;
% 
%                 df_ep = model.discountFactorRisky(eventTime, payTime,numeraire.numMatTime,modelStateAt);
%                 aCashflow1 = aCashflow .* df_ep;
%                 aCashflow_npv = model.discountPayoffRisky(eventTime,numeraire.numMatTime ...
%                    ,modelStateAt,aCashflow1);
%                 pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv(:);
%                 pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
%                 
%             end
%             
%             out = aCashflow_npv;
%         end
        
    end
    
end



In [ ]:
classdef PricingColumnInfoGeneric < H_RootObject
    % pricingColumnInfo add or assign payoff for the given column's cashflow
    % payoff, cashflow, cashflow_npv
    % payoffStates 
    
    properties(SetAccess = public)
        pathSize
        cashflowSize
        npv
        payoff
        cashflow
        cashflow_npv
        
    end
    
    methods
        function pricerInfo = PricingColumnInfoGeneric(cashflowSize,pathSize)
            if nargin > 0
                pricerInfo.pathSize = pathSize;
                pricerInfo.cashflowSize = cashflowSize;
                pricerInfo.npv = 0.0;
                pricerInfo.payoff = zeros(1,pathSize);
                pricerInfo.cashflow = zeros(cashflowSize,pathSize);
                pricerInfo.cashflow_npv = zeros(cashflowSize,pathSize);
            end
        end
        
        function out = ProcessAssignPayoff(pricerInfo,model,eventTime,payTime,numeraire,modelStateAt, ...
                                     aCashflow,currentNodeIdx)
            
            % if payTime is positive, then cashflow_npv will be
            % overwritten to payoff
            % EOD handling is done at cashflow level and not in the process
            % payoff level
            
            pricerInfo.cashflow(currentNodeIdx,:) = aCashflow(:);
            if eventTime <= 0 
                
                if payTime >= 0
                    df_p = model.DFRisky(payTime/365.0);
                    aCashflow_npv = aCashflow*df_p;
                    pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv(:);
                    % assign
                    pricerInfo.payoff = aCashflow_npv;
                else
                    aCashflow_npv = aCashflow;
                    pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv(:);
                end
                
            else
%                 df_p = model.discountCurve.DF(payTime/365.0);
%                 aCashflow_npv = aCashflow * df_p;
                df_ep = model.discountFactorRisky(eventTime, payTime,numeraire.numMatTime,modelStateAt);
                
                aCashflow1 = aCashflow .* df_ep;
                aCashflow_npv = model.discountPayoffRisky(eventTime,numeraire.numMatTime ...
                   ,modelStateAt,aCashflow1);
                pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv(:);
                pricerInfo.payoff = aCashflow_npv;
                
            end
            
            out = aCashflow_npv;
        end
        
        function out = ProcessAddPayoff(pricerInfo,model,eventTime,payTime,numeraire,modelStateAt, ...
                                     aCashflow,currentNodeIdx)
            
            % if payTime is positive or zero, then cashflow_npv will be
            % added to payoff
            % EOD handling is done at cashflow level and not in the process
            % payoff level
            
            pricerInfo.cashflow(currentNodeIdx,:) = aCashflow;
            
            if eventTime <= 0
                
                if payTime >= 0
                    df_p = model.DFRisky(payTime/365.0);
                    aCashflow_npv = aCashflow*df_p;
                    pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv(:);
                     pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
                else
                    aCashflow_npv = aCashflow;
                    pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv(:);
                end
                
            else
%                 df_p = model.discountCurve.DF(payTime/365.0);
%                 aCashflow_npv = aCashflow * df_p;

                df_ep = model.discountFactorRisky(eventTime, payTime,numeraire.numMatTime,modelStateAt);
                aCashflow1 = aCashflow .* df_ep;
                aCashflow_npv = model.discountPayoffRisky(eventTime,numeraire.numMatTime ...
                   ,modelStateAt,aCashflow1);
                pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv(:);
                pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
                
            end
            
            out = aCashflow_npv;
        end
        
    end
    
end



In [ ]:
classdef PricingColumnInfo < H_RootObject
    % payoff, cashflow, cashflow_npv
    % payoffStates 
    
    properties(SetAccess = public)
        pathSize
        cashflowSize
        npv
        payoff
        cashflow_e
        cashflow_npv_e
        cashflow
        cashflow_npv
        
    end
    
    methods
        function pricerInfo = PricingColumnInfo(cashflowSize, pathSize)
            if nargin > 0
                pricerInfo.pathSize = pathSize;
                pricerInfo.cashflowSize = cashflowSize;
                pricerInfo.npv = 0.0;
                pricerInfo.payoff = zeros(1,pathSize);
                pricerInfo.cashflow_e = zeros(cashflowSize,1);
                pricerInfo.cashflow_npv_e = zeros(cashflowSize,1);
                pricerInfo.cashflow = zeros(cashflowSize,pathSize);
                pricerInfo.cashflow_npv = zeros(cashflowSize,pathSize);
            end
        end
        
        function out = ProcessAssignPayoff(pricerInfo,model,eventTime,payTime,numeraire, ...
                                     modelStateAt,aCashflow,currentNodeIdx)
            if eventTime <= 0 
                df_p = 1.0;
%                 if payTime >= 0
%                     df_p = model.DFRisky(payTime/365.0);
%                 else
%                     df_p = 1.0;
%                 end
                
                % cashflow is fixed need not to be discounted
                % in the terminal measure. but simply added to the column.
                pricerInfo.cashflow(currentNodeIdx,:) = aCashflow;
                aCashflow_npv = df_p*aCashflow;
                pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv;
                
                if payTime >= 0
                    pricerInfo.payoff = aCashflow_npv;
                end
                
                pricerInfo.cashflow_e(currentNodeIdx) = mean(aCashflow);
                pricerInfo.cashflow_npv_e(currentNodeIdx) = mean(aCashflow_npv);
                out = aCashflow_npv;
            else
                pricerInfo.cashflow(currentNodeIdx,:) = aCashflow;
                aCashflow_npv = model.discountPayoffRisky(eventTime,numeraire.numMatTime ...
                   ,modelStateAt,aCashflow);
                pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv;
                pricerInfo.payoff = aCashflow_npv;

                pricerInfo.cashflow_e(currentNodeIdx) = mean(aCashflow);
                pricerInfo.cashflow_npv_e(currentNodeIdx) = mean(aCashflow_npv);
                out = aCashflow_npv;
            end
        end
        
        function out = ProcessAddPayoff(pricerInfo,model,eventTime,payTime,numeraire, ...
                                     modelStateAt,aCashflow,currentNodeIdx)
            
            if eventTime <= 0 
                % cashflow is fixed need not to be discounted
                % in the terminal measure. but simply added to the column.
                
                pricerInfo.cashflow(currentNodeIdx,:) = pricerInfo.cashflow(currentNodeIdx,:) + aCashflow;
                aCashflow_npv = aCashflow;
                pricerInfo.cashflow_npv(currentNodeIdx,:) = pricerInfo.cashflow_npv(currentNodeIdx,:) + aCashflow_npv;

                if payTime >= 0
                    pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
                end
                
                pricerInfo.cashflow_e(currentNodeIdx) = pricerInfo.cashflow_e(currentNodeIdx) + mean(aCashflow);
                pricerInfo.cashflow_npv_e(currentNodeIdx) = pricerInfo.cashflow_npv_e(currentNodeIdx) + mean(aCashflow_npv);
                out = aCashflow_npv;
            else
                pricerInfo.cashflow(currentNodeIdx,:) = pricerInfo.cashflow(currentNodeIdx,:) + aCashflow;
                aCashflow_npv = model.discountPayoffRisky(currentTime,numeraire.numMatTime ...
                   ,modelStateAt,aCashflow);
                pricerInfo.cashflow_npv(currentNodeIdx,:) = pricerInfo.cashflow_npv(currentNodeIdx,:) + aCashflow_npv;
                pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;

                pricerInfo.cashflow_e(currentNodeIdx) = pricerInfo.cashflow_e(currentNodeIdx) + mean(aCashflow);
                pricerInfo.cashflow_npv_e(currentNodeIdx) = pricerInfo.cashflow_npv_e(currentNodeIdx) + mean(aCashflow_npv);
                out = aCashflow_npv;
            end
        end
        
    end
    
end



In [ ]:
classdef PricerInfoSimple < H_RootObject
    % payoff, cashflow, cashflow_npv
    % payoffStates 
    
    properties(SetAccess = public)
        pathSize
        cashflowSize
        npv
        payoff
        cashflow
        cashflow_npv
        
    end
    
    methods
        function pricerInfo = PricerInfoSimple(pathSize,cashflowSize)
            if nargin > 0
                pricerInfo.pathSize = pathSize;
                pricerInfo.cashflowSize = cashflowSize;
                pricerInfo.npv = 0.0;
                pricerInfo.payoff = zeros(pathSize,1);
                pricerInfo.cashflow = zeros(pathSize,cashflowSize);
                pricerInfo.cashflow_npv = zeros(pathSize,cashflowSize);
            end
        end
        
        function out = ProcessAssignPayoff(pricerInfo,model,eventTime,payTime, ...
                                     aCashflow,currentNodeIdx)
            
            % if payTime is positive, then cashflow_npv will be
            % overwritten to payoff
            % EOD handling is done at cashflow level and not in the process
            % payoff level
            
            pricerInfo.cashflow(:,currentNodeIdx) = aCashflow(:);
            if eventTime <= 0 
                
                if payTime >= 0
                    df_p = model.discountCurve.DF(payTime/365.0);
                    aCashflow_npv = aCashflow*df_p;
                    pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                    pricerInfo.payoff = aCashflow_npv;
                else
                    aCashflow_npv = aCashflow;
                    pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                end
                
            else
                df_p = model.discountCurve.DF(payTime/365.0);
                aCashflow_npv = aCashflow * df_p;
                pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                pricerInfo.payoff = aCashflow_npv;
                
            end
            
            out = aCashflow_npv;
        end
        
        function out = ProcessAddPayoff(pricerInfo,model,eventTime,payTime, ...
                                     aCashflow,currentNodeIdx)
            
            % if payTime is positive or zero, then cashflow_npv will be
            % added to payoff
            % EOD handling is done at cashflow level and not in the process
            % payoff level
            
            pricerInfo.cashflow(:,currentNodeIdx) = aCashflow;
            
            if eventTime <= 0
                
                if payTime >= 0
                    df_p = model.discountCurve.DF(payTime/365.0);
                    aCashflow_npv = aCashflow*df_p;
                    pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                     pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
                else
                    aCashflow_npv = aCashflow;
                    pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                end
                
            else
                df_p = model.discountCurve.DF(payTime/365.0);
                aCashflow_npv = aCashflow * df_p;
                pricerInfo.cashflow_npv(:,currentNodeIdx) = aCashflow_npv(:);
                pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
                
            end
            
            out = aCashflow_npv;
        end
        
    end
    
end



In [ ]:
classdef PricerInfo < H_RootObject
    % payoff, cashflow, cashflow_npv
    % payoffStates 
    
    properties(SetAccess = public)
        pathSize
        cashflowSize
        npv
        payoff
        cashflow_e
        cashflow_npv_e
        cashflow
        cashflow_npv
        
    end
    
    methods
        function pricerInfo = PricerInfo(cashflowSize, pathSize)
            if nargin > 0
                pricerInfo.pathSize = pathSize;
                pricerInfo.cashflowSize = cashflowSize;
                pricerInfo.npv = 0.0;
                pricerInfo.payoff = zeros(1,pathSize);
                pricerInfo.cashflow_e = zeros(cashflowSize,1);
                pricerInfo.cashflow_npv_e = zeros(cashflowSize,1);
                pricerInfo.cashflow = zeros(cashflowSize,pathSize);
                pricerInfo.cashflow_npv = zeros(cashflowSize,pathSize);
            end
        end
        
        function out = ProcessPayoff(pricerInfo,model,currentTime,numeraire, ...
                                     modelStateAt,aCashflow,currentNodeIdx)
            if currentTime <= 0 
                % cashflow is fixed need not to be discounted
                % in the terminal measure. but simply added to the column.
                pricerInfo.cashflow(currentNodeIdx,:) = aCashflow;
                aCashflow_npv = aCashflow;
                pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv;
                pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;

                pricerInfo.cashflow_e(currentNodeIdx) = mean(aCashflow);
                pricerInfo.cashflow_npv_e(currentNodeIdx) = mean(aCashflow_npv);
                out = aCashflow_npv;
            else
                pricerInfo.cashflow(currentNodeIdx,:) = aCashflow;
                aCashflow_npv = model.discountPayoffRisky(currentTime,numeraire.numMatTime ...
                   ,modelStateAt,aCashflow);
                pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv;
                pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;

                pricerInfo.cashflow_e(currentNodeIdx) = mean(aCashflow);
                pricerInfo.cashflow_npv_e(currentNodeIdx) = mean(aCashflow_npv);
                out = aCashflow_npv;
            end
        end
        
        function out = ProcessAssignPayoff(pricerInfo,model,currentTime,numeraire, ...
                                     modelStateAt,aCashflow,currentNodeIdx)
            if currentTime <= 0 
                % cashflow is fixed need not to be discounted
                % in the terminal measure. but simply added to the column.
                pricerInfo.cashflow(currentNodeIdx,:) = aCashflow;
                aCashflow_npv = aCashflow;
                pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv;
%                 pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
                pricerInfo.payoff = aCashflow_npv;

                pricerInfo.cashflow_e(currentNodeIdx) = mean(aCashflow);
                pricerInfo.cashflow_npv_e(currentNodeIdx) = mean(aCashflow_npv);
                out = aCashflow_npv;
            else
                pricerInfo.cashflow(currentNodeIdx,:) = aCashflow;
                aCashflow_npv = model.discountPayoffRisky(currentTime,numeraire.numMatTime ...
                   ,modelStateAt,aCashflow);
                pricerInfo.cashflow_npv(currentNodeIdx,:) = aCashflow_npv;
%                 pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;
                pricerInfo.payoff = aCashflow_npv;

                pricerInfo.cashflow_e(currentNodeIdx) = mean(aCashflow);
                pricerInfo.cashflow_npv_e(currentNodeIdx) = mean(aCashflow_npv);
                out = aCashflow_npv;
            end
        end
        
        function out = ProcessAddPayoff(pricerInfo,model,currentTime,numeraire, ...
                                     modelStateAt,aCashflow,currentNodeIdx)
            
            if currentTime <= 0 
                % cashflow is fixed need not to be discounted
                % in the terminal measure. but simply added to the column.
                
                pricerInfo.cashflow(currentNodeIdx,:) = pricerInfo.cashflow(currentNodeIdx,:) + aCashflow;
                aCashflow_npv = aCashflow;
                pricerInfo.cashflow_npv(currentNodeIdx,:) = pricerInfo.cashflow_npv(currentNodeIdx,:) + aCashflow_npv;
                pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;

                pricerInfo.cashflow_e(currentNodeIdx) = pricerInfo.cashflow_e(currentNodeIdx) + mean(aCashflow);
                pricerInfo.cashflow_npv_e(currentNodeIdx) = pricerInfo.cashflow_npv_e(currentNodeIdx) + mean(aCashflow_npv);
                out = aCashflow_npv;
            else
                pricerInfo.cashflow(currentNodeIdx,:) = pricerInfo.cashflow(currentNodeIdx,:) + aCashflow;
                aCashflow_npv = model.discountPayoffRisky(currentTime,numeraire.numMatTime ...
                   ,modelStateAt,aCashflow);
                pricerInfo.cashflow_npv(currentNodeIdx,:) = pricerInfo.cashflow_npv(currentNodeIdx,:) + aCashflow_npv;
                pricerInfo.payoff = pricerInfo.payoff + aCashflow_npv;

                pricerInfo.cashflow_e(currentNodeIdx) = pricerInfo.cashflow_e(currentNodeIdx) + mean(aCashflow);
                pricerInfo.cashflow_npv_e(currentNodeIdx) = pricerInfo.cashflow_npv_e(currentNodeIdx) + mean(aCashflow_npv);
                out = aCashflow_npv;
            end
        end
        
    end
    
end



In [ ]:
classdef Pricer < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
    end
    
    methods
        function pricer = Pricer(security,model,numMethod)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
        end
        
        function out = computePrice(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.schedule;
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = 2;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);

                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;

                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                    ,modelStateT,payoffStateA.cashflow);
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
                    ,modelStateT,1.0*ones(1,pathSize));
                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DF(currentTime/365.0);
    %             
    %             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
    %             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;

                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);

                    %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                    while endM1TimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endM1TimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endM1TimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    simpleYN = false;
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);

                    for j=numOfTimeSteps:-1:1
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        dcfC = currentTime/365.0;
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate operation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                    ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
                    ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DF(currentTime/365.0);

                end

                nonCall.npv = mean(nonCall.payoff);

                out.cashflow= nonCall.cashflow_e;
                out.cashflowNpv = nonCall.cashflow_npv_e;
                out.df_e = nonCall.df_e;
                out.df_c = nonCall.df_c;
                out.daycountFraction = schedule.dayCountFraction;
                out.rangeProb = nonCall.rangeProb_e;
                out.npv = nonCall.npv;
           
                
            
        end
    end
    
end



In [ ]:
function price = priceBachelier ( sign , strike , forward , tte , vol )
    v = vol * sqrt ( tte );
    x = sign * ( forward - strike );
    if ( abs (x) < eps )
        price = v/ sqrt (2* pi );
    	return ;
    end
    
    parity = 0;
    if (x > 0)
        parity = x;
        x = -x;
    end
    
    d = x / v;
%     price = parity + normpdf ( d) * ( x *0.5* sqrt (2* pi )* erfcx (- d/ sqrt (2) ) + v) ;
    price = x*H_ncdf(d) + v*H_ndf(d);
end

In [ ]:
function value = power_law(theta, param)
% Power-law parameterization
eta = param(1);
gamma = param(2);

value = eta./(theta.^(gamma) .* (1+theta).^(1-gamma));
end

In [ ]:
function [ output ] = posdef_scaled_spectral( varargin )
% Scaled Spect r a l Method
%%%%%%%% Default parameters
    a = 1e-13;

    %%%%%%%% Input parameters
    % Input at l e a s t a matrix C
    if(nargin <1)
        disp('Scaled Spec t r a l Method : Not enough input arguments .' );
        disp('Input : C, [ a ] . Die now . .');
        return
    end
    % This i s our target matrix
    C= varargin {1};
    % s t a r t ing pos i t i on f o r eigenvalue value . a >= 0
    if(nargin >= 2 && varargin{2}>=0)
        a = varargin{2};
    end
    %%% Star t Algoritme
    tic
    % Step 1
    [S,L] = eig(C) ;
    % Step 2
    L = diag(max(diag(L),a)) ;

    % Step 3 ( including the r ight mul t ipl i cat ion of S
    T = sqrt(diag(1./(S.^2*diag(L))))*S;
    % Output
    output.method = 'Scaled Spectral';
    % Step 4
    output.C = T*L*T';
    output.D = T*sqrt(L);
    output.E = chol(output.C,'lower');
    output.dist = norm( output.C-C, 'fro') ;
    output.time = toc ;
    output.a = a ;

    output.valid = test_cor(output.C) ;
end



In [ ]:
function z = piecewise_eval(x,breakpoints,funs)
% PIECEWISE_EVAL: evaluates a piecewise function of x
% usage: y = PIECEWISE_EVAL(x,breakpoints,funs)
%
% arguments (input)
%  x    - vector or array of points to evaluate though the function
%  
%  breakpoints - list of n breakpoints, -inf and +inf are implicitly
%         the first and last breakpoints. A function with only two
%         pieces has only one explicit breakpoint. In the event that
%         you want to define a function with breakpoints [a,b,c],
%         and only two functions, but you do not care what happens
%         for x < a or x > b, then you should specify only the
%         breakpoint b. Alternatively, one could specify all 3
%         breaks, and force the function to return NaN above and
%         below those limits.
%
%         x(i) will be identified as falling in interval (j) if
%         break(j) <= x(i) < break(j+1)
%  
%  funs - cell array containing n+1 functions as scalar constants,
%         strings, anonymous functions, inline functions, or any
%         simple matlab function name.
%
%         Note: use .*, ./, .^ where appropriate in the function
%
%         These functions need not be differentiable or even
%         continuous across the breaks.
%
% arguments (output)
%  z    - evaluated function, result is same shape as x
%
% Example usage:
%  For       x < -5, y = 2
%  For -5 <= x < 0,  y = sin(x)
%  For  0 <= x < 2,  y = x.^2
%  For  2 <= x < 3,  y = 6
%  For  3 <= x,      y = inf
%
%  y = piecewise_eval(-10:10,[-5 0 2 3],{2,'sin(x)','x.^2',6,inf})

n=length(breakpoints);
% there must be n+1 funs for n breaks
if length(funs)~=(n+1)
  error 'funs and breakpoints are incompatible in size'
end

if any(diff(breakpoints)<=0)
  error 'Breakpoints must be both distinct and increasing'
end

% ensure the functions are feval-able
for i=1:(n+1)
  if ischar(funs{i})
    % A string. Make it a function
    f=inline(funs{i});
    funs{i} = f;
  elseif isa(funs{i},'function_handle') || isa(funs{i},'inline')
    % a function handle or an inline. do nothing.
  elseif isnumeric(funs{i}) | isnan(funs{i}) | isinf(funs{i})
    % A scalar value was supplied, may be NaN or inf.
    % Make it a function.
    funs{i}=@(x) funs{i};
  else
    % It must be something that feval can handle
    % directly, so leave funs{i} alone.
  end
end

% initialize as nans
z=nan(size(x));

% below the first break
k=(x<breakpoints(1));
z(k)=feval(funs{1},x(k));

left = k;
for i=2:n
  k=(~left) & (x<breakpoints(i));
  if any(k)
    z(k)=feval(funs{i},x(k));
    left = k | left;
  end
end

% over the top
k=(x>=breakpoints(end));
z(k)=feval(funs{end},x(k));




In [ ]:
function [L_n] = phi_LH(x,T,u,n,r_n)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% This f u n c t i o n computes t h e t h e c h a r a c t e r i s t i c f u n c t i o n f o r a
% log􀀀s t r i k e g r i d p o i n t f o l l o w i n g t h e Mu l t i f a c t o r approach
%
%
% INPUT:
% x : Heston models parame t e r s
% T: o p t i o n s t ime t o ma tur i t y ( s c a l a r )
% u : l o g s t r i k e g r i d p o in t
% n : number o f f a c t o r s
% r_n : mu l t i f a c t o r c o e f f i c i e n t
%
%
%
% OUTPUT:
% char_f : c h a r a c t e r i s t i c f u n c t i o n e v a l u a t e d in u
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% rough Heston model parame t e r s
nu = x(1);
lambda= x(2);
rho = x(3);
theta = x(4) ;
V0 = x(5);
alpha = x(6);

% Fr a c t i o n a l Ri c c a t i e q u a t i on
u= 1i*u;
F = @(v)(0.5*(u.*u-u) + (u*rho*nu-lambda ).*v + (0.5*(nu*v).^2));
% Mu l t i f a c t o r model c o e f f i c i e n t s
i =[1:n ] ;
c=((r_n^(1-alpha )-1)*( r_n^(( alpha-1)*(1+n/2) ) ) *...
( r_n.^((1-alpha).*i)))/(gamma(alpha)*gamma(2-alpha));

x=((1-alpha)*(r_n^(2-alpha)-1)*(r_n.^(i-1-n/2)))/...
((2-alpha)*(r_n^(1-alpha)-1));

%% Computing p h i

%I n i t i a l i z a t i o n
N_times=2e2; %number o f p o i n t s on t h e t ime g r i d
dt=T/N_times ;
phi=zeros(n,N_times+1) ;

for k=1:N_times
phi(:,k+1)=((ones(n,1)./(ones(n,1)+dt.*x')).*(phi(:,k)...
 + dt*F(c*phi(:,k))));
end


% Here we propos e t h r e e d i f f e r e n t methods in orde r t o compute
% g_n . One method i s t h e a n a l y t i c a l i n t e g r a t i o n , one i s t h e
% nume r i cal i n t e g r a t i o n wi t h t h e t r a p e z o i d a l r u l e and t h e l a s t
% i s t h e formula o b t a ine d s o l v i n g t h e i n t e g r a l by hand . We
% o b t a i n t h e same r e s u l t in t h e t r e e d i f f e r e n t cas e s , but t h e
% l a s t one i s t h e f a s t e s t .
%
% Not i c e t h a t t h e l a s t method can be used onl y i f t h e t a i s a
% c ons t ant ( our as sumpt ion )
%
%
%
%% Computing g_n e x p l i c i t l y
%
% i n t 1=z e r o s (n , N_times ) ;
% t ime=l i n s p a c e (0 ,T, N_times ) ;
%
% f o r i =1:N_times
% int e g r and=@( s ) exp(􀀀x .  ( t ime ( i )􀀀s ) ) ;
% i n t 1 ( : , i )=i n t e g r a l ( int e g rand , 0 , t ime ( i ) , ’ ArrayValued ’ , t r u e ) ;
% end
%
% g_n=V0 .  ones (1 ,N_times )+t h e t a  lambda ( c  i n t 1 ) ;
%
%
%% Computing g_n wi t h t r a p e z o i d a l r u l e
% W=d t  ones (1 ,N_times ) ;
%
% DT=[1:N_times ]  d t ;
%
% i n t 1=(c  exp(􀀀x ’ DT) ) . W;
% g_n=V0+t h e t a  lambda  i n t 1 ;

%% Computing g_n by hand
DT=[0:N_times ]*dt ;

int1=(c./x)*(1-exp(-x'*DT));
g_n=V0+theta*lambda*int1;

%% Computing t h e c h a r a c t e r i s t i c f u n c t i o n in each t ime g r i d node
int2=F(c*phi).*flip(g_n);

% Tr a p e z o i d a l r u l e we i g h t s
W=dt*ones(1,N_times+1);
W(1) =0.5*dt;
W(end)=0.5*dt;

L_n=exp(int2*W');

end

In [ ]:
function [char_f] = phi(x ,T, u)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% This f u n c t i o n computes t h e c h a r a c t e r i s t i c
% f u n c t i o n f o r a l o g s t r i k e g r i d p o in t
%
%
% INPUT:
% x : Heston models parame t e r s
% T: o p t i o n s t ime t o ma tur i t y ( s c a l a r )
% u : l o g s t r i k e g r i d p o in t
%
%
% OUTPUT:
% char_f : c h a r a c t e r i s t i c f u n c t i o n e v a l u a t e d in u
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

%Number o f s t e p s in t h e Adams scheme
N=1e3 ;
dt = T/N;

% rough Heston model parame t e r s
nu = x ( 1 ) ;
Gamma= x ( 2 ) ;
rho = x ( 3 ) ;
theta = x ( 4 ) ;
V0 = x ( 5 ) ;
alpha = x ( 6 ) ;

% Fr a c t i o n a l Ri c c a t i e q u a t i on
F = @( v ) (0.5*(-u.*u-1i*u) + Gamma.*(1i*u*rho*nu-1).*v +(0.5*(Gamma*nu*v).^2 ) ) ;
% F = @( v ) (0.5*(-u.*u-1i*u) + Gamma.*(1i*u*rho*nu-1).*v +(0.5*(1.0*nu*v).^2 ) ) ;

% S o l u t i o n o f f r a c t i o n a l r i c c a t i e q u a t i on wi t h z e r o i n i t i a l
% c o n d i t i o n
y0 = 0 ;
y = zeros(1,N) ;

% Arrays c o n t a i n i n g t h e we i g h t s o f t h e c o r r e c t o r and p r e d i c t o r
% formulas
a = zeros( 1 ,N) ;
b = zeros( 1 ,N) ;
% Co e f f i c i e n t s computat ion
for k=1:N
b(k) = ( k^alpha - (k-1)^alpha ) ;
a(k) = ( k+1)^( alpha+1) - 2*k^( alpha+1) + (k-1)^( alpha+1) ;
end

F0 = F(y0);

F_y = zeros(1 ,N) ;

% Fi r s t s t e p
p = b(1)*F0 ;
y(1) = ((dt^alpha)*(F(p)-(-alpha)*F0))/gamma( alpha+2) ;
F_y(1) = F(y(1));


% Cor r e c tor􀀀p r e d i c t o r a l g o r i t hm

for j =2:N
p = ( ( dt^alpha )*(b(j)*F0+sum((fliplr(b(1:j-1) ) ) .*F_y( 1 : j -1) ) ) ) /gamma( alpha+1) ;

y(j) = ( dt^alpha*(F(p)+(( j -1)^( alpha+1)-( j-1-alpha )*( j^alpha ) ) *F0+sum( ( fliplr( a(1 : j -1) ) ).*F_y( 1 : j -1) ) ) )/gamma( alpha+2) ;

F_y( j ) = F( y ( j ) ) ;
end

%Computing t h e f i r s t i n t e g r a l o f t h e c h a r a c t e r i s t i c f u n c t i o n
%wi t h t h e t r a p e z o i d a l r u l e

% Tr a p e z o i d a l we i g h t s
w = [0.5 ones(1,N-2) 0.5] ;
int1 = dt*(w*y');


% Adams we i g h t s

W=ones(1,N+1) ;

W( 1 )=(N-1)^(2-alpha )-(N-2+alpha )*N^(1-alpha ) ;

W(2:N)=(N: -1:2) .^(2-alpha )+(N-2:-1:0) .^(2-alpha ) - 2*(N-1:-1:1) .^(2-alpha ) ;

W=((dt)^(1-alpha ) /gamma(3-alpha ) )*W;


% Fr a c t i o n a l i n t e g r a l wi t h t r a p e z o i d a l r u l e
int2 = W(1 :N+1)*[0; y' ] ;

%Ch a r a c t e r i s t i c f u n c t i o n
char_f = exp( theta*Gamma*int1+V0*int2) ;

end

In [ ]:
function vq = pchipFlatExtrap(x,v,xq)

    vq = pchip(x,v,xq);

    [XMax, idxVMax] = max(x);
    [XMin, idxVMin] = min(x);

    idxMax = xq > XMax;
    idxMin = xq < XMin;

    vq(idxMax) = v(idxVMax);
    vq(idxMin) = v(idxVMin);

end



In [ ]:
x = -3:3; 
y = [-1 -1 -1 0 1 1 1]; 
xq1 = -3:.01:3;
p = pchip(x,y,xq1);
s = spline(x,y,xq1);
figure(11)
% m = makima(x,y,xq1);
% plot(x,y,'o',xq1,p,'-',xq1,s,'-.',xq1,m,'--')
plot(x,y,'o',xq1,p,'-',xq1,s,'-.')

legend('Sample Points','pchip','spline','Location','SouthEast')

In [ ]:
function out = PayAdjust(EODFlag,payTime)
%   PayAdjust is a utility function
%   if EODFlag is YES then apply PayAdjust
%   if NO then do not apply. Include all cashflow on valueDate    

    out = 1.0;
    % past cashflow ->  null
    if payTime < 0
        out = 0;
    else
        if strcmp(EODFlag,'YES') && payTime == 0
            out = 0.0;
        else
            out = 1.0;
        end
    end
end



In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
